In [1]:
# Import model yang diperlukan

import pandas as pd
import re
import numpy as np
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from lime import lime_text
from lime.lime_text import LimeTextExplainer
from imblearn.over_sampling import SMOTE
import nltk
import string
import re

C:\Users\mluqm\anaconda3\envs\BDC\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
# Import data
df = pd.read_csv("Dataku.csv", sep = ";")
df.drop(df.columns[[0]], axis = 1, inplace = True)
df.head()

In [ ]:
# Cek NaN

df.isnull().sum().sum()

In [ ]:
#remove user (Cleaning data)

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt    
df['remove_user'] = np.vectorize(remove_pattern)(df['Tweet'], "@[\w]*")

In [ ]:
#Cleaning data

def remove(tweet):
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'^rt[\s]+', '', tweet)
 
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'^https[\s]+', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    tweet = re.sub(r'%', '', tweet)
    
    #remove coma
    tweet = re.sub(r',','',tweet)
    
    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)
    tweet = re.sub(r':', '', tweet)
    return tweet
df['remove_http'] = df['remove_user'].apply(lambda x: remove(x.lower()))

In [ ]:
#import stopword
stopwords_indonesia = nltk.corpus.stopwords.words('indonesian')
 
#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#tokenize
from nltk.tokenize import TweetTokenizer

In [ ]:
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)

In [ ]:
def clean_tweets(tweet):
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
 
    tweets_clean = []    
    for word in tweet_tokens:
        if (word not in stopwords_indonesia and # remove stopwords
              word not in emoticons and # remove emoticons
                word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            stem_word = stemmer.stem(word) # stemming word
            tweets_clean.append(stem_word)
 
    return tweets_clean
df['Tweet_tokenizer'] = df['remove_http'].apply(lambda x: clean_tweets(x))

In [ ]:
#remove punct
def remove_punct(text):
    text  = " ".join([char for char in text if char not in string.punctuation])
    return text
df['Tweet'] = df['Tweet_tokenizer'].apply(lambda x: remove_punct(x))
df.head()

In [ ]:
df.drop(df.columns[[2,3,4]], axis = 1, inplace = True)
print(df)

In [ ]:
# Cek NaN

df.isnull().sum().sum()

In [ ]:
# Mengubah class tipe karakter menjadi Boolean

def convert(polarity):
    if polarity == 'positive':
        return 0
    else:
        return 1

In [ ]:
# Menerapkan Perubahan

df['Class'] = df['Class'].apply(convert)
df.head(6)

In [ ]:
# Menghitung jumlah negatif dan positif

df['Class'].value_counts()

In [ ]:
# Proses TF-IDF Vectorizer, merubah data teks menjadi numerik

vectorizer = TfidfVectorizer()
vectorizer.fit(df['Tweet'])
vector = vectorizer.transform(df['Tweet'])

In [ ]:
df_tf_idf1 = pd.DataFrame(vector.todense(), index = [f'D{i+1}' for i in range (len(df['Tweet']))],
                      columns = [vectorizer.get_feature_names_out()])
df_tf_idf1

In [ ]:
print("ini :", vectorizer.vocabulary_)
print("itu :", vector.shape)
print(vector.toarray())

In [ ]:
print(vector[626])

In [ ]:
# Buat yang data testing

X = df['Tweet']
y = df['Class']

Training, Testing, train, test = train_test_split(df['Tweet'], y, random_state=42, test_size = 0.2)
#split X and y into training and testing sets. By default, it splits 75% #training and 25% test. random_state=1 for reproducibility.

In [ ]:
# untuk membangun model

X = df['Tweet']
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(vector, y, random_state=42, test_size = 0.2)
#split X and y into training and testing sets. By default, it splits 75% #training and 25% test. random_state=1 for reproducibility.

In [ ]:
# Menyeimbangkan data

from collections import Counter
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(random_state=42)
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_train_sm))

In [ ]:
labels1 = ['Positive', 'Negative']
sizes1 = [107, 693]

fig, ax = plt.subplots()
ax.pie(sizes1, labels=labels1, autopct='%1.1f%%', shadow = True, explode = (0.1,0))
ax.axis('equal')  # Equal aspect ratio ensures the pie chart is circular.
ax.set_title('Data Training Sebelum SMOTE')

plt.show()

labels2 = ['Positive', 'Negative']
sizes2 = [693, 693]

fig, ax = plt.subplots()
ax.pie(sizes2, labels=labels2, autopct='%1.1f%%', shadow = True, explode = (0.1,0.1))
ax.axis('equal')  # Equal aspect ratio ensures the pie chart is circular.
ax.set_title('Data Training Setelah SMOTE')

plt.show()

In [ ]:
# Mencari Model terbaik

random_state = 0
n_estimators = [50, 200, 500, 1000]
criterion = ['gini', 'entropy']
Hasil = []

for i in range(len(n_estimators)):
    for j in range(len(criterion)):
            model = RandomForestClassifier(n_estimators=n_estimators[i],criterion=criterion[j], random_state = random_state)
            y_preds = model.fit(X_train_sm, y_train_sm).predict(X_test)
            accuracy = accuracy_score(y_test,y_preds)
            Hasil.append(accuracy)

In [ ]:
c = 0
for i in range(len(n_estimators)):
    for j in range(len(criterion)): 
            print("n_estimators =" ,n_estimators[i], "," ,"criterion =",criterion[j], ","
                  ,"Akurasi =" ,Hasil[c]*100, "%")
            c+=1

Model Terbaik:
<br>n_estimators = 500 
<br>criterion = 'entropy'

In [ ]:
model = RandomForestClassifier(n_estimators = 500 , criterion =  'entropy' , random_state = 0)
model

In [ ]:
y_preds_training = model.fit(X_train_sm, y_train_sm).predict(X_train)
y_preds_testing = model.fit(X_train_sm, y_train_sm).predict(X_test)

In [ ]:
# penentuan akurasi training dan testing
accuracy_training = accuracy_score(y_train, y_preds_training)
accuracy_testing = accuracy_score(y_test, y_preds_testing)
print('Accuracy: ', accuracy_training*100, "%")
print('Accuracy: ', accuracy_testing*100, "%")

In [ ]:
print("Confusion Matrix : ")
print(confusion_matrix(y_test, y_preds_testing))

In [ ]:
print("Classification report : ")
print(classification_report(y_test, y_preds_testing))

In [ ]:
c = make_pipeline(vectorizer, model)

Peluang kelas Positif dan Negatif pada setiap kalimat data testing

In [ ]:
aktual_testing = pd.DataFrame(y_test)
aktual_testing = aktual_testing.rename(columns = {'Class':'aktual_testing'})
aktual_testing = aktual_testing.reset_index(drop = True)
aktual_testing.head()

In [ ]:
kelas_prediksi_testing = pd.DataFrame(y_preds_testing, columns = ['kelas_prediksi_testing'])
kelas_prediksi_testing.head()
len(kelas_prediksi_testing)

In [ ]:
data_testing = pd.concat([aktual_testing, kelas_prediksi_testing], axis = 1)
print(data_testing.head())
print(len(data_testing))

In [ ]:
Tweet_testing = pd.DataFrame(Testing)
Tweet_testing = Tweet_testing.rename(columns = {'Class':'Tweet_testing'})
Tweet_testing = Tweet_testing.reset_index(drop = True)
Tweet_testing.head()

In [ ]:
probabiliti_testing_positive = []

for i in range(len(Tweet_testing['Tweet'])):
    a = c.predict_proba([Tweet_testing['Tweet'][i]])[0,0]
    probabiliti_testing_positive.append(a)
    
probabiliti_testing_negative = []

for j in range(len(Tweet_testing['Tweet'])):
    b = c.predict_proba([Tweet_testing['Tweet'][j]])[0,1]
    probabiliti_testing_negative.append(b)

In [ ]:
peluang_testing = {'probabiliti_testing_positive': probabiliti_testing_positive, 
                   'probabiliti_testing_negative': probabiliti_testing_negative}
peluang_testing = pd.DataFrame(peluang_testing)
peluang_testing.head()

In [ ]:
data_peluang_testing = pd.concat([Tweet_testing,data_testing,peluang_testing], axis = 1)
data_peluang_testing.head(6)

In [ ]:
for i in range(len(data_peluang_testing['aktual_testing'])):
    if data_peluang_testing['aktual_testing'][i] != data_peluang_testing['kelas_prediksi_testing'][i]:
        print('Index ke- ', i)

In [ ]:
data_peluang_testing.loc[[41,42,49,51,55,111,129,156,168,191,195], :]

In [ ]:
data_peluang_testing.loc[[0],:]

Peluang kelas Positif dan Negatif pada setiap kalimat di keseluruhan data

In [ ]:
y_prediksi_seluruhnya = model.predict(vector)

In [ ]:
accuracy = accuracy_score(y, y_prediksi_seluruhnya)
print('Accuracy: ', accuracy*100,'%')

In [ ]:
prediksi = pd.DataFrame(y_prediksi_seluruhnya, columns = ['Prediksi'])

In [ ]:
Data_join = pd.concat([df,prediksi], axis = 1)
Data_join.head()

In [ ]:
probabiliti_positive = []

for i in range(len(df['Tweet'])):
    a = c.predict_proba([df['Tweet'][i]])[0,0]
    probabiliti_positive.append(a)
    
probabiliti_negative = []

for j in range(len(df['Tweet'])):
    b = c.predict_proba([df['Tweet'][j]])[0,1]
    probabiliti_negative.append(b)

In [ ]:
peluang = {'probabiliti_positive': probabiliti_positive, 'probabiliti_negative': probabiliti_negative}
peluang = pd.DataFrame(peluang)
peluang.head()

In [ ]:
data_peluang = pd.concat([Data_join,peluang], axis = 1)
data_peluang.head()

In [ ]:
for i in range(len(data_peluang['Class'])):
    if data_peluang['Class'][i] != data_peluang['Prediksi'][i]:
        print('Index ke- ', i)

In [ ]:
data_peluang.loc[[30,96,309,312,314,408,800,823,829,837,879], :]

In [ ]:
data_hapus = data_peluang.loc[[30,96,309,312,314,408,800,823,829,837,879], :]
data_hapus = data_hapus.reset_index(drop=True)
data_hapus

In [ ]:
data_untuk_interpretasi = data_peluang.drop(index = [30,96,309,312,314,408,800,823,829,837,879])
data_untuk_interpretasi.tail()

In [ ]:
data_untuk_interpretasi.loc[25:31]

In [ ]:
data_untuk_interpretasi = data_untuk_interpretasi.reset_index(drop=True)

In [ ]:
data_untuk_interpretasi.tail()

In [ ]:
data_untuk_interpretasi.loc[25:31]

INTERPRETASI

In [ ]:
# Proses definisi index ke-0 positif, ke-1 negatif

class_names = ['positive','negative']

In [ ]:
# Mendefinisikan LIME dengan fungsi untuk interpretasi text untuk teks dengan nama kelas yang sudah ditentukan

explainer = LimeTextExplainer(class_names=class_names)

In [ ]:
idx = 626
exp = explainer.explain_instance(df['Tweet'][idx], c.predict_proba, num_features = 30)
print('Document id: %d' % idx)
print('Probability [Negative] =', c.predict_proba([df['Tweet'][idx]])[0,1])
print('Probability [positive] =', c.predict_proba([df['Tweet'][idx]])[0,0])
print('True class: %s' % class_names[df['Class'][idx]])

In [ ]:
# Melihat hasil explainer kalimat dalam bentuk list

exp.as_list()

In [ ]:
idx_hapus = 7
exp_hapus = explainer.explain_instance(data_hapus['Tweet'][idx_hapus], c.predict_proba, num_features = 30)
print('Document id: %d' % idx)
print('Probability [Negative] =', c.predict_proba([data_hapus['Tweet'][idx_hapus]])[0,1])
print('Probability [positive] =', c.predict_proba([data_hapus['Tweet'][idx_hapus]])[0,0])
print('True class: %s' % class_names[data_hapus['Class'][idx_hapus]])

In [ ]:
exp_hapus.as_list()

In [ ]:
# Melihat apabila kata yang berpegaruh negatif pada hasil klasifikasi negatif dihilangkan

print('Original prediction:', model.predict_proba(vector[idx]))
tmp = vector[idx].copy()
tmp[0,vectorizer.vocabulary_['pajakperasrakyat']] = 0
tmp[0,vectorizer.vocabulary_['balut']] = 0
tmp[0,vectorizer.vocabulary_['lampau']] = 0
tmp[0,vectorizer.vocabulary_['jarak']] = 0
tmp[0,vectorizer.vocabulary_['satu']] = 0
print('Prediction removing some features:', model.predict_proba(tmp))
print('Difference:', model.predict_proba(tmp) - model.predict_proba(vector[idx]))

In [ ]:
# Visualisasi

%matplotlib inline
fig = exp.as_pyplot_figure()

In [ ]:
# Visualisasi

exp.show_in_notebook()

In [ ]:
Text = data_untuk_interpretasi['Tweet']
kelas = data_untuk_interpretasi['Class']
print(Text[0])
print(kelas[0])
print(len(Text))
print(len(kelas))

In [ ]:
Hasil = []

for i in range(len(Text)):
    exp = explainer.explain_instance(Text[i], c.predict_proba, num_features = 30)
    Hasil.append(exp.as_list())
    print("index ke -",i, 'Selesai')

In [ ]:
print(len(Hasil))
print(Hasil)

In [ ]:
input = ("[('taxamnesty', -0.04870817306408237), ('gimana', 0.04687106104490137), ('pingin', 0.04682686662711727), ('itunganya', 0.04627041758618145), ('mboh', 0.04624537171990718), ('ppn', -0.04324112817113634), ('genjot', 0.039173505433647046), ('pajak', 0.0003688806326204346)], [('gapapa', -0.03297906120115371), ('sri', -0.02986476425221318), ('uang', 0.015665755060299824), ('papers', 0.014277422636647587), ('pandora', 0.013734035077764582), ('jabat', 0.012272193161863915), ('gimana', 0.011003209391690933), ('taxamnesty', 0.0071503806653955915), ('lumayan', -0.007068357314820698), ('negara', 0.006729645289348529), ('ehh', 0.005993520854463561), ('ppn', -0.005693617192348307), ('pajak', 0.005417992272180159), ('mboh', 0.0044970314798995475), ('bangun', -0.004360457457084355), ('itunganya', 0.004135894057596649), ('bal', 0.003630277486931012), ('ndang', 0.0035422740136954556), ('yo', 0.0026495167524793937), ('pingin', 0.001988315880523371), ('berita', 0.0019577564355780525), ('genjot', -0.0017434065105959029), ('bu', 0.0014266148144501081), ('menteri', 0.0005802610909729797)], [('rakyat', 0.07944537349932697), ('indonesia', -0.06484791290235131), ('dana', -0.02426276694961699), ('kasihan', 0.018471882903869835), ('panjang', 0.016497229596474608), ('oknum', 0.014827364060159541), ('senang', 0.0128875212407268), ('habis', 0.012130377281417692), ('korupsi', 0.01124194973174743), ('ppkm', 0.011231996660139191), ('covid', -0.007918145739557918), ('pajak', -0.002641871788974697), ('bayar', 0.0008348215565076252)], [('kaya', 0.055595414418667895), ('jabat', 0.03483078682964093), ('bayar', -0.028761561807704894), ('sedih', 0.023795222303420925), ('ramai', 0.022469121630915606), ('uang', 0.021308486841585855), ('hindar', -0.014660784906167967), ('harta', -0.00784075304887612), ('pajak', 0.00620489023409685), ('negara', 0.005270841478570402), ('sembunyi', 0.0029451075214981)], [('kaya', 0.07364669852601351), ('rakyat', 0.07292791096672262), ('wajib', -0.057019234646508606), ('masyarakat', -0.04739903516445227), ('pandora', 0.027639171147590083), ('papers', 0.017689759071704256), ('beban', 0.016262232575497193), ('aset', 0.01572385728015462), ('apes', 0.01442204883522635), ('sisi', 0.013624298113990326), ('orang', 0.012511596610215855), ('pajak', 0.012335301410227583), ('jelata', 0.0018193927753677294), ('sembunyi', 0.0014749881187229655), ('laku', 0.0002413195153802907), ('hindar', -0.000129165775929813)], [('wajib', -0.2889261525662022), ('patuh', -0.12192432842465822), ('sosialisasi', -0.11868232526148004), ('iii', -0.06425502469354256), ('pemkot', -0.04722172644054768), ('kanwil', -0.045592171228192924), ('jabar', -0.04271199957805611), ('bekas', -0.04262659842258686), ('djp', -0.04081845983975831), ('pajak', -0.02359161465095834)], [('rakyat', 0.09153145762212425), ('ngumpetin', -0.030520073512006096), ('nakuti', 0.027998530351161226), ('nakut', 0.0273256936868879), ('no', 0.023719390873564256), ('harta', -0.02286181863595263), ('munafik', 0.019452963532397945), ('biar', -0.017462657433792644), ('ga', -0.015978674859062666), ('pajak', 0.010180965506457185), ('kena', -0.004053546634293357)], [('tutup', -0.15050562315430074), ('adil', -0.14964981325604299), ('sistem', -0.12123752969041197), ('tata', -0.09093158091782924), ('ruuhpp', -0.08121663251257098), ('penegasaan', -0.07423621622730392), ('komitmen', -0.06185799872757287), ('hindar', -0.053966472984684453), ('celah', -0.04514198249296352), ('pajak', -0.024897564515564793), ('rakyat', 0.0167201742228549)], [('sistem', -0.0723205802979905), ('harap', -0.033629277205847914), ('negeri', -0.030283429968319366), ('henti', 0.021679741894269616), ('ubah', -0.015775580791704446), ('muslimahnews', 0.015543111810923169), ('zalim', 0.014552332141791717), ('pajak', -0.0031424013268623475)], [('bijak', 0.04054580952358376), ('pajak', 0.030471012276701567), ('makan', 0.02269147822926778), ('taat', -0.01337524874562992), ('jahat', 0.009445800931641983), ('engga', 0.008715625239114714), ('ajak', 0.008596372942745541), ('gila', 0.006315171767755386), ('orang', 0.004578676481489249)], [('rakyat', 0.11343532432074585), ('bayar', -0.10178428031714619), ('negara', -0.06588131218000971), ('pajak', -0.04395069388942265), ('kompak', 0.04259794781004448), ('gak', -0.027579001246886215), ('hukum', 0.027493568247238833)], [('bayar', -0.11516992589191395), ('gw', 0.06752719250760642), ('shadow', 0.06105531388977119), ('pajak', -0.03898832368341158), ('gak', -0.02984118197307196)], [('rakyat', 0.02043874930619315), ('butuh', -0.0029499651947829694), ('muslimahnews', 0.002324444030791681), ('sejahtera', 0.002112794367735955), ('topang', 0.0020780382513400507), ('tumpu', 0.002005504321123969), ('sumber', 0.0007978012846060221)], [('rakyat', 0.042250940414119385), ('perintah', -0.028848392334311572), ('pihak', -0.023464874867815667), ('atur', -0.01950533869095141), ('uu', -0.012302787562769008), ('kontra', 0.008040764476056459), ('pajak', 0.00708452386745666), ('jadi', 0.006917402459181338), ('harmonisasi', -0.006828614915563955), ('tampak', 0.005906521215083577), ('editorial', 0.004860971943264495), ('aju', 0.004487007537836915), ('penting', 0.004138379096288246), ('kesah', 0.003803397931192942), ('zalim', 0.0035554328168913643), ('hpp', -0.0034420043558923675), ('objek', 0.0033232269840486116), ('buru', 0.003284248673836286), ('nilai', 0.0020138826258514005), ('dpr', 0.0012439557820536293)], [('cerdas', -0.18478999194358806), ('rakyat', -0.1685662672246902), ('layak', -0.16440928595352416), ('pimpin', -0.1379361743300436), ('tinggal', -0.0980894817747274), ('kerja', -0.09081036646257239)], [('bayar', -0.06188726544358961), ('ketua', 0.03496132777490633), ('bolos', 0.033451690342824365), ('tim', 0.031931840288827076), ('nasional', 0.021826643188242657), ('pajak', 0.004865703347356505)], [('bebas', -0.04327959176055945), ('ya', 0.02485304594606494), ('holiday', 0.018762740436623616), ('royalti', 0.018302358029044705), ('pajak', -0.017977547151377733), ('banget', 0.017762784865486502), ('rugi', 0.01671534021099087), ('hapus', 0.016411577352672048), ('tega', 0.01639059062652985), ('gede', 0.015786870890913587), ('tax', 0.013848578319722991), ('batu', 0.013270065450860553), ('bara', 0.012649113785962259), ('negara', 0.008950251340267307)], [('bunga', -0.024845951455177304), ('rakyat', 0.017058012313805125), ('utang', 0.011927277304038303), ('jerat', 0.0039082527859061805), ('online', 0.0032854840094679394), ('negara', 0.0032295686505202444), ('sadar', 0.002336126412303739), ('pinjam', 0.0016379806361704653), ('minjam', 0.0014922277509301393)], [('pajakperasrakyat', 0.03269037525751287), ('kaya', 0.02174537650234867), ('rakyat', 0.021347870734943507), ('orang', -0.016707458553862114), ('dibebaslan', 0.010904998240154175), ('peras', 0.010010189885925853), ('pajak', 0.0017377703201659216)], [('pajak', -0.022070787296654855), ('focus', 0.00954013260700871), ('negara', 0.009440573770068166), ('ngotot', 0.009273131206806439), ('utak', 0.009059012408989707), ('sih', 0.007053208006934596), ('banget', 0.0069128703423939645), ('atik', 0.0068896076072087805), ('evaluasi', 0.0067552571567520816), ('deh', 0.006177983950517966), ('coba', 0.004967637481794464), ('terima', -0.002182477766192188)], [('pajakperasrakyat', 0.016098363914926955), ('pajak', 0.009464197504906024), ('diadzab', 0.001063233865137141), ('mungut', 0.000857868713810212), ('dawud', 0.0008462468477932623), ('abu', 0.0006714281609040155), ('tarik', 0.0005221515293136088), ('ahmad', 0.0005192686509485622), ('neraka', 0.00046840451642870916), ('sungguh', 0.0004159869098378879)], [('adil', -0.12383424451414556), ('ruuhpp', -0.06852840579316122), ('nilai', 0.04274978557430919), ('pandora', 0.03693983885168686), ('papers', 0.034064740699021524), ('pks', 0.031725727771445646), ('tolak', 0.029208233422034168), ('anis', 0.027558758453381606), ('bukti', 0.025675885282343627), ('hindar', -0.015524586532539594), ('pajak', -0.0009630981573314893)], [('masyarakat', -0.06334377136958307), ('paksa', 0.038451134552884396), ('pajak', 0.03311420825188961), ('entar', 0.019929882319905372), ('nunggu', 0.019196704657021064), ('mempan', 0.018037461448684237), ('sabar', 0.017722795822215694), ('uganda', 0.01712565969676716), ('ga', 0.008465887166001268), ('bayar', -0.0003272454880341248)], [('hindar', -0.06342724117772676), ('bayar', -0.02923105113253561), ('jabat', 0.029040691987270016), ('pandora', 0.024595146644637293), ('nyapres', 0.024233891118578283), ('kuak', 0.022772825431276348), ('papers', 0.01998688873405086), ('skandal', 0.01956486345925307), ('sibuk', 0.01749457329037485), ('negara', -0.011120047130843668), ('pajak', 0.006405704750916335)], [('bayar', -0.12243181616958261), ('rakyat', 0.1183345299143001), ('pajak', -0.061161533093265504)], [('rakyat', 0.01045915607635606), ('peras', 0.007260190013673867), ('hapus', 0.005345723900285885), ('nelayan', 0.004737549174623413), ('batu', 0.004448064230431383), ('oligarki', 0.00429224044337854), ('untung', -0.0033011975458623077), ('bara', 0.0032861837096193943), ('royalti', 0.0027826464083147377), ('rizalramli', 0.0021034561513399133), ('pajak', -0.0006636118355052345), ('persen', 0.0005016382960060521)], [('ancam', -0.0185113020293107), ('nelayan', 0.015314020395347969), ('laut', 0.007698914870976587), ('daerah', 0.0018195060825690885), ('izin', 0.0010397743308441186)], [('rakyat', 0.08872109240877334), ('bebas', -0.06038129453044738), ('berat', 0.02939414422977069), ('kakap', 0.022160989824800106), ('taipan', 0.020910734092732378), ('pajak', 0.0208042379651383), ('ringan', 0.01387427726810358), ('usaha', 0.010080265808371111), ('beban', 0.006767286826780539)], [('tuju', -0.08523111794967116), ('pph', -0.07401126306500048), ('presiden', 0.015815226586573295), ('hasil', -0.015039519215101375), ('langgar', 0.011132265178684275), ('kelola', 0.010859408922379155), ('jelasin', 0.009740088779158098), ('objek', 0.00805855207070389), ('profesor', 0.007726261157598141), ('pinginnya', 0.007289952754890901), ('kkn', 0.005513672543712272), ('makna', 0.00509946127725138), ('perlu', 0.00475343668491872), ('pasal', 0.00435308633905209), ('uud', 0.004272277085799693), ('negara', 0.0041534730633559095), ('amandemen', 0.0036299057256998996), ('pajak', 0.002727679497847237), ('dpr', 0.0024589977632973265), ('perintah', 0.00024329644767096307)], [('integrasi', -0.11278584966180562), ('uuhpp', -0.09843923895691548), ('nik', -0.07468365528513239), ('data', -0.06517908012892547), ('duduk', -0.05855120115742499), ('administrasi', -0.05449214205222216), ('npwp', -0.05351621414618099), ('sistem', -0.04266553444514356), ('basis', -0.03961466792393236), ('mudah', -0.037764887535962244), ('pajak', -0.026695253689787646)], [('rakyat', 0.014307644130459221), ('malu', 0.013498085613446982), ('belagu', 0.013059045294609474), ('tau', 0.009077731169215194), ('makan', 0.008261274708704087), ('uang', 0.00694143458428413), ('gak', -0.006592318476424745), ('pajak', -0.003448189805775614)], [('pajakperasrakyat', 0.07146071103529712), ('kena', -0.02100953768990243), ('dipajakin', 0.004318318706754329), ('libur', 0.0034810765968481478), ('thailand', 0.003053127288922574), ('pajak', -0.0007984246682178546)], [('rakyat', 0.03324960873134858), ('pajakperasrakyat', 0.029798854992098157), ('lini', 0.014209501220057989), ('modal', 0.013535934129492253), ('hutang', -0.012827510681223826), ('menteri', -0.011159847641407396), ('pajak', -0.008523387993143866), ('hebat', 0.008504341493675202), ('hidup', 0.007792101409883729), ('negara', -0.007554643404189647), ('bangun', 0.005765942136945473), ('uang', 0.002552052452777016)], [('pakpolnya', -0.14569391525382677), ('biar', -0.13280809252780876), ('rajin', -0.12635059088101636), ('semangat', -0.12187665627365994), ('ayo', -0.08637166245062161), ('bayar', -0.05074051328855546), ('pajak', -0.039607320084782675)], [('hasil', -0.044558441218036635), ('uang', 0.016911969342253565), ('monas', 0.009823428485810545), ('ijtima', 0.008572677415739442), ('pohon', 0.008181708184925137), ('ulama', 0.007968031534851386), ('tukang', 0.006996525761617116), ('garong', 0.006144202687047818), ('pajak', 0.0038394307732067187)], [('wajib', -0.3072077506393714), ('kpp', -0.14177286032268815), ('pratama', -0.10582901195031649), ('tegal', -0.08881055507171125), ('pahlawan', -0.07753348546910142), ('gabung', -0.06677631936590449), ('daftar', -0.039855537783294544), ('pajak', -0.028799286927244344), ('negara', 0.0067592463373589975)], [('wajib', -0.20259165529316991), ('duduk', -0.15139669576557252), ('nik', -0.1328807766099456), ('manfaat', -0.061950289402340625), ('nomor', -0.04822435325200307), ('elsam', -0.04482966490251434), ('pokok', -0.04307527736985936), ('pres', -0.034765946461747764), ('cantum', -0.0318337233247797), ('induk', -0.024197229541137632), ('nilai', -0.002927557016097991)], [('bayar', -0.020876848051391908), ('aja', 0.01892260714826075), ('minyak', 0.01290761392163652), ('btw', 0.012097680084930505), ('pajak', 0.011669828108297944), ('lu', 0.011096463083871274), ('ngeluh', 0.010828130583858751), ('sih', 0.01008085395940643), ('dah', 0.00955788250426753), ('beli', 0.009299899324751157), ('miskin', 0.005359437975198842)], [('gelar', -0.020304755651710388), ('negara', 0.01614328081528673), ('nelayan', 0.013072541598073572), ('biaya', 0.009123089463386903), ('jawa', 0.007348755584729907), ('terima', -0.00732487395012599), ('ratus', 0.006877672881909837), ('putih', 0.006296891716584027), ('bendera', 0.006239585346476695), ('kibar', 0.006153251469709543), ('pati', 0.0061443887105745165), ('aksi', 0.005732582503640765), ('pajak', -0.0023581723233794443)], [('uuhpp', -0.20139478764713128), ('adil', -0.15108749485594156), ('masyarakat', -0.09980442147691257), ('tuju', -0.08016966267559257), ('salah', -0.05126596152016195), ('menengah', -0.03257113892408708), ('cipta', -0.03120923064240741), ('tara', -0.023859193766709932), ('bawah', -0.023049837388576053)], [('wajib', -0.0811459005765548), ('milik', 0.04847008074977425), ('ngemplang', 0.03601625770645968), ('nampak', 0.033823254031229655), ('keren', 0.029828467700444772), ('koq', 0.025736381913558714), ('kilas', 0.021746073336309887), ('npwp', -0.017566622996608137), ('pajak', -0.008652466326953863), ('nik', -0.006519383347830487)], [('bayar', -0.09039274016814149), ('ikhlas', 0.06652284371725518), ('gw', 0.06203136041716626), ('bikin', 0.02524404149440982), ('pajak', -0.021526561672544187), ('gak', -0.018571478354470344)], [('kemplang', 0.04759949089465597), ('pajak', -0.042259305044485496)], [('adil', -0.20753422511014083), ('wajib', -0.1691552432319892), ('pajak', -0.02704824515935803), ('ktp', -0.020846645139394767), ('otomatis', 0.013198581379292943), ('lapor', 0.005289696369707683), ('umur', 0.0043733855964607875), ('udah', 0.0036700884090676085), ('nih', 0.0025076895860515354)], [('rakyat', 0.021921462836692017), ('utang', 0.00730981485715313), ('sejahtera', 0.004836429476889683), ('pundak', 0.003953005759683118), ('pajak', 0.0031283900640637955), ('beban', -0.0029537797831752535), ('ujung', 0.002610614835557252), ('topang', 0.0024363123927409447), ('sumber', 0.0019574630138055236), ('butuh', -0.0005784903792438067)], [('wajib', -0.08111366909208721), ('milik', 0.04795359518228031), ('ngemplang', 0.03651857426516631), ('nampak', 0.03467089287570824), ('keren', 0.027592718156705644), ('koq', 0.023944615823680154), ('kilas', 0.02318923897741936), ('npwp', -0.017611840207424376), ('pajak', -0.01261868088188502), ('nik', -0.008419543714643384)], [('bela', -0.034918283245639635), ('rakyat', 0.019493038346584852), ('jenderal', -0.01260561708977374), ('jabat', 0.008962005948319682), ('negara', 0.006950123696766951), ('kpk', 0.0029852408831902858), ('jujur', 0.0027278585094208346), ('gilir', 0.0021982943513989095), ('gelap', 0.002157810954979552), ('kritis', 0.0012416304563066891), ('anggota', 0.0010585711889918663), ('copot', 0.0007170160177544855), ('pecat', 0.0006484641847453808), ('tni', 0.0005529284645732243)], [('rakyat', 0.07309021467731705), ('nik', -0.05727531557395281), ('npwp', -0.023138237320109674), ('komunis', 0.020544290011819642), ('milik', 0.020311968775409934), ('rezim', 0.012032094344779377), ('ktp', -0.008137664940123988), ('slogan', 0.008000740938554005), ('peras', 0.007877300455512881), ('habis', 0.005427810611262653), ('hidup', 0.003971503311221726)], [('nama', 0.010509666815229024), ('selundup', 0.009993875397726818), ('cangkang', 0.009275350983386967), ('luhut', 0.008242769083007093), ('pajak', 0.0070163732872017565), ('usaha', -0.006540279323848663), ('pakai', 0.005946890766371386)], [('indonesia', -0.05414934144097023), ('mabok', 0.029020780480007623), ('pajakperasrakyat', 0.025461406756467825), ('rakyat', 0.02095252753066705), ('utang', 0.009027712623245898), ('jabat', 0.008734299426392395), ('korupsi', 0.004688122956566088), ('pajak', -0.002917042222520131), ('perintah', -0.0017460246592962887)], [('rakyat', 0.09451277651156018), ('bayar', -0.058565305729719686), ('rugi', 0.015587200131078623), ('manfaat', -0.013859868680631535), ('pajak', -0.013385812362068043)], [('adil', -0.17096033224638824), ('rakyat', 0.07450931840114118), ('pajak', -0.0239485414544316), ('konglomerat', 0.022071966599142737), ('tamanjakan', 0.01384709141637811), ('mana', 0.013754373316272491), ('dpr', 0.013235732976383606), ('anggota', 0.012439940548383823), ('peduli', 0.012417804993429747), ('letak', 0.00888887611005017), ('ringan', 0.006971526822118934), ('fasilitas', 0.004829997549193458)], [('kuasa', 0.04633697245035774), ('hutang', -0.04193658837395146), ('banyakin', 0.02235322589115183), ('gedein', 0.022221704644665437), ('cokin', 0.021561010878628842), ('rezim', 0.015595781370512528), ('negara', -0.012185931712604533), ('hancur', 0.009934361752166353), ('pajak', -0.008427936291533876)], [('masyarakat', -0.11205028314740899), ('lo', 0.08633723218785262), ('institusi', 0.024140496937416794), ('mentang', 0.022915291628881698), ('bangkrut', 0.020337829986862465), ('kritik', 0.019555752078169876), ('gaji', 0.015484531234328004), ('pajak', 0.013134246299474581), ('negara', 0.011537486494052635), ('gak', 0.009844899845940079)], [('umkm', -0.2357381463925834), ('uuhpp', -0.20910293328979582), ('bruto', -0.051994049338582794), ('hasil', -0.02841124776062985), ('laku', -0.028410848803925765), ('atur', 0.024762517692636713), ('kena', -0.015211317258610337), ('pajak', -0.010583353332820728), ('kait', 0.0009876726608747997), ('khusus', 0.0005243825495404144)], [('rakyat', 0.015931209013614247), ('subsidi', 0.012507891520946267), ('negara', 0.00413655961842201), ('kasih', 0.0025237412031389457), ('jahat', 0.0025032814068756725)], [('kaya', 0.08247801820663896), ('rakyat', 0.07390966540559536), ('wajib', -0.05149596742141102), ('masyarakat', -0.035256094723932), ('pandora', 0.03165405182612742), ('aset', 0.01937214806998215), ('papers', 0.019013755411786553), ('beban', 0.016144804038624987), ('sisi', 0.015991017013903767), ('orang', 0.013814513170702349), ('pajak', 0.013718707625929477), ('apes', 0.01271105495043455), ('opini', -0.00953866020133455), ('laku', 0.008493551046517432), ('sembunyi', 0.005085647573501591), ('hindar', 0.002294820409819552), ('jelata', 0.0012564231591187898)], [('pajakperasrakyat', 0.03925589917373634), ('duta', 0.007872206673666315), ('keren', 0.00607903088302806), ('tuh', 0.004736849126610223), ('pajak', -0.0004496821136533507)], [('doang', -0.10839537827086487), ('bayarin', 0.02568805722922924), ('pajak', -0.024761814856698178), ('capek', 0.01606985780466011)], [('jokowi', 0.016238141963843076), ('pajak', -0.011296222392252253), ('cepat', 0.010757175677479813), ('daftar', -0.008937983436638318), ('kereta', 0.007392273861443252), ('bijak', -0.006722442628658671), ('amnesti', 0.00472666689272359), ('plinplan', 0.003019304019200136)], [('uuhpp', -0.26998813428777063), ('karbon', -0.18739665493760974), ('implementasi', -0.15690278512253542), ('adil', -0.1456767042508853), ('pajak', -0.06860406706878852)], [('karbon', -0.23933871760390812), ('indonesia', -0.12039803412463315), ('iklim', -0.11095954623300816), ('terap', -0.10016283350186153), ('salah', -0.07944840007997658), ('ubah', -0.07686869437271458), ('mitigasi', -0.06517423617345237), ('alas', -0.03842239488259817), ('pajak', -0.023836115762098393)], [('kerja', -0.048424950667032285), ('ketok', -0.04634757132609636), ('pajak', 0.02040864183772567), ('rakyat', -0.01724131293016478), ('dpr', -0.003832909116878784), ('hormat', 0.0032422188622847456), ('era', 0.00308153682856969), ('ngaku', 0.002715989145415952), ('palu', 0.00265705026065577), ('ngabisin', 0.002405022688782576), ('ambyar', 0.0020599389119121824)], [('wajib', -0.21750088079666846), ('paja', -0.15098243035274186), ('kpp', -0.1297436767179168), ('laksana', -0.11008397424325903), ('harap', -0.0921955171559895), ('pajakkuatindonesiamaju', -0.05759643479145591), ('edukasi', -0.053116596236746666), ('pajakkitauntukkita', -0.0445154090834135), ('pajak', -0.037408932031467064)], [('kemplang', 0.07779401146561517), ('taxamnesty', -0.05900554407293746), ('sepele', 0.05346762870979278), ('anggap', 0.036549025238789425), ('bijak', 0.02672631998251488), ('jilidii', 0.026679912295198133), ('program', 0.021465485298879386), ('pajak', -0.00749546901498937)], [('negeri', -0.06108257623226409), ('langit', 0.02087788327828741), ('sakit', 0.018148841734030915), ('pajak', -0.0043846407788859666)], [('masuk', 0.009761957105524766), ('pandora', 0.008406941678124518), ('papers', 0.007046244921813202), ('ngemplang', 0.006711174761333493), ('susah', 0.005836810283541015), ('gak', -0.005696083186375291), ('gede', 0.005651514547021464), ('hidup', 0.00445271796752271), ('pajak', 0.004449205408352264), ('korupsi', 0.0037341013530703196), ('aman', -0.0008849223706442625)], [('karbon', -0.18580287914886198), ('terap', -0.09459213132214121), ('iklim', -0.09344835703255311), ('indonesia', -0.08722394981297174), ('salah', -0.07013387399219757), ('ubah', -0.06863743208579756), ('potensi', -0.06237172723740948), ('mitigasi', -0.04633198911123564), ('hasil', -0.03533878918354546), ('laku', -0.027452954764610262), ('alas', -0.024111054695765823), ('pajak', -0.01878155912459825), ('aktivitas', -0.009511525442326006), ('gas', -0.008730480670033654), ('perilaku', -0.008060456606090597), ('emisi', -0.007475799613910822), ('kaca', -0.004095908117830193), ('rumah', -0.003443873889485217), ('ekonomi', 0.0020271124540564094)], [('defensif', 0.012912121992460614), ('evaluasi', 0.012751100318224368), ('institusi', 0.011643688222881724), ('sial', 0.010926690894836987), ('mentang', 0.009891383758725666), ('bangkrut', 0.009248976331125742), ('gak', -0.007504023248072966), ('negara', 0.00031859729738379137)], [('bayar', -0.11731668745049398), ('biar', -0.05128923013805147), ('hasil', -0.04476838834450738), ('jelata', -0.017322411845141186), ('warga', 0.012378922208652098), ('obrak', 0.008864331021830932), ('kritik', 0.008410723840767485), ('pajak', -0.004350382688888327), ('perintah', 0.00386955687014629), ('kasih', 0.0038378661568672515), ('nek', 0.003376577637099595), ('identitas', 0.003375570152144601), ('beda', 0.0032145306906463983), ('dicyduk', 0.0031809631102429287), ('rutin', 0.0031557104826844375), ('akun', 0.0031438945610813363), ('dprd', 0.003028217363486788), ('iku', 0.0029543679210136623), ('anonim', 0.002437567270703388), ('kudu', 0.0023874894976122856), ('anggota', 0.001809938486565742), ('solusi', 0.0017729366793099639), ('ngasih', 0.0013059887776650768)], [('rakyat', 0.02033595390161481), ('pajak', 0.013342795376506789), ('bayar', -0.011343649459513428), ('mungkinkahpresidendarktriad', 0.009907562305090442), ('subsidi', 0.00947207651518033), ('kasih', 0.007666335202497989), ('negara', 0.004978958133165227), ('gile', 0.003943096158950067), ('elu', -0.0029192433671230804), ('droo', 0.0023793308959631264), ('jahat', 0.0022954622691368533)], [('bebas', -0.0792776362118753), ('nelayan', 0.03567526768940165), ('ya', 0.03420809985519272), ('kena', -0.026738959606331936), ('banget', 0.024286690104289473), ('rizalramli', 0.022560006982474925), ('tega', 0.021961324225719205), ('gede', 0.02026475609158488), ('pajak', -0.019364311886459883), ('persen', 0.017048302257848535)], [('indonesia', -0.05441265498887439), ('aja', 0.02965775907104016), ('biar', -0.029637142645773405), ('utang', 0.022656708539956376), ('ngomongin', 0.016465633995167386), ('macem', 0.015778061641346788), ('diapain', 0.014474826217851377), ('kantor', 0.014247618673234715), ('emang', 0.013358245608472017), ('misuh', 0.013326493528666722), ('indo', 0.01187836177356262), ('dah', 0.010384276518297552), ('gin', 0.010120513878060889), ('bayar', -0.008194507969445791), ('pajak', 0.004383154021061343), ('orang', -0.0002434755593203529)], [('integrasi', -0.2045799839002614), ('nik', -0.12871190687925066), ('implementasi', -0.12638477504758064), ('data', -0.12579626565092103), ('duduk', -0.12254624094948037), ('npwp', -0.09049031076416346), ('atur', -0.04511424134021477), ('pajak', -0.015960887874903674)], [('umkm', -0.21501346840946522), ('adil', -0.13707847466708523), ('rp', -0.07468085367442454), ('ruuhpp', -0.06708594856045492), ('pph', -0.059383561391728755), ('hasil', -0.057412711887163445), ('ubah', -0.045726932661883074), ('ulang', -0.025250537910385194), ('pajak', -0.02124948965684022), ('batas', -0.019123091888358156), ('juta', -0.019078289364611728), ('milik', -0.016997532942916765), ('atur', -0.010606111048735985), ('kena', 0.003000919089855054)], [('kaya', 0.08567855001428949), ('rakyat', 0.0778192450343582), ('wajib', -0.05077677592375678), ('masyarakat', -0.03646148377683036), ('pandora', 0.02989175620780194), ('aset', 0.02051548150743954), ('papers', 0.01861223317357098), ('beban', 0.01688037649190734), ('apes', 0.014744439905326588), ('sisi', 0.013262058166448404), ('orang', 0.013105116125930271), ('pajak', 0.012680157594215667), ('opini', -0.008895521312400461), ('laku', 0.005954401984243823), ('jelata', 0.00489934027331047), ('sembunyi', 0.001209220950009668), ('hindar', 0.0012023255973279685)], [('koruptor', 0.007758044407111397), ('rezim', 0.005424980392996892), ('bangsat', 0.005337439896191718), ('oligarki', 0.004495292412858034), ('hamba', 0.003509232000966586), ('ni', 0.0022460493197588305), ('jongos', 0.0015681918861842525), ('jahanam', 0.0015483444906858046), ('lindung', 0.0015366820944297391), ('china', 0.001375584661490751), ('anggur', 0.0012371082742343212), ('gilir', 0.0008662342612402062), ('usaha', -0.0006928363434442062)], [('pimpin', -0.045712982549722336), ('picu', -0.022971020016768266), ('menteri', -0.020954965048950818), ('nyata', -0.0197370057128468), ('alas', -0.016900635034829806), ('rencana', -0.010791691580681828), ('nilai', -0.00519795730256114), ('undang', 0.004248978959084177), ('pajak', 0.003145402350068552), ('pulih', 0.002206072725725133), ('ekonomi', 0.001990379443643798), ('dukung', 0.0019598520124980098), ('uu', -0.0018155011806039332), ('lontar', 0.0016205431103652006), ('mangkir', 0.0015762335618035813), ('kumolo', 0.001567546455550322), ('tjahajo', 0.0015274770408171335), ('sang', 0.0013022486959607454), ('muncul', 0.0011724492054268753), ('kpk', 0.001146937992732393), ('panggil', 0.001132286676932121), ('hati', 0.0011225958287999862), ('negatif', 0.0010505891477070903)], [('rakyat', 0.0800864111819155), ('kaya', 0.03596263402824865), ('harta', -0.017334126612453805), ('cukong', 0.016320765462836926), ('oligarki', 0.015927153506925824), ('opini', -0.0154113958491704), ('das', 0.01395994691298961), ('hidup', 0.012351314522560277), ('bentuk', 0.010359673743908445), ('zalim', 0.009455358676013884), ('bayar', -0.008175879784514166), ('bangun', 0.007758725476653165), ('nikmat', -0.007700172054913464), ('taxamnesty', -0.007550336072271964), ('bijak', 0.005740016506185214), ('kuasa', 0.0038900018511888743), ('pajak', 0.0033975568387941612), ('sulit', -0.0007572647533837045)], [('paksa', 0.02535497143415932), ('masyarakat', -0.02367824853482417), ('mempan', 0.018750143754499966), ('pajak', 0.016559534305315704), ('bayar', -0.015760589308125555), ('uganda', 0.013781618107928346), ('nunggu', 0.011653663214800786), ('entar', 0.010473558194151937), ('sabar', 0.009373353392800736), ('negara', 0.007411315626152467), ('sesat', 0.007119377072753889), ('alir', 0.006801761213837777), ('penjara', 0.004826257037698702), ('banget', 0.004342794738846698), ('ga', -0.004139182469574211), ('perintah', -0.003568422421377498), ('kiri', 0.0029778559398138623)], [('kemplang', 0.03025567256318805), ('rakyat', 0.026152431679095072), ('kaya', 0.017390195328801195), ('biar', -0.015635020064413988), ('fakta', -0.007871298443360326), ('sembunyi', -0.007672646643749533), ('pajak', -0.007279881619722264), ('uang', 0.006963924045387312), ('kena', -0.0032311504204449427), ('pinjol', 0.003088893033617302), ('debt', 0.002735191138607186), ('kolektor', 0.0027340812402570328), ('radikal', 0.0024254372277635627), ('liar', 0.0022145011574298498), ('putar', 0.002112354816695758), ('buzzerrp', 0.0017508446524296434), ('gelap', 0.0014658418390862218), ('rampok', 0.000808440632697108), ('nggak', -0.00034027477944348746)], [('uuhpp', -0.27522876382636974), ('wajib', -0.20467601357456705), ('badan', -0.11638313623561411), ('tarif', -0.10892917581769493), ('pph', -0.10887267101567262), ('turun', -0.037763667686426756), ('bijak', -0.027362797212210563), ('pajak', -0.025567021176453007), ('waji', -0.0195899555474404)], [('indonesia', -0.06960611429728206), ('kemplang', 0.06802453726473523), ('enak', 0.01768523625708452), ('pidana', 0.01319442550884951), ('speechless', 0.01239080198885893), ('sanksi', 0.010155103553244489), ('ringan', 0.005910053904439742), ('pajak', 0.0011267701106146752)], [('efektif', -0.030864426533082984), ('aja', 0.01410653158168449), ('kali', 0.010620042367392678), ('mesti', 0.006009586377755847), ('amnesti', 0.005934432274447912), ('modus', 0.005345613673458439), ('nama', 0.0050890598104546035), ('kalo', 0.0038829651478609565), ('pajak', 0.0036085216140605485)], [('pajak', -0.03389286569930505), ('kaya', 0.02641219276775831), ('negara', -0.015160994007034275), ('kas', 0.009304009374755423), ('kaum', 0.008930038402816849), ('kecuali', 0.007922469473899388), ('mungut', 0.0074401393413986204), ('kritis', 0.007036127022555494), ('muslim', 0.0064336649562265966), ('pungut', 0.00359342769055174)], [('pajakperasrakyat', 0.0266034486450673), ('kembang', -0.02330836747982614), ('beban', -0.011572846029986844), ('aja', 0.010514541447294612), ('sapa', 0.0072711374265339536), ('pajak', -0.005908758968652292), ('kempis', 0.0058922040956581245)], [('wajib', -0.10064554220527666), ('dzimmi', 0.053009256002256706), ('kaya', 0.04640264048713133), ('pungut', 0.02400542458357683), ('bayar', -0.02242575804826413), ('balig', 0.017317520364665006), ('kaum', 0.0168073437959197), ('jizyah', 0.015288253244909369), ('kas', 0.014084870603978752), ('laki', 0.013139359968617727), ('muslim', 0.01256385749783352), ('kritis', 0.012341468747008333), ('kecuali', 0.0118383651500015), ('miskin', 0.011204203749649875), ('mungut', 0.006275626967072199), ('pajak', -0.0035927605367234434), ('negara', 0.003364958213443537)], [('jokowi', 0.0051711118280083545), ('puja', 0.002429505292197836), ('nya', 0.0023260606973160864), ('baris', 0.0023193957938197705), ('mikir', 0.0022703150873812175), ('tonjol', 0.0021144693361630725), ('goblok', 0.0018774238901325534), ('pikir', 0.001633217399172018), ('gak', 0.0008050501019594662)], [('rakyat', 0.01706422532295502), ('dpr', -0.0033464377644895856), ('tuh', 0.0017266361116275639), ('mental', 0.0015404695840573995), ('bong', -0.0006871635768249119), ('krupuk', -0.000620380888559946), ('keujanan', -0.0005534666146894588), ('yaahh', -0.000497193375064004), ('wakli', -0.00023262578570266187), ('kacrut', 0.00019262765316076797)], [('uuhpp', -0.27778843632356953), ('wajib', -0.20349333773915615), ('badan', -0.11998461313136435), ('tarif', -0.11682290371661457), ('pph', -0.10466580024848415), ('turun', -0.03941322078208154), ('pajak', -0.022519436033517966), ('bijak', -0.018294151600831716)], [('rp', -0.116891506577503), ('negara', 0.032306562778665705), ('bebas', -0.03230616264206155), ('rugi', 0.030841653812421432), ('ya', 0.02809770941283677), ('hapus', 0.02726785677651472), ('bara', 0.024937877998641417), ('holiday', 0.023568567250659134), ('triliun', -0.022688575614176164), ('gede', 0.02104800197315178), ('batu', 0.02025237564198297), ('banget', 0.019658123996953623), ('tega', 0.01909161176843994), ('royalti', 0.018369790221983904), ('tax', 0.01676562148551432), ('pajak', 0.011676679919594996)], [('maksa', 0.0282608175647864), ('prokes', 0.011952800346863746), ('vaksin', 0.011386371216789806), ('gak', -0.006351277523701798), ('maling', 0.005226361747509525), ('pajak', 0.0028453600568387506), ('ngemplang', 0.0022442220378601297), ('mlh', 0.001850207673260834)], [('masyarakat', -0.08826646866222897), ('lo', 0.06162065153247872), ('denial', 0.0172289790731792), ('benah', 0.01694564253175209), ('pajak', 0.015849236012703757), ('institusi', 0.014669299674101098), ('negara', 0.013549504652756903), ('kritik', 0.013526464326193812), ('bangkrut', 0.013076565917184004), ('mentang', 0.012552370356270406), ('gak', 0.009448402215504718), ('gaji', 0.006167934833193266)], [('wajib', -0.0961812989156836), ('kemplang', 0.08100246295354506), ('badan', -0.04958340711418904), ('kaya', 0.046893220778660136), ('uang', 0.022813766095273715), ('sia', 0.01617990849144122), ('bayar', 0.014260709187977645), ('amanah', 0.013859350290811593), ('orang', 0.011030703776747518), ('korupsi', 0.00959176198163245), ('sosial', 0.009482569666800042), ('pintar', 0.00810188119547942), ('perintah', -0.004043707135018239), ('agama', 0.002170645792037868), ('pajak', -0.0004252955294474057)], [('pajakperasrakyat', 0.03897630439127431), ('sulit', -0.01704031386041731), ('tani', 0.016459283752024007), ('kena', -0.0075986124470513935), ('hidup', 0.005447836378831617), ('nilai', 0.004189236636123009), ('produk', 0.002221824395165346), ('pajak', 0.0016444286889992588)], [('orang', -0.008483897847908182), ('uang', 0.007701633747593501), ('eh', 0.0044839966565231254), ('sembah', 0.0025083123996044415), ('trilyunan', 0.002485251315545099), ('dusta', 0.002175385457097634), ('rebu', 0.0016411395589016396), ('boong', 0.0014031670714802983), ('tukang', 0.0013899787641881867), ('siluman', 0.0006951928815997359), ('gaib', 0.0005680551676371328), ('dukung', -0.0005444039609497338), ('mobil', 0.0005060276894568293), ('si', 0.00046089782383172196), ('nuduh', 0.00042139812796671927)], [('imbang', -0.26259415207373576), ('moga', -0.20745178696342306), ('rakyat', -0.14105801332388831), ('pajak', -0.046635552290599305)], [('rela', -0.11812743461384226), ('sistem', -0.08079075713589123), ('rakyat', 0.05879221276591606), ('wujud', -0.023375993817570273), ('gagal', 0.01286746856389516), ('temba', 0.011951719354355093), ('hadir', 0.011586186976713724), ('pemberientah', 0.011306615008478006), ('tuntut', 0.010188502045094064), ('perah', 0.009476288769589761), ('bobrok', 0.008373549264223129), ('hak', 0.008251606876956658), ('malak', 0.008191580836750806), ('serasa', 0.008101891321718603), ('rekayasa', 0.0077231511256586685), ('revolusi', 0.00770512967575185), ('sapi', 0.007342257585619278), ('goblok', 0.006609608807011299), ('basa', 0.006110594047739371), ('pasrah', 0.0052142161390518315), ('basi', 0.005109279180812807), ('pajak', 0.004531755348336833), ('bilang', 0.004217584687346295), ('korupsi', 0.0036775737172819443), ('bangsa', 0.0035128582122685684)], [('aja', 0.016996660144398466), ('rakyat', 0.012284905402413643), ('tenang', 0.008589661058172476), ('ko', 0.008472152255332452), ('nanggung', 0.008303658104512978), ('pajak', -0.005599739537578237)], [('rakyat', 0.10918933805519192), ('bayar', -0.09450220586406326), ('negara', -0.06567785189291962), ('hutang', -0.06374717906354459), ('sengaja', 0.054835105772792686), ('bantu', 0.04669272626382438), ('pajak', -0.04532629917634873), ('korupsi', 0.03112355617139511)], [('pimpin', -0.039289348964801414), ('nelayan', 0.02615431623027254), ('oligarki', 0.019578823589166464), ('tinggal', -0.017496810739645544), ('rezim', 0.01460091098331444), ('tampil', 0.004226245254661283), ('lahir', 0.004086484657702196), ('usung', 0.0009824486017502257)], [('hindar', -0.048041108489949386), ('kaya', 0.04305580671879576), ('bingung', 0.021941373289503713), ('pajak', 0.0024014738554576807)], [('bayar', -0.19380678639726018), ('luas', -0.18571975925260065), ('sektor', -0.1824277485160201), ('hutang', -0.13839108220770296), ('negara', -0.11863443681085255), ('pajak', -0.10050418195477993)], [('karbon', -0.25371146489377394), ('ri', -0.2349745445421918), ('iklim', -0.09794576944733305), ('dunia', -0.07716523126802512), ('ubah', -0.07221685572694271), ('juara', -0.05237510301932845), ('kendali', -0.048801063344337405), ('nilai', 0.023154928205363606), ('pajak', -0.011621136450106791)], [('komisi', -0.1565430811512934), ('xi', -0.1512325087800214), ('tajir', -0.14186605494499074), ('diskon', -0.09969013683109747), ('nikmat', -0.09438974726305417), ('orang', -0.04141931285166044), ('pajak', -0.004954796324520019)], [('pajak', -0.02169948793013173), ('rakyat', 0.011209417122223243), ('bangsat', 0.006061994756997022), ('cukong', 0.005883628753559598), ('kasi', 0.005662882903083875), ('peres', 0.0056101961083259485), ('sisa', 0.005291435487335166), ('amnesti', 0.005178820195922769), ('land', 0.005058913916330582), ('taik', 0.004724931263626121), ('negara', 0.0041101100029237335), ('beda', 0.0036966522239647716), ('habis', 0.002443160140215828)], [('rakyat', 0.05884925508172825), ('kena', -0.025442828118496356), ('harga', -0.016900004136161453), ('atur', -0.012099684052599561), ('kontra', 0.010681180231767373), ('mahal', 0.010176422438192143), ('paj', 0.009055497107441628), ('pro', 0.008907387170538596), ('barang', 0.0034745250621568925), ('pajak', 0.0034011435912130595)], [('pajakperasrakyat', 0.0682087755545214), ('bayar', -0.035363575603402815), ('rakyat', 0.029799583446243463), ('jajah', 0.010726779429440622), ('upeti', 0.009608157624241094), ('pajak', -0.007830315413396016), ('nama', 0.005508961733510135)], [('kuat', -0.17519631252427043), ('rakyat', 0.043094480633749314), ('biar', -0.018301450707024172), ('utang', 0.01588745135229361), ('menteri', -0.009412364490165939), ('uang', 0.005869809071753839), ('pajak', 0.003377921012347098), ('ya', 0.0019671251434634127), ('kali', 0.0011509788219177732), ('imun', 0.0005208685174056957), ('bicara', 0.0004955374209312368), ('terang', 0.0004903088034835894), ('stabil', 0.00040433220283607837), ('muncul', 0.00037023808213828034), ('kek', 0.0003642195426242135), ('media', 0.00032709050346440275), ('bangga', 0.00030037013115147146), ('lihat', 0.00025682591182625954), ('bosan', 5.6540864456170706e-05), ('senang', -4.2275487321583685e-05), ('muka', 2.6098596167141593e-05)], [('pajakperasrakyat', 0.018798233298938885), ('rezim', 0.004587603610782307), ('nguber', 0.004017590315026443), ('koruptor', 0.0036623174795546256), ('tukang', 0.0028726590142627295), ('setor', 0.0027498161004708022), ('palak', 0.002441238622377655), ('pajak', 1.0382431291969235e-05)], [('ngga', -0.14185776181106116), ('temen', -0.11472913594908982), ('yaa', -0.11077520289963032), ('bayar', -0.09895761192175513), ('ktp', -0.09021914312608843), ('pajak', -0.04136108217012344)], [('integrasi', -0.13012531983778708), ('uuhpp', -0.10848549670818217), ('data', -0.07950940397047229), ('nik', -0.07142331530723821), ('duduk', -0.06765099601152301), ('administrasi', -0.06409718739379396), ('npwp', -0.05959224612873586), ('sistem', -0.052237219414690755), ('basis', -0.04741818951367746), ('pajak', -0.02230629270376524)], [('maju', -0.1468605561785238), ('paja', -0.11102542180959925), ('kuat', -0.10598101417500266), ('administrasi', -0.09691488736332822), ('sistem', -0.08315974667274317), ('harap', -0.053537422689582025), ('tingkat', -0.04974494657027093), ('informasi', -0.03217700020850113), ('terima', -0.02414795627323951), ('digital', -0.02121058040238994), ('atur', -0.020162353056141946), ('harmonisasi', -0.018196972374525902), ('teknologi', -0.016282943264351424), ('uu', -0.010689041012153386), ('pajak', -0.0022623771753106973)], [('terap', -0.12082539364336804), ('minimum', -0.09723265163303256), ('khawatir', -0.09175851301073286), ('hindar', -0.08784888897903922), ('hasil', -0.08216386941516697), ('eksesif', -0.06360222145466614), ('pajak', -0.0546378610326981), ('norma', -0.04532657731066143), ('amt', -0.043644002686103), ('berkat', -0.043205159296519545), ('alternatif', -0.03962537217801303), ('multitarif', -0.03954076464239331), ('hapus', 0.014543633681223357), ('tambah', -0.013231797538975439), ('tax', -0.011702349813442614), ('nilai', 0.007165161894160176)], [('tentu', -0.20266846930240842), ('potensi', -0.1099018941979582), ('hindar', -0.10178674713762279), ('eksesif', -0.0738791744302243), ('jasa', -0.052626872994929304), ('pokok', -0.04323208701478155), ('asuransi', -0.02907684011908089), ('maja', -0.026762201964972025), ('medis', -0.026128406900218174), ('sehat', -0.025688660013136185), ('agama', -0.02531936817679923), ('didik', -0.025006884873428933), ('pajak', -0.01407228383934619), ('barang', -0.011445128591108491), ('butuh', -0.010827262768893125), ('pungut', -0.003766757372509986), ('uang', 0.001938529507215627)], [('wajib', -0.17304079125540328), ('cegah', -0.16516281293718546), ('tentu', -0.16402337988943333), ('khawatir', -0.08244164134344475), ('pajak', -0.03985342626108706), ('diesesuaikan', -0.035107631758107355), ('tunai', -0.028072669897816815), ('pasti', -0.027783378225776048), ('wenang', -0.023867728808475434), ('sidik', -0.020532337441434175), ('tari', -0.02051508501141548), ('hap', -0.020512227918044634), ('aman', -0.018098470377436632), ('agresif', -0.015396795298980744), ('hukum', -0.013858889452448377)], [('integrasi', -0.15367861858236279), ('tentu', -0.1406657748180788), ('paja', -0.10313701013173908), ('pph', -0.08291907729570672), ('ubah', -0.06597693245017021), ('tata', -0.0629351312469695), ('hasil', -0.06115365857767829), ('atur', -0.03609446279967212), ('jasa', -0.03144563852139671), ('pajak', -0.024690212057839417), ('cukai', -0.020715677217977343), ('mewah', -0.02002139870064618), ('uu', -0.018500229622368993), ('tambah', -0.016462572913077494), ('jual', -0.012070957064030559), ('nilai', -0.011887134212867696), ('barang', -0.007533869423956277), ('kait', -0.004983871918429574)], [('jalan', -0.04101362007736186), ('kaya', 0.03731847705686284), ('harta', -0.02864957773832138), ('cek', -0.025313573814975614), ('sembunyi', -0.021724032564467238), ('uang', 0.018964728438689304), ('tukar', -0.010273738820438605), ('pajak', 0.009183726357840045), ('ekonomi', 0.008993165624499724), ('iring', 0.006499335640618515), ('stuck', 0.006346798372880842), ('lapang', 0.005304881990887317), ('nilai', 0.002965790318712758), ('transparan', 0.0029656710177781488), ('bulu', 0.0025684249612400803), ('jelas', 0.002301018604479464), ('pulih', 0.0022857664596914185), ('susul', 0.0013339839668129547)], [('rp', -0.1153193531124855), ('negara', 0.03338840651860005), ('bebas', -0.03209236492693913), ('rugi', 0.031272245203862896), ('ya', 0.02577728921277278), ('holiday', 0.025458687248775695), ('hapus', 0.025396161753802923), ('bara', 0.022877388976895063), ('batu', 0.02131678357246688), ('triliun', -0.02112747203119971), ('banget', 0.02097272465131096), ('gede', 0.019876822583482452), ('tega', 0.01882977103530848), ('royalti', 0.016719168974577828), ('tax', 0.01597169879531341), ('pajak', 0.01120217527532979)], [('hasil', -0.039785124632289774), ('rezim', 0.013729324066492141), ('dungu', 0.011132181533949272), ('pajak', 0.006647343404367617), ('citra', 0.006090615859660074), ('upeti', 0.005391824699341305), ('hutangpajak', 0.005259971775897861), ('bacot', 0.005244153835632629), ('anjing', 0.005076109755492175), ('hutang', -0.004498061724205615), ('gayarezim', 0.0034676340072148063)], [('pilih', -0.07498216615871595), ('duliu', 0.032188675724405216), ('sumbat', 0.03211538211326326), ('inkonsistensi', 0.025348982609480278)], [('rakyat', 0.09601767718367736), ('sri', -0.033811213477342505), ('mulyani', -0.03379558145744734), ('hutang', -0.018776565845622074), ('jurus', 0.018326826229328092), ('nambah', 0.014674186497709813), ('cari', 0.012964891371655697), ('beban', 0.009854405019111553), ('pajak', -0.008735932046242135)], [('negara', 0.010599780004305217), ('pemgecuttt', 0.008069677639184154), ('ama', 0.006870249762960015), ('parelmenmya', 0.00673471854880266), ('oramg', 0.006543799555227263), ('beda', 0.005553993670665575), ('isi', 0.00492152166422529)], [('pilih', -0.04012789052874363), ('hasil', -0.036331032149026804), ('sumbat', 0.015661175950663152), ('pajak', -0.01350098920959376), ('kendaran', 0.013073619885943659), ('manis', 0.012818382432470098), ('inkonsistensi', 0.012772139569543288), ('kenatipupks', 0.012634405725837052), ('hapus', 0.01180987452114126), ('kendara', 0.011632040594161666), ('duliu', 0.010646553530617277), ('bismilah', 0.009470266560639823), ('nya', 0.009374362410982653), ('bapa', 0.009104838663381448), ('kait', 0.0090583876517952), ('janji', -0.006635567427274133)], [('rakyat', 0.051154456760134734), ('bayar', -0.02319722713913038), ('anjim', 0.02107441963602751), ('jabat', 0.020616955234168247), ('pajak', -0.014015867752191533), ('dah', 0.013269016557435119), ('banget', 0.011093194068418535), ('uber', 0.011059764637200363), ('ngemplang', 0.010994122783199747)], [('umkm', -0.27214346179837984), ('rp', -0.13145453344762556), ('bruto', -0.08595086979828961), ('syarat', -0.07575647369348375), ('laku', -0.06837888565529038), ('hasil', -0.055018090775835284), ('juta', -0.034017043261649116), ('edar', -0.027210796965023744), ('ptkp', -0.021474745672167415), ('pajak', -0.015425316343049951), ('bayar', -0.01419322917697015), ('ya', 0.013708956164641515), ('orang', -0.011096886970973014), ('kena', -0.0007644310907371752)], [('integrasi', -0.05199187479084032), ('uuhpp', -0.045844870191195326), ('reformasi', -0.04371589610707072), ('data', -0.03842655654393635), ('duduk', -0.031563776181906845), ('wajib', -0.030860101585644957), ('administrasi', -0.028786882305053114), ('nik', -0.027998844789720975), ('mudah', -0.023483966964304118), ('kuat', -0.021823897719567435), ('npwp', -0.021006205515612542), ('basis', -0.019656167577368575), ('paja', -0.017760196337201366), ('sistem', -0.0170366369235267), ('pajak', -0.013279620649722346), ('bijak', 0.004893937094794722), ('masyarakat', -0.0028769284157150227), ('layan', -0.0021522739862498063), ('publik', 0.0008539368777067648)], [('wajib', -0.16243429614969818), ('dorong', -0.15746698504526463), ('adil', -0.15440427201932394), ('badan', -0.08003822156977432), ('pandemi', -0.057705881710841685), ('tarif', -0.0565393655842203), ('turun', -0.03195931283981501), ('pajak', -0.022923426031590186), ('akselerasi', -0.016967510663449724), ('dampak', -0.015578748321468756)], [('jilid', -0.03161010386510084), ('pajak', 0.024463065792261363), ('utang', 0.024185854957536935), ('kabar', 0.011109362850064563), ('gunung', 0.008484653984167914), ('buruk', 0.007928427265591427), ('tka', 0.007563363320882175), ('dibanyakin', 0.0062151219283240675), ('gede', 0.005874907007691336), ('ngomong', 0.005287204257597746), ('ppkm', 0.0049642902236474995)], [('rakyat', 0.07748140174582847), ('hutang', -0.04179964065573452), ('pajak', -0.04086789621934778), ('susah', 0.004039382532672336)], [('indonesia', -0.10408513735540066), ('laut', 0.028776097182712977), ('nelayan', 0.028298263910715014), ('nelyan', 0.012426388924737696), ('air', 0.010624689495809149), ('cina', 0.009083133174232072), ('beban', 0.007993133164873208), ('pajak', -0.0003719009832246582)], [('negara', 0.02182954678378711), ('gelar', -0.019670052180506485), ('nelayan', 0.011053645448760205), ('biaya', 0.007900124942030731), ('ratus', 0.007171403147352274), ('jawa', 0.006932047938804512), ('kibar', 0.004712965003610417), ('terima', -0.004278364035738751), ('aksi', 0.00421290622240381), ('pati', 0.004116027317990395), ('bendera', 0.004035726569057473), ('putih', 0.0033554235327813682)], [('reformasi', -0.2950061674005943), ('karbon', -0.18689300578346127), ('paja', -0.13578582772448242), ('pandemi', -0.07512218116625761), ('strategis', -0.03358972410758225), ('pajak', -0.028816974359537003), ('kenal', -0.0264915030255645), ('tangan', -0.02615561319883424), ('bijak', 0.007016798261538211)], [('uuhpp', -0.27603534715436934), ('wajib', -0.2066247850319531), ('badan', -0.12192640142350054), ('tarif', -0.11218144688530579), ('pph', -0.107323596743457), ('turun', -0.05378523250851564), ('pajak', -0.029563662966459733), ('bijak', -0.027792024137689397)], [('reformasi', -0.19094333085407791), ('paja', -0.08325667004373925), ('kuat', -0.08048921680543857), ('masyarakat', -0.07970699348927973), ('basis', -0.06337074556529786), ('apbn', 0.027710728886354397), ('stabilitas', -0.025085046258488185), ('sejahtera', 0.016850304833064794), ('ekonomi', 0.01473113735936058), ('tingkat', -0.006049012549460808), ('mantap', 0.0052094039636798335), ('pajak', 0.002506319745666094), ('sehat', -0.0015997908507424004)], [('reformasi', -0.22759695170535454), ('uuhpp', -0.1993766766107582), ('paja', -0.08202524946592213), ('pajak', -0.0310423101056143), ('tingkat', 0.01949015274763981), ('laku', 0.005671566300186623), ('estimasi', 0.002856618009421385), ('terima', 0.0014018016726103579), ('rasio', 0.0007492308400807282)], [('strategi', -0.05210497614734446), ('ppn', -0.04040726555351011), ('beli', 0.0204516913677641), ('uang', 0.012591508729990434), ('harga', -0.011871940248115054), ('pajak', 0.011388415903728522), ('yak', 0.00838294050344648), ('tu', 0.00830418654749236), ('negara', 0.007938292993168903), ('si', 0.007679288863414905), ('kemarin', 0.007612806798667968), ('arah', 0.006839320282026514), ('potong', 0.006728465158223948), ('dinaikin', 0.006379815935155729), ('mobil', 0.006107139482361628), ('mana', 0.005951817471956211), ('gaji', 0.004113429229205059), ('udah', 0.003932212336657366), ('hilang', 0.003323970485954733), ('barang', 1.1504047057846003e-05)], [('pajak', -0.06103614350059763), ('jabat', 0.017368223097387676), ('ngemplang', 0.0036474877056662095), ('perkara', -0.0016126167227235998)], [('bayar', -0.13941227652838784), ('stop', 0.08282851269278399), ('pajak', -0.05823568527618041)], [('rakyat', 0.013804093494812585), ('pajakperasrakyat', 0.009249621665355177), ('beban', -0.0061151416833356235), ('utang', 0.004229249281981247), ('mafia', 0.0016315880142769219), ('tanggung', 0.0009000554321641543)], [('sektor', -0.07882699978192181), ('kaya', 0.0339192772368001), ('aja', 0.02220544338541703), ('hasil', -0.021995254838412352), ('orang', -0.007134187739234897), ('saat', 0.006437894884409512), ('juang', 0.006087556411490951), ('ya', 0.006025896436218888), ('persentase', 0.0057338335515543844), ('gue', 0.0056599589484161595), ('perintah', -0.0056295415409201655), ('gin', 0.005538477668363274), ('miris', 0.004795992654325117), ('dekat', 0.004786402800068963), ('putar', 0.003553552031761115), ('singgung', 0.002899289218346121), ('naikin', 0.002676287871040265), ('gitu', 0.002243307378849744), ('pajak', 0.00148634622891295), ('swasta', 0.0012121663284420656), ('negara', 0.0007586209370369085), ('ekonomi', 0.000686604016788456)], [('rakyat', 0.02291333621533934), ('pajak', -0.015152864142695227), ('ya', 0.009757263603695963), ('rendah', -0.008314681441712195), ('bangsa', 0.00525216268060508)], [('rakyat', 0.0420564121995368), ('bayar', -0.031029882171393262), ('aja', 0.023994856358372833), ('liat', 0.019774439747086415), ('gampang', 0.019528220298945253), ('ngutang', 0.014741676305040353), ('tuh', 0.013567258661558173), ('pajak', 0.001996995681758934)], [('rakyat', 0.027029968395630544), ('ngelunjak', 0.01406293464209778), ('gak', -0.009343468917742903), ('pake', 0.008292629083570388), ('gaji', 0.0077758470219062865), ('tau', 0.007424992821556811), ('pajak', -0.006926897597208689)], [('kismin', 0.04059437660199858), ('perkosa', 0.039778624119806046), ('bayar', -0.031611580123037185), ('oknum', 0.02859087136558831), ('pajak', 0.021242662577443897), ('gaji', 0.021240763153299626), ('orang', -0.009298548026622314), ('bela', -0.007325355593760791)], [('utang', 0.026943456051633643), ('kreatif', 0.02375376698120626), ('perintah', -0.01756727201787703), ('cari', 0.01530535464488299), ('pajak', -0.009493364242663256)], [('adil', -0.11411817163882768), ('pajakperasrakyat', 0.06846088114027082), ('rakyat', 0.05179033531128991), ('embat', -0.042441630529186244), ('makan', 0.018829391109020467), ('bijak', 0.018367993596955533), ('orang', -0.00893660927602461), ('depan', 0.00878413373037795), ('koruptor', 0.007242656475746509), ('busuk', 0.006950303735672654), ('pajak', 0.006074519193796669), ('prinsip', 0.0049578393229690295), ('maling', 0.003849497773440993), ('rakus', 0.003817528906489569), ('taat', 0.0017804718351841302), ('beban', 0.001744878199881946)], [('jenius', -0.0672540824869941), ('kejar', 0.06025121673243406), ('rakyat', 0.058723692697583596), ('tull', 0.030817941113566897), ('sampe', 0.026128674609556608), ('iya', 0.020799757589866325), ('pajak', 0.01670473791186478), ('banget', 0.01526424286919588)], [('pajak', -0.02170741671693513), ('haram', 0.013747021640581508), ('saw', 0.012244625624229215), ('rasulullah', 0.012026731892601603), ('islami', 0.011330160423624938), ('laknat', 0.010953951543066633), ('cela', 0.01054743121715093), ('keras', 0.00949454693347646), ('perintah', -0.006495344185377378)], [('rakyat', 0.13669665941828074), ('milik', 0.03336825553176774), ('nik', -0.019646909739051942), ('pajak', 0.011221458964109402), ('ktp', 0.009671175670845456), ('kemenkeu', -0.00708116226351323), ('beban', 0.005876958070613675), ('orang', -0.005857194410409871), ('npwp', -0.005567784596839474), ('tarif', 0.005322774924262343), ('ppn', 0.0019402752832548304), ('bayar', 0.001228014345007829), ('bilang', 0.0011867681249628)], [('wajib', -0.20999467432483268), ('integrasi', -0.20168593051298267), ('data', -0.13439644996500563), ('mudah', -0.11688125942281914), ('administrasi', -0.11684302042505916), ('pajak', -0.041928898182335646)], [('rakyat', 0.014369532194429017), ('ironis', 0.013337066779546306), ('alangkah', 0.010542736045887359), ('indoensia', 0.010299761836045945), ('kejar', 0.004689507648156953), ('pajak', -5.301998558419943e-05)], [('tanda', 0.022655450943835084), ('perintah', -0.020434914334117645), ('hancur', 0.011679658551443753), ('ibnu', 0.007043486507975266), ('khaldun', 0.006773876945077617), ('pungut', 0.006734514530362437), ('pajak', 0.001108550149173428)], [('hapus', 0.009364634287991102), ('nelayan', 0.008577808758555952), ('batu', 0.007243367338891252), ('royalti', 0.006221889024840924), ('oligarki', 0.004629226888530929), ('rizalramli', 0.004299925474133605), ('bara', 0.004137657595136803), ('untung', -0.0038941145342284915), ('persen', -0.0025626188594140833), ('pajak', -0.0018810559781823075)], [('jilid', -0.0364650325413594), ('utang', 0.026979395454537636), ('pajak', 0.023908300694239465), ('kabar', 0.016690115469201953), ('tka', 0.011823706196648692), ('buruk', 0.011163495898193636), ('gunung', 0.010039629912339382), ('ngomong', 0.008255700752639032), ('ppkm', 0.008119243725687012), ('gede', 0.0077803240679132505)], [('wajib', -0.2270035037892133), ('tuju', -0.13838682767709048), ('wamenkeu', -0.1355602704803509), ('patuh', -0.10206331652114176), ('tingkat', -0.0785140307612468), ('pps', -0.07619393208836132), ('utama', -0.0744592455365916), ('pajak', -0.009582370610063602)], [('atur', -0.035602516246041396), ('kolusi', 0.010675994321851377), ('pajak', 0.0103485414964842), ('patgulipat', 0.010253434263252311), ('kuasa', -0.006568648236849996), ('usaha', -0.005710347802365496)], [('cegah', -0.25628957778093725), ('tuju', -0.23112002260542583), ('taxamnesty', -0.1364648593355541), ('hindar', -0.09664217122210758), ('jilidii', -0.07147074451101053), ('pajak', -0.038866931043532685)], [('doang', -0.1457246512751815), ('rakyat', -0.14146305143266483), ('petinggi', -0.10450495544551688), ('embat', -0.10302557445229367), ('gapapa', -0.08795472369491929), ('mah', -0.08578342986973726), ('pajak', -0.07245840878128848), ('nikmat', -0.04973286063358654), ('jelata', -0.04828382945591484), ('sengsara', -0.044496625701621655), ('udah', -0.029389623958895588), ('jabat', 0.02781049033000003)], [('rakyat', 0.07815698247631107), ('kaya', 0.02422831545171248), ('harta', -0.01878979752493151), ('cukong', 0.016640935690411595), ('oligarki', 0.016022700057653695), ('das', 0.015857315642795227), ('taxamnesty', -0.013864472552563265), ('bentuk', 0.01294356758729159), ('hidup', 0.012818869272847718), ('zalim', 0.012249900880502365), ('bayar', -0.01172595815592409), ('nikmat', -0.010269556318513343), ('bangun', 0.008599587540640254), ('bijak', 0.006356514313295867), ('kuasa', 0.006261795609462312), ('pajak', 0.0018602541355416595), ('sulit', -0.0012319461099468694)], [('bebas', -0.07868954507816224), ('nelayan', 0.034488514843102405), ('banget', 0.02787210177873502), ('ya', 0.026657892800812502), ('tega', 0.02650875736690909), ('rizalramli', 0.022888965875451105), ('pajak', 0.021172625296197706), ('gede', 0.020940287617598984), ('persen', 0.018845919435280564), ('usaha', 0.0108721025649351)], [('harga', -0.029847732828912527), ('mobile', 0.008800679846280818), ('copras', 0.008275773403909375), ('got', 0.008021624283458439), ('dusta', 0.006906636497555817), ('esemkah', 0.006620598176311814), ('caperes', 0.0063518014119105575), ('rebubtrlyun', 0.006265023209955449), ('cocot', 0.006004477046054854), ('gak', 0.005926243784622199), ('mikir', 0.005719621315227458), ('masuk', 0.005115715239395026)], [('juta', -0.014516759458309362), ('sogok', 0.013160610175675358), ('ya', 0.011358342038908634), ('gampang', 0.01124358536508838), ('entar', 0.009886448647478518), ('udah', 0.003617210948906095), ('pajak', -0.003282709366813888)], [('tingkat', -0.028062791703128045), ('pajakperasrakyat', 0.02779591437751333), ('rakyat', 0.023427808990530815), ('terima', -0.016072481622178267), ('negara', 0.00731427776435408), ('peras', 0.006555517374810049)], [('moga', -0.08547294093508473), ('boong', 0.017502056003773966), ('jujur', 0.015199054433429464), ('rakyat', -0.015126886134444462), ('ngalahin', 0.014389174244228807), ('kasi', 0.014291171768600135), ('diboongin', 0.013815838568512453), ('gunung', 0.01368941982768357), ('dosa', 0.013340035817444673), ('uhud', 0.013061452503273537), ('juta', -0.0007902989480177297)], [('pajak', -0.03298642315770239), ('lieur', 0.02466899564737532), ('bikin', 0.011443197657397087)], [('pimpin', -0.07656971133574204), ('hindar', -0.0568559697207465), ('urus', -0.04663371995002894), ('jabat', 0.02995510947318095), ('bayar', -0.010165530490728238), ('rakyat', 0.008210232444254967), ('pajak', -0.007533724646823655), ('woy', 0.00472103234915484), ('negara', 0.004484281741173102), ('ikut', 0.004117067372734614), ('pindah', 0.0036534334734600387), ('cari', 0.0031725089306455755), ('dosa', 0.0031153868179852567), ('korup', 0.0030466267432934444)], [('ga', -0.040968093481033356), ('dipalakin', 0.03018636805976007), ('kerja', 0.029708560171195518), ('lapang', 0.026814898692777502), ('bikin', 0.021402194478507604), ('pajak', -0.0073206826641678815), ('negara', -0.0016852300892082328)], [('kemplang', 0.03901231063526648), ('pajak', -0.02270117809315426), ('aja', 0.015114921837600683), ('bayar', -0.012068078839223244), ('darimana', 0.005929374750848528), ('duit', -0.004717447852646523), ('susah', 0.004573941526311568), ('bodo', 0.0045361300222704825), ('hidup', 0.004411574396287694), ('driver', 0.004027770421408428), ('miliar', 0.00353091087503284), ('online', 0.0035291349755154225), ('biaya', 0.0028969144462182967), ('kasih', 0.002315222253844513), ('ringan', -0.0010840968172301474)], [('rakyat', 0.006545672791495106), ('kemplang', 0.0063983465616000365), ('pajak', 0.006167084859509206), ('utang', 0.005731072146495429), ('aset', -0.004082452659430332), ('koruptor', 0.0019095570742341933), ('bencana', 0.0008934375376526418), ('produktif', 0.0007934521194161219), ('daya', 0.0007054820391151768), ('bangkit', 0.0006971859816663895), ('sita', 0.0006098639861507913), ('tagih', 0.0005589497284557655), ('dll', 0.0003541491422868098), ('ekonomi', 0.00015918413127791282)], [('sulit', -0.022727046178671363), ('jabat', 0.021714400278377274), ('rungkad', 0.013794037436644606), ('mangap', 0.010786370655428961), ('pajak', -0.008469591537965001), ('ekonomi', 0.007035635312687779), ('ngomong', 0.0054317230649226144)], [('tugas', -0.06877366483462732), ('ongkang', 0.06548439811924638), ('rumah', 0.058828586140844186), ('jenius', -0.03827372021811605), ('jokowi', 0.031472181082280413), ('koruptor', 0.01848875856873361), ('boong', 0.01834598117163303), ('habis', 0.016108262125509424), ('pajak', 0.009407716044309193)], [('bayar', -0.15614590599229222), ('negara', -0.03363707545018876), ('pajak', -0.027892552162942657), ('usaha', 0.017194015763862678), ('prioritas', 0.0135794546080096), ('kontribusi', 0.013098846014354487), ('kosong', 0.012470891782909749), ('bisnis', 0.012161285661265777), ('tu', 0.010702722264758832), ('omong', 0.010159746359674782), ('nih', 0.00856644596304445), ('dalih', 0.00851712027683955)], [('biar', -0.022909762395439976), ('aja', 0.016591347402908185), ('kaya', 0.011887913858140167), ('pajak', 0.005362448648435086), ('ekonomi', -0.001954006501069162), ('roket', 0.0018937487912332623), ('dikit', -0.0017919791316584048), ('bong', 0.0013768761803443204), ('ben', 0.0010968050130933209), ('dungu', -0.0004189374114986327), ('referensi', 0.0004112292086238626), ('cari', 0.0003810080944345132), ('digedein', -0.00036095361643788046), ('jing', 0.0003525384136036984), ('mainstream', 0.0003071115290962661), ('nelayanpetani', -0.00024781690867617), ('ngomong', -0.00023420099858434087), ('pinter', -0.0001878423790344796), ('netral', 0.00010436539612999069), ('isi', -9.649383613494955e-05), ('berita', 9.35424507225756e-05), ('otak', 8.638989770732852e-05), ('tel', -5.8333536060287236e-05), ('kubang', 3.4882048219170074e-05)], [('kaya', 0.04609678210204562), ('hindar', -0.0361947967141323), ('jabat', 0.025252274571193136), ('harta', -0.020507304009534155), ('uang', 0.01315376338311283), ('pajak', 0.009315738732620931), ('sembunyi', -0.009033095754028253), ('sedih', 0.0037261199147104232), ('revolusi', 0.003567530918631825), ('mental', 0.0024536670846990083), ('ramai', 0.002168621312517883), ('tambah', -0.0021343417311116284), ('bayar', 0.001755876219039026), ('negara', 0.0017107061451449325), ('mana', 0.0013534090594311873), ('angka', 0.001055405414537624), ('defisit', 0.0009662183401585762), ('ngawur', -0.0003934025546093517), ('miskin', 0.0003829311078842731), ('puruk', -0.00022546134889384268)], [('ppn', -0.0612474347076844), ('perintah', -0.01648909498425507), ('timbang', 0.008708966881793765), ('kalang', 0.006849272232520044), ('rencana', -0.0036654367851607986), ('usaha', 0.0008446042486772909)], [('kemplang', 0.1420800551529216), ('uuhpp', -0.11859824501451845), ('pidana', 0.03474991852066491), ('sanksi', 0.03294425544403957), ('dpr', 0.028835260929403025), ('atur', 0.022655858375950706), ('hilang', 0.022063478164037677), ('uu', 0.013318138974194475), ('harmonisasi', 0.013145446022039133), ('tuju', -0.00978672393526326), ('perintah', 0.007497769604944418), ('pajak', 0.00204685815979659)], [('rakyat', 0.09930606584928861), ('bayar', -0.05801161659164953), ('negara', -0.03292918277664627), ('pajak', -0.03288355118776731), ('malak', 0.03250935361437235), ('muslimahnews', 0.03153336093848744), ('ampun', 0.03146666934828806), ('ubah', -0.005312702106201744)], [('kena', -0.028408773977069486), ('oligharki', 0.017513174423724902), ('moral', 0.01607092518741054), ('konglohitam', 0.015877589435579435), ('panama', 0.01344995664483207), ('papers', 0.012573772749677593), ('nkri', 0.011119312582980202), ('ngemplang', 0.011017739230516476), ('pajak', 0.010932668833736702)], [('kemplang', 0.08689000435943496), ('indonesia', -0.07173250215824555), ('enak', 0.02135974525522429), ('nya', 0.014346053226551808), ('pokok', -0.0020884445226275463), ('pajak', 0.0006658509263357171)], [('pajakperasrakyat', 0.0980518296729125), ('wajib', -0.06514264906564779), ('rakyat', 0.05260274513249373), ('kaya', 0.0396273570500116), ('kabar', 0.01085451770613252), ('pajak', 0.010420838561713146), ('sembunyi', 0.004998725195773936), ('orang', -0.003985213921499029), ('bayar', -0.003704696400004642)], [('bijak', 0.035126550781525795), ('orang', 0.025353116603379477), ('makan', 0.02447327612925199), ('pajak', 0.024275488697902754), ('taat', -0.014484209242535136), ('ajak', 0.012016925574106073), ('jahat', 0.010835326876562659), ('gila', 0.0101880674267822), ('gak', 0.004097992471244569)], [('uuhpp', -0.17986240319981828), ('gotong', -0.06920412574443899), ('kemplang', 0.058630193389669925), ('rakyat', 0.030070229511860988), ('kaya', 0.025391369664002777), ('pajakperasrakyat', 0.015705715128822843), ('taxamnesty', 0.013956728901368008), ('konten', 0.009686910083139431), ('jabat', 0.009387270906424175), ('profesor', 0.00532329073331651), ('nasional', -0.004772329714690676), ('selasa', -0.003890202677483316), ('orang', -0.003649048448103341), ('rezim', 0.003297030405018474), ('kabinet', -0.0030755080600479642), ('seminar', 0.0028859822322863625), ('buta', -0.0014503397451523495), ('pajak', -0.0014318284563903925), ('baim', 0.0013139588223143176), ('kolega', -0.0012358505755327117), ('garong', -0.0009155953667025935), ('mustahil', -0.0008424937178294487), ('babi', -0.0005013105744598543), ('beban', 0.0004975791507280113)], [('kerja', -0.019225341453101837), ('malesan', 0.0186894118981035), ('kantong', 0.015502077385593099), ('pajak', 0.014681260489209067), ('kek', 0.013242282593767678), ('masuk', 0.013215132172442296), ('gitu', 0.011749058042130714)], [('pajakperasrakyat', 0.02770271886892756), ('lupa', -0.021377224818702353), ('pajak', 0.010569476603187668), ('iman', 0.0018323763008103113), ('palak', 0.0015390211736428904), ('sodaraku', 0.0006220337770144889), ('bpak', 0.0004870424270392997), ('bin', 0.00048414854955952804), ('juang', -0.00025402440016025405)], [('pajakperasrakyat', 0.07653807114610608), ('bayar', -0.04213895828619279), ('ujung', 0.02403940512942216), ('percumah', 0.022615961923349193), ('nya', 0.01354982246761424), ('korupsi', 0.013191626805439591), ('pajak', -0.0005846932349124049)], [('atur', -0.02994195622799159), ('hpp', -0.02927051699879315), ('fpks', 0.02332458656321542), ('batal', 0.02128564989182926), ('kup', 0.020852557480708703), ('sembako', 0.013564327149439956), ('cantum', 0.008795060431960742), ('uu', -0.008574398437397636), ('ppn', -0.005328223684922773)], [('gede', 0.05209211370312611), ('kaya', 0.03855860526736538), ('hutang', -0.03441177959018058), ('hiii', 0.03078024670170367), ('bayar', -0.030333702878709432), ('numpuk', 0.027837228152904122), ('cekik', 0.014799855047182814), ('negara', -0.01450544812649703), ('pajak', 0.004871737118036878)], [('kemplang', 0.022945605149197947), ('singkir', 0.010551797216410712), ('nangkap', 0.01038517946455604), ('pecat', 0.010170601491119348), ('pajak', 0.0016501415940006797)], [('uuhpp', -0.20170124799710448), ('pajakperasrakyat', 0.09088933335396301), ('rakyat', 0.04672508567301399), ('taxamnesty', 0.01911146261279701), ('klepon', 0.0164360445522834), ('profesor', 0.016139387460751574), ('dewe', 0.01422524858705586), ('konten', 0.01220312950468959), ('konco', 0.009626704090017217), ('baim', 0.008032147884949976), ('uber', 0.007299991636452936), ('seminar', 0.006890604907588908), ('prestasi', 0.006383796998839403), ('pajak', 0.0006617000427472613), ('nasional', -0.0003038001748250768), ('selasa', 5.9754797285603956e-05)], [('pajakperasrakyat', 0.19200425989330935), ('nik', -0.03661132554131955), ('bayar', -0.027519872989678954), ('milik', 0.025468468196154927), ('npwp', -0.015657011891185162), ('pajak', -0.012266030205533253), ('ktp', -0.0016732809765007265)], [('bayar', -0.027429983579972367), ('jabat', 0.023513702975038384), ('pandora', 0.01949584075419082), ('ogah', 0.018974633416672215), ('legislator', 0.018191623105487532), ('pks', 0.016073785420129848), ('papers', 0.015007188451625342), ('bukti', 0.014287325410443289), ('covid', -0.013182270664996191), ('pajak', 0.0059130410650334525)], [('kereta', 0.011267637559719037), ('janji', -0.010978810324086085), ('cepat', 0.00831921928648041), ('apbn', 0.006932553622580416), ('ga', -0.0059178893867606434), ('kampanye', 0.001970250916316669), ('cuman', 0.0018924221981923587), ('pake', 0.0017357887449826416), ('susah', 0.0015995240847408955), ('tepat', 0.0015563428673121028), ('listrik', 0.001458733565462613), ('boong', 0.0014084422646819291), ('lolos', 0.0011586889534866782), ('twk', 0.0011214726466341265), ('agenda', 0.0009008337002250782), ('tani', 0.0007778055048872349), ('kecebong', 0.0007397784272480856), ('impor', 0.0007032313668205538), ('pegawai', 0.0006859850517310745), ('masy', 0.0006370814891695298), ('nanggung', 0.0006303456721110482), ('garam', 0.0006249726488441359), ('statuta', 0.0005173860091980436), ('ternak', 0.0004864120927396852), ('panen', 0.0004838710997936159), ('jual', 0.00043922530375200343), ('pajak', 0.00033127911103678434), ('kpk', 0.0003252173001977039), ('omong', 0.0002777434825482788)], [('rakyat', 0.03509371250575445), ('tumbuh', -0.03136841486716337), ('sulit', -0.01452989060873979), ('bicara', 0.006138060777108801), ('apbn', 0.005572176463899225), ('ekonomi', 0.005316985328279568), ('kunci', 0.003929339211151929), ('pajak', 0.0024849616910145485), ('lockdown', 0.0024477082186146974), ('now', 0.0016826115695133639), ('anggar', 0.001370186080936209)], [('pajak', -0.037256534111485794), ('lesat', 0.025065343823285745), ('korupsi', -0.00619500128775094)], [('rakyat', 0.026547265299632448), ('harga', -0.01509559472912161), ('muji', 0.014272758195970552), ('mati', 0.008138067146106934), ('sengsara', -0.007110591704748881), ('cekik', 0.006302651672344625), ('pajaktuh', 0.00590131504589726), ('nkri', 0.003568091645654509), ('junjung', 0.0033209149091297024), ('koreksi', 0.002965071246416904), ('bisa', 0.002309317992529654), ('bijak', 0.00021536381902937003)], [('rakyat', 0.033409190052595156), ('hasil', -0.03079974142525578), ('sulit', -0.013736343142032368), ('perintah', -0.007154228824517821), ('ya', 0.006177222463281042), ('peras', 0.0038863566987787554), ('rezim', 0.0032238160176562285), ('pajak', 0.002976351746770497), ('bikin', 0.0028930312113641526), ('hidup', 0.0010150773236035093), ('uras', 0.0003803530092647603), ('patung', 0.0002492743677701267), ('unfaedah', 0.00024172840763434503), ('bumi', 0.00019663054927079027), ('tumbang', 0.00019145644827560176), ('sibuk', -0.00013399027893818305), ('singguh', 5.814023412062152e-05), ('dzalim', 1.9657062765857593e-05)], [('tugas', -0.0838054987175093), ('rakyat', 0.05458819964023406), ('absurd', 0.004086746576745122), ('erti', 0.0038807703714283976), ('asli', 0.003846926209097687), ('demokrasi', 0.003803806577326165), ('nih', 0.003251119513006161), ('presiden', 0.0025394567948006366)], [('kaya', 0.05398725074781353), ('ngga', -0.033544112819182674), ('bikin', 0.020166266731605063), ('yayasan', 0.017355347162319835), ('cuan', 0.015113376728972025), ('biar', -0.011683561061083184), ('orang', -0.01111040003178178), ('pajak', 0.009308762315791272), ('kalo', 0.009133612517123019), ('adsense', 0.007826111989011194), ('kanan', 0.006136307400092587), ('beneran', 0.0059348740787322106), ('bantu', 0.005346536928168862), ('mindahin', 0.005290049759386386), ('youtube', 0.0049211759991191345), ('sosmed', 0.0043228916576322236), ('kiri', 0.0035140986996867163), ('rekam', 0.0026854261641409863), ('masukin', 0.0025423482066491737), ('kena', 0.0023343574298045297), ('tangan', -0.00013573579952694817)], [('rela', -0.2868330333765501), ('bayar', -0.15597837916940946), ('urus', -0.15580252316997487), ('ga', -0.12987257720800674), ('pajak', -0.0971434461721888), ('sulit', -0.08418516026910605)], [('pajak', 0.01783654778275369), ('nelayan', 0.013560019948028655), ('bayar', -0.012177671261405938), ('cuan', 0.008097146145688164), ('monggo', 0.0064280139296311115), ('minggir', 0.005236215574489439), ('sanggup', 0.005134313007130883), ('lokal', 0.005056425773986577), ('cina', 0.004544840097906), ('kapal', 0.004521531105597987), ('balang', 0.004252894545996841), ('silah', 0.003997892402998298), ('o', 9.324294077090745e-05)], [('eh', 0.013264750075913578), ('bangsat', 0.007837171211136957), ('nggak', -0.004707708390850696), ('sih', 0.0018185997908555323)], [('kerja', -0.03215308785093474), ('rakyat', -0.017205906749589873), ('jabat', 0.016563958413851967), ('pajak', 0.015044970426192537), ('makan', 0.007747240660799039), ('utit', 0.0030842494948138343), ('ngeliatnya', 0.0029752748046644496), ('enak', 0.0027761546695014433), ('adoooh', 0.001881481521323947), ('ngenes', 0.0013699549753331832), ('keras', 0.0009558346880084723)], [('pajakperasrakyat', 0.0527252165450853), ('bayar', -0.019166540563181265), ('dondon', 0.012274815758165816), ('pajak', 0.010298659704056868), ('nambahin', 0.010005039334757763), ('alhamdulillah', 0.00967545598962367), ('pasu', 0.009521987822264197), ('nambah', 0.0076747186328372545), ('lumayan', 0.0014564656121232142)], [('rakyat', 0.020866586791317566), ('palak', 0.01301163237307344), ('jabat', 0.010699960988789402), ('pajak', -0.010345857786955454), ('hidup', 0.009292028086948576), ('gaji', 0.004165246551346552), ('mewah', 0.001642019027026915)], [('urus', -0.08219120170130322), ('ribet', 0.0328801776434135), ('banget', 0.02538594947726314), ('pajak', -0.024366784834002668)], [('harta', -0.06759717123005679), ('pajak', 0.018135898846049246), ('clear', 0.005520660730605088), ('sukses', 0.005377471682646994), ('clean', 0.005274797238243478), ('and', 0.005133563391078974), ('ya', 0.004932654241213263), ('nambah', 0.004123427434977785), ('luhut', 0.003976014698813325), ('si', 0.0037593994157749575), ('aman', -0.003249849759238163)], [('pandemi', -0.10020862142591835), ('atur', -0.03300735877177909), ('pajak', 0.004927541416607573), ('alumni', 0.004874181072843204), ('regulator', 0.004539602872713132), ('setan', 0.004483766993941225), ('pasca', 0.004397952864770506), ('ngritik', 0.00424089933209487)], [('rakyat', 0.07774183565341661), ('urus', -0.07462625513373046), ('pandemi', -0.0519284011025745), ('ga', -0.026657190323887494), ('cekik', 0.007071405943810484), ('negara', 0.005680605598667725), ('akibat', 0.005374518789389155), ('pnbp', 0.004852969837131621), ('laut', 0.004699697555218362), ('ijin', 0.004586052619768635), ('loh', 0.004564741543286174), ('gin', 0.003091861107396381)], [('rakyat', 0.029725451074934277), ('pajak', -0.016070711136087285), ('peras', 0.010347990007376254), ('opung', 0.006408240921513516), ('main', 0.005570936580426466), ('nya', 0.005084517575206083), ('sembako', 0.002756433038698043), ('presiden', 0.0018610892022340969), ('beli', 0.0017356774083745904), ('gilir', 0.00158702527918063), ('tau', 0.0009362755325992934)], [('rakyat', 0.1288250572762619), ('milik', 0.035791212174093714), ('nik', -0.02682766994503876), ('bayar', -0.022422234145039997), ('ktp', 0.008632069257094144), ('ppn', -0.00670988193616517), ('tarif', -0.004330413229996256), ('pajak', 0.0015408762672734953), ('npwp', -0.0008485514902676119)], [('rakyat', 0.13295403695297084), ('laksana', -0.056791195596277724), ('milik', 0.02747838774462166), ('nik', -0.024751745681279566), ('ugal', 0.014238817501860901), ('hutang', -0.013629918985066962), ('ppn', 0.011930205832031287), ('pajak', 0.011563550872601053), ('ktp', 0.01009250488835206), ('npwp', -0.006039692599104883), ('sandwich', -0.0033410634231260507), ('bayar', 0.002271206525067632), ('tarif', 0.0017392015538374827)], [('periode', 0.13090167456259766), ('ulang', 0.1270516627095069), ('kelompok', 0.1209187534391735), ('bebas', -0.08591886954504462), ('taxamnesty', -0.01974870484714141), ('pajak', -0.00928655822766287)], [('pajakperasrakyat', 0.08538409849460166), ('tugas', -0.06410574610960292), ('rakyat', 0.06156203351152554), ('nolak', -0.04794767953354675), ('bijak', 0.02561919307635193), ('orang', -0.019374681594175762), ('pajak', 0.00841290219695597), ('taat', -0.00625333220883235), ('pungut', 0.003596726010927203), ('labu', -0.0015250327735515908), ('jargon', -0.0012487012548792718), ('kelabu', -0.001180153195726784), ('berani', -0.001084545199895154)], [('rakyat', 0.12431483409902978), ('laksana', -0.05690332759220147), ('milik', 0.0265591514304202), ('nik', -0.02511502596125571), ('ppn', 0.016706965538138854), ('rezim', 0.016666811501109133), ('npwp', -0.010045879711279206), ('hutang', -0.009582797541609346), ('bayar', 0.008841233677109943), ('pajak', 0.008428756144622764), ('ktp', 0.00833631510095294), ('ugalan', -0.005590308703901295), ('orang', -0.005057369445109337), ('sandwich', -0.0029833813973663267), ('tarif', -0.0005552912272394011)], [('yudikatif', -0.25152016268516214), ('eksekutif', -0.2506134862598044), ('legislatif', -0.2271176635990931), ('pilih', -0.14553728712399644), ('percaya', -0.11357923073256501)], [('pajakperasrakyat', 0.016260019773755286), ('kejar', 0.010994862004641889), ('rakyat', 0.004279511361521352), ('koruptor', 0.004174483764165973), ('miskin', 0.0016913610802672117), ('pajak', -0.000633850943838873), ('agree', 0.0004092180765682004)], [('pajakperasrakyat', 0.06840007013464956), ('bayar', -0.03661347352801961), ('rakyat', 0.032823888229481485), ('pajak', -0.010411324333973761), ('rezim', 0.008412869920465074), ('ngutang', 0.005862167005755961), ('pungut', 0.004013823464874299)], [('simpan', -0.025642953834827873), ('pajakperasrakyat', 0.020351776381125633), ('aja', 0.011578963425900635), ('rakyat', 0.009545601101574815), ('pajak', 0.00536605019802063), ('kalee', 0.003038630018982121), ('porotin', 0.0022528039457546194), ('sawah', 0.002006401876913935), ('hutan', 0.0016849210832128082), ('gunung', 0.0016469027343164993), ('laut', 0.0013960813664792965), ('kaya', -0.0010751442124141374)], [('pajakperasrakyat', 0.048390589032305), ('pandemi', -0.04755256879010166), ('rakyat', 0.04333473213003445), ('pajak', -0.016294502616727307), ('sulit', -0.012304304822379179), ('ekonomi', 0.0054652644159637435), ('akal', 0.0051865301246787785)], [('pajakperasrakyat', 0.06186496380975537), ('kena', -0.013318186994829909), ('rakyat', 0.01230349707700842), ('perintah', -0.011370360849080097), ('batas', -0.0048935421911903785), ('sembako', 0.003982371420432096), ('pajak', 0.0015215240938827913)], [('bayar', -0.05549418853218739), ('kemplang', 0.03254004381195777), ('aje', 0.032339042416822464), ('enak', 0.01871112264265534), ('koruptor', 0.018242586562393116), ('tuh', 0.018051268610522316), ('suruh', 0.017189967309207275), ('pajak', 0.0030585166803495573)], [('pajakperasrakyat', 0.09456738252087181), ('pajak', -0.017219817596061334), ('kena', -0.012624625202119181), ('gak', -0.009481156077913014), ('mewah', -0.008417731414272104), ('ngemplang', 0.004910185083289426), ('sembako', 0.0035586442069376893), ('sekolah', 0.0026530084641608375), ('sehhh', 0.002303126814694071), ('dima', 0.0022779326599969107), ('afkeun', 0.0021077636310935788), ('lha', 0.001767285619762477), ('barang', -0.0012291283923562448)], [('rakyat', 0.04981660192589), ('kemplang', 0.04866403343299389), ('diskon', -0.018890009430261204), ('analis', -0.01493493967373889), ('peras', 0.013383519318228322), ('pajak', -0.007605933255501537), ('sorot', 0.006799378364205513), ('bijak', 0.003163495479463077), ('publik', 0.0023353883354307486), ('uu', -0.00044845577804878794)], [('rakyat', 0.09065072999934419), ('pandemi', -0.0382095774160717), ('bayar', -0.018163610830080956), ('hitam', 0.017597284281206476), ('jepit', 0.016508748473188357), ('kasih', 0.013494716143279246), ('pajak', -0.012532569959822765), ('amnesti', 0.012242048882266158), ('ekonomi', 0.008487168571724804), ('konglo', 0.007867614237136852)], [('pajakperasrakyat', 0.0628670248205374), ('rakyat', 0.041943496128510734), ('kaya', 0.03734804861787472), ('up', 0.016829070173909094), ('bayar', -0.016578962100478503), ('rajin', -0.015856137393841133), ('dinaikin', 0.012107770906476334), ('pajak', 0.004117982951065418), ('orang', -0.0022821883991776637)], [('pajakperasrakyat', 0.013962970294482898), ('aja', 0.005479824695242642), ('rakyat', 0.004082058730181581), ('halu', 0.0038996277752388433), ('mingkem', 0.003800273550577305), ('dinaikin', 0.002845412753513315), ('presiden', 0.0022147101884070256), ('pajak', -0.0021411147856118615)], [('rakyat', 0.035256072300000114), ('kemplang', 0.020032840575367493), ('pajak', -0.012176902282417886), ('kali', 0.0027742178602841493), ('kasih', 0.0011096723315444084), ('amnesty', -0.0005904572116787539), ('usaha', -0.00040176273538941083)], [('adil', -0.1521342294603773), ('pajakperasrakyat', 0.10457859195773513), ('kemplang', 0.05520206800886304), ('pajak', -0.017116493481764833), ('mesti', 0.009362501691685655), ('menghidari', 0.0008542951865494207)], [('pajakperasrakyat', 0.04401843792431797), ('biar', -0.036387477665210305), ('pimpin', -0.01599511827584805), ('bohongin', 0.012345059117459416), ('malaikat', 0.01209478670941999), ('siksa', 0.010954889527114202), ('bohong', 0.010366615293449823), ('pajak', -0.004503400070453501), ('dukung', 0.0013008485352174607)], [('rakyat', 0.11292963775757446), ('tutup', -0.10173244180467371), ('rugi', 0.03987408887908532), ('bijak', 0.03218296038386208), ('pajak', -0.020582485926662782), ('setor', 0.01481958976179213), ('plang', 0.012815539090758898), ('negara', 0.012442170293089146), ('beban', 0.012133863400693634), ('taxamnesty', -0.01142572623432822), ('dapat', 0.010546860091180358), ('hilang', 0.010012305735817675), ('jenius', -0.007236102047986926), ('mata', -0.004652573631706495), ('jilidii', 0.002485065543669401)], [('jokowi', 0.026935536374496395), ('cepat', 0.01885515125648185), ('ubah', -0.016407447812431187), ('jilidii', -0.016211904544942323), ('kereta', 0.014737821630640599), ('apbn', 0.013780639681537874), ('bijak', 0.007444694097602266), ('ditaxamnesty', 0.00668076627455548), ('inkonsistensi', 0.0039043180619876697), ('telan', 0.0027826512652851143)], [('pajakperasrakyat', 0.03439068731948448), ('bijak', -0.003029133072187603), ('miskin', 0.0010534129157064055), ('kaya', 0.00018941959133866453), ('kurus', -0.0001142621208859254)], [('rakyat', 0.004896611200459012), ('untung', -0.00477058949604701), ('peras', 0.003837264829585803), ('hapus', 0.003586819623042313), ('batu', 0.0029937434665576104), ('nelayan', 0.0029778206444539186), ('bikin', 0.002514833364672906), ('bara', 0.0022386014301866864), ('susah', 0.0017183133198597064), ('oligarki', 0.001495694963962592), ('ya', 0.0014830080321571117), ('royalti', 0.00125836046312773), ('pajak', 0.0009649845533525297), ('ko', 0.0007288267429184367), ('tega', 0.0005751844840841275)], [('ga', -0.038473651522555205), ('aja', 0.01790398897375771), ('rakyat', 0.016834043703039297), ('ngasih', 0.012768859659032197), ('dalih', 0.011422711313238673), ('rezim', 0.008040928593391108), ('kejar', 0.007919720796298256), ('makan', 0.007788617241320704), ('koruptor', 0.007662758026533785), ('sehat', 0.006200411480925271), ('pajak', -0.002522706079911768)], [('taxamnesty', -0.021889772016154076), ('utang', 0.01980488262293686), ('penting', 0.015671405299331562), ('plang', 0.015106259661133041), ('jelas', 0.014959769787172204), ('presiden', 0.013376653394448164), ('penuh', 0.013068126583913793), ('blbi', 0.012817698175892895), ('bijak', -0.005235699872559947), ('pajak', -0.0037573349233626352), ('usaha', -0.002726077913601319)], [('uuhpp', -0.4468889367155792), ('minimum', -0.1333364310265019), ('relevan', -0.11532650015201806), ('kena', -0.06827854399293481), ('pajak', -0.0504754695956661), ('global', -0.030905779202317164)], [('biar', -0.05503165651997565), ('dunia', -0.05150207475620131), ('cepat', 0.042569394022476635), ('bayar', -0.02077907587944874), ('nya', 0.018218382327828922), ('siksa', 0.017138654432843483), ('doa', 0.015559211960379977), ('akhirat', 0.014068224482093758), ('pajakin', 0.011248995129466373), ('pajak', -0.004886152026276204)], [('rakyat', 0.0275718508331367), ('bilang', -0.01834853874741641), ('udah', -0.006865705847328012), ('pajak', 0.004061299803259345), ('murah', 0.0034818918841151396), ('rugi', 0.0032509921330353946), ('sejahtera', 0.0029828251233760565), ('beban', -0.0022298723616731796), ('teuteup', 0.0020141347891142174), ('tetep', 0.0012809525938511025), ('jual', 0.0011877504988212946), ('bangun', -0.00022546738126757578)], [('uuhpp', -0.44056946198406893), ('minimum', -0.13326132349321057), ('relevan', -0.11605176610183712), ('kena', -0.05783761383884983), ('pajak', -0.03535200934462061)], [('pajak', -0.03151461822112428), ('sasar', 0.013204255321455083), ('diam', 0.010815707069016927), ('amnesti', 0.01050643055405493), ('rakyet', 0.008940566715656248), ('kejar', 0.008524801389040099), ('kolongmerat', 0.0070216838865197325)], [('rakyat', 0.15416451757980923), ('kaya', 0.05347785917434482), ('peras', 0.038583088046129664), ('pagi', -0.036364672990469825), ('bebas', -0.032731949299731666), ('pajak', 0.028187867918093936), ('kabar', 0.023159093269356966), ('nya', 0.022972460980476343), ('selamat', -0.02138741554748935), ('negeri', -0.016135273014691545), ('orang', 0.0037284389315149985)], [('pajakperasrakyat', 0.05462763340053053), ('rakyat', 0.038171852596413494), ('ngumpetin', -0.035873054880540024), ('menteri', -0.0229181395194489), ('pajak', -0.0107372020930042), ('asemm', -0.0022771510969413723), ('uber', -0.0015590086620693778), ('eh', 0.00018497619990956226)], [('rakyat', 0.1072854434165075), ('bayar', -0.04762411680728143), ('jabat', 0.04051970321838979), ('suaka', 0.029347547345745306), ('peras', 0.026965860726392354), ('negara', -0.025560536514601724), ('pajak', -0.01060926507388489), ('gak', -0.009915954282087932), ('sembunyi', 0.006501846427943595), ('harta', -0.002279892307033003)], [('nelayan', 0.025863304157759018), ('pimpin', -0.02490031439726061), ('nikmat', -0.01964348680416623), ('oligarki', 0.019314421415863346), ('rendah', -0.014268423930572274), ('negara', 0.013331929262032905), ('tinggal', -0.009222954264742553), ('pajak', 0.006762884869862625), ('bijak', 0.004831617754706468), ('tampil', 0.0045825842831550756), ('fasilitas', 0.0037828208227359186), ('lahir', 0.0034738930822249894), ('blbi', 0.003425191691446396), ('megawati', 0.003128823849261444), ('arah', 0.0030176757070734996), ('usung', 0.0029547184619156492), ('plang', 0.0021815593618757442), ('mau', 0.0018187905525862283), ('era', 0.0011637327888188117), ('rezimoligarki', 0.0003303247743615866)], [('nelayan', 0.02597175976813213), ('pimpin', -0.02550005381985931), ('oligarki', 0.0197962652614419), ('nikmat', -0.019484686027871505), ('rendah', -0.014500675421415653), ('negara', 0.01300702724557271), ('tinggal', -0.00950664592192766), ('pajak', 0.0066222852993356555), ('tampil', 0.004664868690071874), ('bijak', 0.0046502998395012134), ('fasilitas', 0.003992823727213112), ('lahir', 0.0037461629175589177), ('megawati', 0.0036665090198887995), ('blbi', 0.0032422268489097237), ('arah', 0.003190701050312549), ('usung', 0.0025952088511597892), ('mau', 0.002446188805455751), ('plang', 0.0020784868746570014), ('rezimoligarki', 0.001473367246805676), ('era', 0.0009918320409906746)], [('uuhpp', -0.30101613528970267), ('rp', -0.15112767991025494), ('gotong', -0.10254084288767477), ('royong', -0.09357346084337623), ('bebas', -0.0814734105306039), ('juta', -0.04804368437595058), ('efek', -0.041051431823248025), ('lengkap', -0.015108021895125884), ('gaji', -0.008724918638633557), ('pajak', -0.007773173145915161)], [('ppn', -0.03904744163652614), ('rakyat', 0.03075339624123586), ('kejar', 0.017343760372889115), ('dll', 0.013178706631821482), ('cukai', 0.00615789601667739), ('genjot', 0.004190843129875934), ('pajak', -0.0002152432848917136)], [('umkm', -0.4570266050422911), ('bebas', -0.23428307333633805), ('pajak', -0.09321482675933551)], [('rakyat', 0.01573109889938132), ('segi', 0.01056500049649352), ('negara', -0.010416579672018922), ('utang', 0.009961835608836726), ('gunung', 0.008548593491989467), ('aneka', 0.00811925013930595), ('palak', 0.007731144317616293), ('hancur', 0.006438298132309479), ('tanda', 0.0063184906313506), ('pajak', -0.004190690648806079), ('ekonomi', 0.000787851239397625)], [('pajakperasrakyat', 0.08520271289256788), ('gak', -0.015864363708238658), ('kena', -0.013804341689610256), ('pajak', -0.01146011223563893), ('ya', 0.006745953415830048), ('dipajakin', 0.0039441235972954135)], [('pajakperasrakyat', 0.026620458318724362), ('perintah', -0.012335524215992502), ('fak', 0.004176473834025207), ('ya', 0.0037082068992574857), ('islami', 0.0028332764659367844), ('inget', 0.0027678165858983817), ('kapitalisme', 0.002690682741195252), ('tegak', 0.002547423412912011), ('pajak', 0.002058866528009309)], [('karbon', -0.19295652132617894), ('adil', -0.1562582880230442), ('masyarakat', -0.07814253839579184), ('klaim', -0.05093189151360233), ('terap', -0.03185495024861016), ('pajak', 0.010082698199042088), ('perhati', 0.009918394839565133), ('asas', 0.009641560101464042), ('kemenkeu', -0.004414941922154248)], [('bayar', -0.09289108421532065), ('heran', 0.044134569781261374), ('buzzer', 0.038560268868114866), ('pake', 0.034239777401183974), ('pajak', -0.009250606379381755), ('kerja', 0.007123956834544856)], [('jelata', -0.027259341635677635), ('pajak', -0.020344849395170634), ('rakyat', 0.013998314451701148), ('nya', 0.012701996477384542), ('objek', 0.007573595762921192)], [('rakyat', 0.09819335473512522), ('bebas', -0.04388935978683956), ('kaya', 0.02792030214215428), ('jabat', 0.02303683257960536), ('pajak', 0.017636612516179262), ('drastis', 0.01573903512699978), ('dinaikin', 0.013636165681722347), ('gede', 0.01113520647812177), ('harta', -0.009429794958878997), ('usaha', 0.006811746437287224)], [('pajakperasrakyat', 0.03456356149934003), ('rakyat', 0.02045118869464612), ('hasil', -0.017979800021272616), ('pora', 0.006756866036072459), ('pesta', 0.005534361274136621), ('utang', 0.005009445427851255), ('elit', 0.004483536796830163), ('tanggung', 0.003743884360043738), ('korupsi', 0.00346824946984734), ('negara', -0.0033817005490709338), ('pajak', -0.0017686526845415384)], [('reformasi', -0.27795428288752194), ('paja', -0.15057213820100784), ('sistem', -0.09539160022278731), ('kumpul', -0.06935720817011291), ('salah', -0.06261544216351676), ('taxamnesty', -0.044599667541137214), ('perintah', -0.04408474977410755), ('program', -0.034106038276257425), ('pajak', -0.028626222741445374), ('target', -0.016480525603378048)], [('nelayan', 0.030062876342669586), ('ppn', -0.020215096708306797), ('kondisi', 0.013890585237425195), ('tani', 0.012133166924599455), ('sembako', 0.010453945551754591), ('nilai', 0.008254956457129472), ('rencana', 0.004028816667670683), ('sulit', -0.0032580936589719687), ('beban', -0.0001370459458017819), ('sengsara', 7.857973506828252e-05)], [('ruuhpp', -0.0916581867530452), ('bebas', -0.05669195241060033), ('pokok', -0.025056734432122377), ('kena', -0.018077374766362925), ('sembako', 0.016623736691778756), ('sekolah', 0.015577416981269668), ('tenang', 0.01418009089728768), ('pajak', -0.012043284498519534), ('butuh', 0.006218381766719882), ('jasa', -0.005065226568203235), ('barang', 0.0035114467411871183), ('didik', 0.002919052173975209)], [('gelar', -0.02073324395801139), ('negara', 0.016452653882049653), ('nelayan', 0.013799521037791576), ('biaya', 0.008783926133970749), ('jawa', 0.00762295675118734), ('terima', -0.007568537049715949), ('ratus', 0.007370373168725964), ('kibar', 0.006371280226141508), ('putih', 0.006349924546225096), ('bendera', 0.006090830617626573), ('aksi', 0.00550674152710744), ('pati', 0.005502894428480218), ('pajak', -0.0018719460139050533)], [('pajakperasrakyat', 0.030792762719433688), ('nelayan', 0.008475740614490778), ('juwana', 0.0058438476027420725), ('ngeluh', 0.004770915358774466), ('pati', 0.004189229903688403), ('pajak', -0.0010575544650804683)], [('rakyat', 0.06666667883856571), ('harap', -0.04308248230026653), ('jenius', -0.02177373169396808), ('pajak', -0.01345864220809978), ('kentut', 0.01177510291070036), ('pajakmemerasrakya', 0.011414222688028485), ('bijak', 0.011027217686156936), ('hidup', 0.010520345012319039), ('super', 0.010179185838104215), ('kalo', 0.009585717053722546), ('hancur', 0.009507424258415886), ('sejahtera', 0.009465217950065394), ('presiden', 0.008061920452400765), ('tanggung', 0.0070673746926878086), ('kena', -0.0021669225893260766)], [('pajakperasrakyat', 0.07536361830438029), ('yaa', -0.07080764108898377), ('pajak', -0.00920007955946522), ('banget', 0.0008325573009733391), ('prie', 0.0002037310364713182), ('rumit', 0.0001612195011503863)], [('rakyat', 0.025547121063335055), ('perintah', -0.017410898337010785), ('dr', 0.011858243865639929), ('pajak', -0.005612298958805871), ('dizolimi', 0.00526786635845946), ('dapet', 0.004522879974215459), ('duit', 0.004217142288133969), ('nagih', 0.004213805454649029), ('ngotot', 0.0014322973337086442)], [('adil', -0.1475980342720422), ('rakyat', 0.04166080154014678), ('pajakmemerasrakyat', 0.022211626039286534), ('kaya', 0.021787888581104732), ('ampun', 0.015247832061616708), ('rakyet', 0.014587073133463578), ('sasar', 0.010937349878709292), ('kejar', 0.009735563473135766), ('zhalim', 0.008921504377071515), ('situ', 0.008863019298668536), ('amnesti', 0.00760267259125282), ('kolongmerat', 0.007132704207301779), ('pajak', -0.005268345942098336), ('si', 0.004830003072850249), ('jelata', -0.004205903544451036)], [('rakyat', 0.015436421186765685), ('bayar', -0.013992959927338268), ('olok', 0.012723826924887577), ('anak', 0.012444337848263725), ('buzzerp', 0.011718868365644447), ('kadrunbusuk', 0.011139620226315528), ('tagar', 0.008893649327734754), ('bangsa', 0.008868696728326287), ('habis', 0.008218468479836188), ('uang', 0.007279119079172199), ('pajak', 0.006749675233272347), ('negara', -0.0001677023853833726)], [('pajakperasrakyat', 0.02469208423509613), ('simpan', -0.022261869252304573), ('aja', 0.013962322608776626), ('rakyat', 0.00766965846782211), ('kalee', 0.0031888896884641423), ('pajak', 0.0029487742098713117), ('sawah', 0.002879353767126614), ('porotin', 0.002833152903257251), ('hutan', 0.0027010602493476206), ('gunung', 0.0021243967503664645), ('laut', 0.0018125078556992876), ('kaya', -0.0009649541267637079)], [('maju', -0.21769483960561692), ('adil', -0.1904920823912247), ('indonesia', -0.1317232631093281), ('perata', -0.06575158277358042), ('wujud', -0.0525018391449326), ('mutlak', -0.04121997425936831), ('negeri', -0.04109210596654307), ('bangun', -0.025730677837874187), ('jokowi', 0.02315706178926533), ('pajak', -0.01838030479941186), ('ekonomi', -0.015286555065482568)], [('uuhpp', -0.3180011150462708), ('karbon', -0.238596535780356), ('implementasi', -0.15445097585564468), ('pajakberkeadilan', -0.0759446937898109), ('kawanpajak', -0.03800418455593821), ('pajak', -0.03706517222620446)], [('dorong', -0.14799617123008865), ('cegah', -0.12782390323555556), ('investasi', -0.09164677777863797), ('indonesia', -0.07726319897907485), ('tingkat', -0.05458508713506645), ('peran', -0.05457499110875265), ('jokowimembangunnegeri', -0.044250783278220636), ('inflasi', -0.037183511765267976), ('ukm', -0.031993605295262226), ('indonesiamaju', -0.029568838906720143), ('kembang', -0.02731514439020362), ('negeri', -0.022682979595944005), ('bangun', -0.020396525609763884), ('pajak', -0.019919748346581532), ('nasional', -0.011638940726941613)], [('rakyat', 0.04503131427597052), ('ga', -0.024513235107817354), ('bebal', 0.01815695995331469), ('kritisi', 0.017906560128707536), ('tekateki', 0.01790573550686442), ('nanggung', 0.015171920961773418), ('hutang', -0.014053918274994467), ('penuh', 0.013439893612241568), ('kalo', 0.009382877082613547), ('beban', 0.004484494496309132), ('perintah', -0.004016803367163033), ('pajak', -0.0016459523641103877)], [('ugalan', -0.014326527553643059), ('rakyat', 0.013938056119950226), ('aja', 0.007429215929377111), ('kaya', 0.006275809726269418), ('pajak', 0.004758773365112722), ('jabat', 0.004157747824131779), ('genggam', 0.003914181669704475), ('amanah', 0.003697112542815874), ('orang', -0.0027451547320371944), ('kuasa', -0.0027405434778053897), ('beban', -0.0026511192501275233), ('jiga', 0.00165916922395205), ('tanggung', 0.00157541088044491), ('cela', 0.0014139288674238602), ('anggap', 0.001210371460066708), ('kalo', 0.0011156221903927462), ('gak', 0.00010242890254836588)], [('rakyat', 0.0392492592915209), ('pajak', -0.0371076563589226), ('ga', -0.017845243654574), ('untung', -0.01118355961514173), ('ampun', 0.007188975134647733), ('cukong', 0.006117773025035756), ('konglomerat', 0.005042510530618562), ('sembako', 0.0029631088183849005), ('gede', 0.0026208159966849584), ('iyah', 0.0015430838064509817), ('akal', 0.0015101710483145145), ('bayat', 0.0014538270337545883), ('fasilitas', 0.001181637219817738), ('objek', 0.0010396469259264904), ('neng', 0.0010220518203604306), ('aseng', 0.0007882046277620837), ('repot', 0.0007000560498126176), ('pailit', 0.000679368102035267), ('mutih', 0.0003430619922683803)], [('pajakperasrakyat', 0.10650067270663781), ('hubung', -0.03542445261058149), ('hutang', -0.021288360476539668), ('pajak', -0.009022167274326304), ('tumpuk', 0.0040232826766494634)], [('bayar', -0.024528629865568165), ('rakyat', 0.021982855602543216), ('enggan', 0.019409153245434057), ('aja', 0.01428219092626567), ('skandal', 0.013250088322953749), ('jabat', 0.011695861883084038), ('pandora', 0.01113674717505829), ('suruh', 0.010821146216907421), ('papers', 0.0103379442042149), ('pajak', 0.0016277606467130013)], [('rakyat', 0.05573955350099611), ('jabat', 0.03878899458745393), ('nikmat', -0.030525046673578336), ('bayar', -0.019230308134073684), ('pangeran', 0.016747600858530966), ('bak', 0.016740810599612587), ('budak', 0.013593441120956049), ('pajak', -0.0016061675855612984)], [('pajakperasrakyat', 0.027432116948261573), ('negara', -0.011339618885080461), ('rakyat', 0.010959817774332688), ('ragam', 0.008530150977521576), ('pajak', -0.007882841670348695), ('aneka', 0.006612811748843171), ('hancur', 0.006422548670258411), ('pungut', 0.005772771021947257), ('tanda', 0.005681927400221927)], [('pajakperasrakyat', 0.03493096626306765), ('rakyat', 0.028786349308913144), ('giat', -0.020296271166351892), ('jilid', -0.019547635678533987), ('aja', 0.010583634316960324), ('pajak', 0.010528153561029213), ('ppkm', 0.003740591039501339), ('digedein', 0.003649663832275981), ('majakin', 0.003243605567429846), ('lha', 0.0029000486812250322), ('batas', -0.002195190731554242)], [('buzzerrp', 0.016066190441027264), ('gak', -0.012757017254507563), ('pajak', -0.011345635762027595), ('uang', 0.010894682586022145), ('sih', 0.010698769543408204), ('pakai', 0.009691763745666663), ('biaya', 0.009157485586771312)], [('yudikatif', -0.2517618807779438), ('eksekutif', -0.250448685721774), ('legislatif', -0.22651995933951205), ('pilih', -0.1449741634831473), ('percaya', -0.11292037524607847)], [('uuhpp', -0.30413838622930234), ('rp', -0.14550788948716195), ('gotong', -0.10008312003081501), ('royong', -0.09042387008439465), ('bebas', -0.08828535911791091), ('juta', -0.040544026626405306), ('efek', -0.03799053504849932), ('gaji', -0.008362562026481092), ('pajak', -0.005446741724502471)], [('pajakperasrakyat', 0.06970063972074207), ('kena', -0.024101656293723146), ('timbang', 0.0032957593881977453), ('bang', 0.002897470566562437), ('khusus', 0.0022981584140705116), ('pajak', -0.0013326348838889196)], [('pajakperasrakyat', 0.1055893000001356), ('sederhana', -0.028591096788014123), ('pajak', 0.0072677617540601), ('perintah', -0.005394079529051144)], [('mudah', -0.18837767586776755), ('nik', -0.1798917508065829), ('tugas', -0.14233743062961907), ('npwp', -0.1418996196646652), ('kerja', -0.052816324505858325), ('pajak', -0.05222232701459342)], [('hasil', -0.04105225548213444), ('biadab', 0.022968624495777135), ('pajak', -0.022721125810643443), ('warga', 0.022328754925798423), ('taat', -0.021679731183147265), ('hoeeek', 0.020775988623306558), ('adab', 0.020056196318421808), ('istilah', 0.018984168814414597), ('bullshit', 0.01895619538056391), ('negara', 0.010019859286899397), ('b', 0.0006325066704122111)], [('rakyat', 0.02697900508343607), ('taat', -0.01606451778960719), ('bijak', 0.011972470078897405), ('pajak', 0.01181775018331433), ('orang', -0.010803781210445245), ('asiiik', 0.0076672019741333775), ('senyum', 0.007384127590302299), ('kudu', 0.006444221482561704), ('rezim', 0.0063450193797993685), ('halal', 0.006049045849893875), ('korupt', 0.005878507679714821), ('koruptor', 0.00550666658359865), ('palak', 0.005068016606280022), ('duit', -0.00106755349598264)], [('rp', -0.09684495724797185), ('negara', 0.025349181359510434), ('nelayan', 0.02310502886052803), ('rugi', 0.022780022664679527), ('bebas', -0.022550345231026763), ('bara', 0.017541649181094777), ('hapus', 0.017081053864377593), ('ya', 0.016042740003842006), ('triliun', -0.015592037980344215), ('batu', 0.014651925920480592), ('gede', 0.012299470096900557), ('holiday', 0.01211487213735522), ('banget', 0.012053110099296636), ('tega', 0.010998790538152236), ('royalti', 0.010911714680323505), ('tax', 0.00846920427165029), ('eh', 0.008016197939300378), ('rr', 0.007389584420594601), ('pajak', -0.004508892153960834), ('persen', -0.001587625649641771)], [('integrasi', -0.16163717271253458), ('data', -0.10814836590009698), ('duduk', -0.09122563575719937), ('implementasi', -0.08845170711381614), ('nik', -0.08067461380531131), ('wajib', -0.07344596231132287), ('paja', -0.06620179625991265), ('npwp', -0.04748471578539919), ('atur', -0.041828180796374057)], [('tuju', -0.12230645604894608), ('pajakperasrakyat', 0.09425743319799976), ('koruptor', 0.02037715604510234), ('mati', 0.009516897475367447), ('pajak', -0.006092377697899222), ('hukum', 0.0056940011026915946)], [('rakyat', 0.01890723755457998), ('yaaa', 0.013547468534646914), ('bayar', -0.010306788619773855), ('kereta', 0.00866889408628064), ('cepat', 0.008564588215668446), ('pajak', 0.006383220873527775), ('sing', 0.005308689191778908), ('ibukota', 0.0049626563194919366), ('pindah', 0.004683873434375383), ('penting', 0.004356663844436852), ('bangun', -0.0011036923468602283)], [('maju', -0.23964431014712737), ('laksana', -0.11014579333892466), ('hindar', -0.08617449345208396), ('tugas', -0.07802531574073368), ('peran', -0.07456537129678956), ('pajak', -0.05276837500140604), ('suap', -0.04987317165154084), ('andal', -0.033802976370736065), ('awas', -0.03352657880328103), ('dapat', -0.024226646065757813), ('negara', -0.018334074158551764), ('korupsi', -0.006356851980462936)], [('ruuhpp', -0.10225155021096717), ('pajakperasrakyat', 0.058157492023231436), ('kemplang', 0.042823215712399845), ('thread', 0.020070027170200434), ('ironi', 0.017748749536957886), ('merah', 0.014746484102483434), ('karpet', 0.014701149939745492), ('pajak', 0.009058729454629637), ('a', 0.00020213163226489862)], [('bayar', -0.23340686616995143), ('biar', -0.1640600050484936), ('hutang', -0.15846886464466406), ('pajak', -0.13602130989346156), ('negara', -0.129219051920632), ('gak', -0.07365193776814126)], [('pajak', -0.02696133236191796)], [('pajakperasrakyat', 0.032857159376631855), ('rakyat', 0.01364639163235943), ('fakta', -0.010529470483161973), ('pajak', -0.009543068034377532), ('utang', 0.004729922928320012), ('bangun', -0.0014864216665382032), ('rezim', 0.0013487859276624806), ('sejahtera', 0.0011868345995395921), ('belenggu', 0.0010010204353182082), ('jamin', 0.0005943214825629009), ('kelabu', 0.0003843247750907705)], [('doang', -0.0659567939625572), ('pajakperasrakyat', 0.0465224672684492), ('ga', -0.022076977039745222), ('aja', 0.017362717364856457), ('kirain', 0.011422122246451477), ('belit', 0.011033062039312307), ('simpel', 0.011025103543644204), ('ya', 0.00958344320200941), ('gitu', 0.0072187026884905844), ('pajak', -0.00301643982540552)], [('rakyat', 0.026910975396249016), ('pajak', 0.015879313563041696), ('bayar', -0.012063420955650508), ('seru', 0.010468827563892724), ('lu', 0.007695370743458102), ('lbp', 0.00743338649380199), ('sial', 0.007428596136898642), ('airlangga', 0.006716690379661021), ('muncul', 0.005590130651418435), ('mati', 0.0049333375405326025), ('tangkap', -0.004430897324963698)], [('rakyat', 0.02724849753631434), ('pajak', 0.01769642066783625), ('ngumpet', 0.012697809616467427), ('bayar', -0.01063086105291352), ('seru', 0.007700234214166582), ('luhut', 0.006288086246596946), ('situ', 0.0052845172807846836), ('tangkap', -0.004976746729822782), ('airlangga', 0.004860037541205614), ('muncul', 0.004217043252300712), ('mati', 0.003934711179612726)], [('pajakperasrakyat', 0.062066580027291485), ('haram', 0.0322354247007229), ('harta', -0.0297406738035695), ('pajak', -0.011500176117526706), ('riba', 0.009949519685164136), ('sekuler', 0.009105869125702947), ('negara', -0.0012412064182678974)], [('pajakperasrakyat', 0.020636156830933505), ('pajak', -0.013315069749975364), ('ga', -0.010618381339828448), ('rakyat', 0.008302517424059791), ('aja', 0.005795122631093123), ('peras', 0.003619867693880687), ('tau', 0.003087194864355207), ('nya', 0.0019188980104999096), ('ya', 0.0017596001569597929), ('sembako', 0.0015150511271016034), ('opung', 0.001481431610084903), ('beli', 0.0013958305473533335), ('main', 0.0011829305871941822), ('president', 0.0009596506510699875), ('pura', 0.0007485182391867676), ('gilir', 0.0006926301076221144), ('takut', 0.0005469522332808933), ('diem', -0.0003040167391468115)], [('pajakperasrakyat', 0.012325180508305366)], [('pajakperasrakyat', 0.035408060265567656), ('kuasa', -0.00901182860380883), ('daging', 0.006659932305601326), ('darah', 0.00618305766223332), ('ngutang', 0.005764811931465742), ('pajak', -0.002402799568155598)], [('pajakperasrakyat', 0.041289306826406136), ('susu', 0.011813233566872644), ('tuh', 0.006781075716077006), ('peras', 0.006068965262726003), ('pajak', -0.003578611062451053)], [('taxamnesty', -0.012710006914383294), ('pajakperasrakyat', 0.011186783602181553), ('rakyat', 0.006344578926231611), ('situ', 0.005772203551691626), ('horang', 0.005637408257197921), ('raya', 0.005199083628731487), ('bong', 0.005082728717620834), ('oligarki', 0.004999874806851751), ('dinaikin', 0.0046737658120718775), ('waras', 0.004248170576939188), ('kek', 0.0042313622930451504), ('nelayan', 0.004071800826763157), ('kaya', 0.0033728656753653163), ('pajak', 0.0001444677038608299)], [('tentu', -0.18171386253093477), ('pajakperasrakyat', 0.17197194723797146), ('wajib', -0.02868795855032035), ('bayar', -0.02755391786489588), ('orang', -0.01112233460726986), ('kelas', 0.00793639602336971), ('pajak', 0.0038189118665209703)], [('adil', -0.2241226538151947), ('masyarakat', -0.17408837107105274), ('konsumsi', -0.1299532748806447), ('bebas', -0.1271382891141843), ('hasil', -0.08787036739491857), ('pajak', -0.029387872526591464)], [('pajakperasrakyat', 0.02506061510211639), ('beban', -0.020972089261234655), ('pajak', -0.019295594745905476), ('takut', 0.013953193418320449), ('alas', -0.01017050770823131), ('uang', -0.007160394857134157), ('hapus', 0.0055907645692922425), ('oligarki', 0.0028213038341787055), ('pergi', 0.0026438013933111045), ('batubara', 0.0023702892934816)], [('integrasi', -0.1958764011344033), ('data', -0.12882234181377725), ('nik', -0.11702659070460207), ('duduk', -0.1090357471879576), ('administrasi', -0.10603403088161212), ('npwp', -0.0918446744212063), ('sistem', -0.0799891025901291), ('pajak', -0.024761101658098406)], [('reformasi', -0.40928831405965316), ('indonesia', -0.15011263766405308), ('pajak', -0.09208520270736265), ('adilsehat', -0.07466891676609272), ('akuntabel', -0.0702600221074104), ('efektif', -0.04805485967745577)], [('reformasi', -0.35462692277024965), ('maju', -0.2198495336590978), ('indonesia', -0.149728918778454), ('cita', -0.11754207469011668), ('pajak', -0.03666203515202757)], [('umkm', -0.22697614277370617), ('reformasi', -0.19384242013702052), ('ruuhpp', -0.11418805802548335), ('masyarakat', -0.10617513073769257), ('langkah', -0.026231652859748975), ('cek', -0.02552128562930352), ('fakta', -0.024463826293378052), ('pajak', -0.003143047774222217)], [('reformasi', -0.3962491060660426), ('rantai', -0.10346803157762675), ('pandemi', -0.09514821167256808), ('jalan', -0.07467513337521627), ('mata', -0.06222326355552094), ('pajak', -0.060113999710887114), ('pisah', -0.057529255832632305)], [('reformasi', -0.30212925568793625), ('paja', -0.11858362157093721), ('ruuhpp', -0.07143897349998249), ('sah', 0.022045145675742582), ('atur', -0.016873895111045048), ('ruu', 0.01654043262265848), ('komponen', 0.015720314473385493), ('hpp', -0.012137556569793505), ('harmonisasi', -0.005308469473763465)], [('salah', -0.1032543749556103), ('duit', -0.011411096239010262), ('bayar', -0.011193542091816703), ('pajak', 0.0068267581543900195), ('gitu', 0.005385963854468812), ('kocak', 0.0036199688810475265), ('milih', 0.003611644732906166), ('sayur', 0.0034781389651442375), ('ngegaji', 0.0034558434468454384), ('ketupat', 0.0033287004377011344), ('milu', 0.0031012098820552107), ('smh', 0.003052894364527241), ('tau', 0.002932627298380762), ('dah', 0.0019314265909944068)], [('kepala', 0.025259300251107387), ('sakit', 0.017430358932584204), ('perkara', 0.017092171881306642), ('pajak', 0.0018822460766688303)], [('reformasi', -0.2291864040935803), ('dorong', -0.10410630644889614), ('indonesia', -0.09778400921757907), ('adil', -0.09287710647079657), ('iklim', -0.04921841312574373), ('akuntabel', -0.0321959447665092), ('pajak', -0.026113550482875513), ('efektif', -0.01686146274718591), ('ekonomi', 0.013051641553681879), ('sehat', -0.007682029234331834)], [('pajakperasrakyat', 0.154563922127813), ('wajib', -0.1341987019110676), ('pajak', -0.015501046640190943)], [('bayar', -0.04823605816305336), ('gua', 0.030556780736548227), ('buzzer', 0.030373563692095977), ('mahal', 0.02161622143850081), ('sedih', 0.021385437398758353), ('makan', 0.020282792111526553), ('uang', 0.01394701396920536), ('pajak', -0.00011321997154797427)], [('dunia', -0.11182881181948034), ('level', -0.1047282152374329), ('skenario', -0.09291639758283605), ('gerak', -0.08367043915545798), ('reputasi', -0.08350322649835885), ('laju', -0.07861612060916826), ('perekonimian', -0.07837957834141972), ('strategi', -0.06964214403420804), ('giat', -0.0621370518688038), ('nggak', -0.01758764521853297), ('negara', 0.007693370042371231), ('l', 0.0015771635980520843)], [('dorong', -0.22135962477660245), ('properti', -0.11121678637597657), ('sektor', -0.10871564886081941), ('indonesia', -0.10643307797463301), ('berarap', -0.09822747835595466), ('rei', -0.0978767813938646), ('tumbuh', -0.05918461802924208), ('taxamnesty', -0.04674581465542916), ('pajak', -0.03410548972174774), ('ampun', 0.029245900601287688), ('bijak', -0.010268534834140024)], [('aja', 0.025439979109584527), ('ga', -0.015533122131429237), ('pajak', 0.014700490788206908), ('menteri', -0.01122033296548611), ('kyknya', 0.010162877255210704), ('tetangga', 0.008303103803144675), ('jago', 0.007904340629994785), ('kalo', 0.006880076033169312), ('repot', 0.006746772445402679), ('naikin', 0.006615863091797526), ('bu', 0.006190237551007315), ('bantu', 0.0061743895352920665), ('ngutang', 0.005370517144027625), ('kerja', -0.004143170169713206)], [('pajakperasrakyat', 0.09559190176837529), ('horang', 0.05994981409078799), ('jenius', -0.054063773181505105), ('libur', 0.02086761733961229), ('kayah', 0.01977743855413691), ('otak', 0.019068537755602535), ('miskin', 0.015974801901988002), ('pajak', 0.0010023165009166023)], [('kemplang', 0.05703423632336508), ('indonesia', -0.053570781506492346), ('rakyat', 0.0414719601707442), ('pajak', -0.014206400683365782), ('enak', 0.013149592638073608), ('peras', 0.008975306990895023), ('sanksi', 0.007816924499765787), ('ringan', 0.0075729097163826226), ('pidana', 0.00644699724703013), ('habis', 0.0033168825842421726)], [('pajakperasrakyat', 0.01658481273018558), ('pajak', 0.004024521148667053), ('nya', 0.003828702222577005), ('upeti', 0.0015926982738332684), ('nama', 0.0015204996789689859), ('jaman', 0.0014520285511909932), ('ah', 0.001450023191058281), ('raja', 0.0014363280446050492), ('udah', -0.0006763053983771509), ('pungut', 0.0006255252353396558)], [('pajakperasrakyat', 0.042214042171724815), ('tarif', -0.029363883826058222), ('kaya', 0.024356192618622447), ('rakyat', 0.021564564675552712), ('pajak', 0.01566240637183435), ('kontradiktif', 0.013003050326259471), ('insentif', 0.010451927137151441), ('tari', 0.007521815378842472), ('kena', -0.007511333079510968), ('orang', -0.0074153262900653015)], [('pajakperasrakyat', 0.03767555190290657), ('rakyat', 0.01117748766251343), ('rebah', 0.006350015656654148), ('pajak', -0.0039206630576794485)], [('rakyat', 0.04756965079912539), ('negeri', -0.03669216719482868), ('pajak', -0.015476443936483434), ('cekik', 0.009316976684638002), ('hancur', 0.008549989565516143), ('beban', 0.0068882090575117265), ('tanda', 0.00666814972928115)], [('pajakperasrakyat', 0.06905121110569135), ('pajak', -0.03130156158022739), ('oh', -0.009823893914274576)], [('umkm', -0.28224575800417584), ('uuhpp', -0.2328084782022271), ('laku', -0.059433245932949665), ('wujud', -0.05831603847092993), ('tuang', -0.05669791476258205), ('mikro', -0.03198008996241206), ('usaha', -0.008033628261675715), ('perintah', -0.007008244915432385)], [('pajakperasrakyat', 0.06557895148067534), ('bayar', -0.027911496567735665), ('alhamdulillah', 0.015326436630716894), ('nambahin', 0.015249869716520861), ('nambah', 0.010229117446646925), ('pasu', 0.01009553355138197), ('pajak', 0.0052660457048249905), ('lumayan', 0.0025537157667172975)], [('umkm', -0.1892920999288195), ('uuhpp', -0.1595224493564236), ('rp', -0.06216009598284387), ('bruto', -0.04561234071574042), ('laku', -0.04524725577036233), ('wujud', -0.043869352237515015), ('hasil', -0.03743613683135158), ('pajakberkeadilan', -0.03601985384197798), ('tuang', -0.028099157550776153), ('juta', -0.020939644328845326), ('mikro', -0.0196080922363843), ('pihak', -0.01849036118171112), ('usaha', -0.013178682134823188), ('pajak', -0.009858013923893806), ('perintah', 0.0022876080357695006), ('kena', -0.00178793916229116)], [('doang', -0.04386423095679424), ('kemplang', 0.02685337959349415), ('pajakperasrakyat', 0.0247446358364312), ('ampun', 0.009866320549604455), ('jokowi', 0.00882749818252949), ('pajak', -0.003774820509105591), ('mazzz', 0.002063822731417938), ('eksekusi', 0.0018725036121138515), ('omong', 0.0007983542282282275)], [('pajakperasrakyat', 0.08696829688058173), ('sri', -0.037400995503116985), ('ppn', -0.02366283732179021), ('mbak', 0.011719979800836649), ('tanggal', 0.01105134438473113), ('pajak', 0.009421850794374891), ('april', 0.00838512860335694)], [('rakyat', 0.049190426846837665), ('harta', -0.019269167062924888), ('makan', 0.00937805425708406), ('berkah', 0.008812785739592459), ('allah', 0.007574428531091651), ('minum', 0.007505294900711276), ('pajak', -0.006853468845500892), ('syariat', 0.00652760884804007), ('izin', 0.005565127609056855), ('pake', 0.004055077598620468), ('negara', 0.003885010699503362)], [('pajakperasrakyat', 0.09770830012972942), ('layak', -0.03072037486007679), ('pajak', -0.01923601517894613), ('hapus', 0.010480505918153825), ('negara', -0.008886737250385742), ('islam', 0.006820879938819934), ('haram', 0.006120661586960347)], [('rakyat', 0.08730278759319396), ('negeri', -0.0335679608282047), ('harta', -0.024233066879291575), ('makan', 0.010551496035368384), ('kontrak', 0.004856421244544235), ('hina', 0.0037582814771852254), ('hakekatnya', 0.0032310094837799948), ('pajak', -0.0002774425983257201)], [('rakyat', 0.06250461824616266), ('harta', -0.024864498615364824), ('jelata', -0.02438130487063381), ('peras', 0.01330978124074977), ('sejati', 0.008942322123496467), ('budak', 0.008106998103661878), ('pajak', -0.0054626041951446516), ('bahasa', 0.00433491529101745), ('kedzoliman', 0.0036216234758643796)], [('orang', -0.039720947044376856), ('dinaikin', 0.027333381048377365), ('rakyat', 0.026963070675451246), ('sedih', 0.023104782222355076), ('miskin', 0.01554205906709941), ('pajak', 0.007001375815289767)], [('sektor', -0.07948957688251516), ('utang', 0.041689563719041055), ('plus', 0.029231699779992897), ('basmi', 0.021325225149417905), ('pajak', -0.00882407988642033), ('kerja', -0.005199869451866947), ('tingkat', 0.00027393933470012645)], [('pajakperasrakyat', 0.02188632811334065), ('rencana', -0.006272828353158226), ('sukamta', 0.0056297502018038325), ('daya', 0.005174346117911655), ('pks', 0.005002861076846817), ('sembako', 0.004571195506298236), ('pajak', -0.002974613962678414), ('rakyat', 0.0017194840946179262)], [('uuhpp', -0.2224032831717297), ('kemplang', 0.14551481750070358), ('ruuhpp', -0.0421719432102278), ('jalan', -0.03262651996784708), ('law', 0.02082346749874918), ('mulus', 0.019753777553954126), ('kelas', 0.018860053195274815), ('kakap', 0.018543866968656373), ('omnibus', 0.015883749838491733), ('ruu', 0.013962309221728348), ('cipta', -0.010591250348518813), ('pajak', 0.008699934842637988), ('kerja', 0.007249195484943649), ('laku', 0.00023546025454718365)], [('pajakperasrakyat', 0.04062462066068315), ('rakyat', 0.006622973141219861), ('pajak', -0.006444764516311389), ('korban', 0.0054627749491067525), ('iya', 0.004198689508848249), ('target', 0.0014011014346366152)], [('rakyat', 0.02556337150935452), ('pajakperasrakyat', 0.018499373100202148), ('menteri', -0.012780813958038333), ('lini', 0.009524033791339993), ('modal', 0.007678933685394151), ('hutang', -0.006228092939174311), ('hidup', 0.005714377629229847), ('rengkuh', 0.005330281045150544), ('hebat', 0.004888268393252624), ('sejahtera', 0.004645772648194278), ('bangun', 0.004173970136160461), ('pajak', -0.003850979787337268), ('negara', -0.002226692433131698), ('uang', 0.002194530364622264)], [('rakyat', 0.08211832057920014), ('bayar', -0.058262198929545206), ('ya', 0.01919071737702735), ('pajak', -0.019074148926620803), ('usaha', 0.014157200772734036), ('produk', 0.004531816142493532)], [('pajakperasrakyat', 0.02500828259281405), ('kaya', 0.010881843821425943), ('rakyat', 0.007188898789141736), ('pajak', -0.0034165518073670216), ('sda', 0.003398453861075422), ('sejahtera', 0.0025487984040292603)], [('pajakperasrakyat', 0.03752241412145295), ('tarif', -0.03208604240113753), ('kaya', 0.021070175658769036), ('rakyat', 0.018998124966392728), ('pajak', 0.01747143905115534), ('kontradiktif', 0.00795120115741643), ('orang', -0.007508715337465215), ('kena', -0.006914040331212363), ('insentif', 0.006607949809737171)], [('tarif', -0.03828982125900865), ('risiko', 0.032086923690258354), ('pajak', 0.020645000915026884), ('terhada', 0.012235675892949854), ('ekonom', 0.012125011871305531), ('persen', 0.009255787364358909), ('ppn', -0.004219802819203971)], [('ri', -0.14506200574622238), ('kalang', 0.05048887886379794), ('januari', 0.04691992765570135), ('orang', -0.015423269687999548), ('beban', 0.01456738589174404), ('pajak', 0.0013433492832931068)], [('pajakperasrakyat', 0.05509075824924478), ('sisi', 0.011386966580310435), ('pajak', -0.011289717676843837), ('ya', 0.009660420803984568)], [('wajib', -0.19964099647345393), ('negara', 0.006598335032237182), ('nya', 0.004931175563397468), ('gak', 0.004626994143001346), ('bayar', 0.0042994059030390126), ('institusi', 0.003781581205186515), ('usaha', -0.0031926984885307275), ('konsumen', 0.0025602975013300443), ('udah', 0.002245198200260718), ('ni', 0.002242017542751175), ('evaluasi', 0.0018650275798052487), ('defensif', 0.0018346803666619083), ('swasta', 0.0011532925973041725), ('bangkrut', 0.0010847388226533211), ('polri', 0.0010524621511748618), ('pajak', 0.001035571315504909), ('mentang', 0.0009667283232342016), ('tetep', 0.0008247060853226494), ('bintang', 0.000771561276835482), ('atas', 0.0007479600954005501), ('peduli', 0.0007436192427287647), ('sial', 0.0004187457610800691), ('review', -0.00011336842403044791), ('funding', -1.795545017281319e-05)], [('pajakperasrakyat', 0.040660718655933216), ('kerja', -0.02499974727844817), ('sulit', -0.012111817373243466), ('rakyat', 0.008873979405786925), ('basa', 0.008018480442395809), ('cekik', 0.007815204548849918), ('basi', 0.007133286657730821), ('tol', 0.005428263929144406), ('pajak', 0.000706145558042793)], [('pajakperasrakyat', 0.018464615024709433), ('nya', 0.004746543064099507), ('pajak', 0.0032109921654118996), ('nama', 0.0021854650918553746), ('raja', 0.0019221640444906523), ('jaman', 0.0017955140426484531), ('upeti', 0.0017524417766091086), ('pungut', 0.0015120128451177894), ('udah', -0.0007785493643459518)], [('biar', -0.05527297781607883), ('hancur', 0.016700313412021627), ('biasa', 0.01608122738615905), ('tindak', 0.010994953385898164)], [('infrastruktur', -0.061861264788434615), ('pajakperasrakyat', 0.06171005891256873), ('ruwet', 0.02904879135158493), ('harga', -0.022967734318978406), ('pajak', 0.016293760130395858), ('jual', 0.007397252516527086), ('murah', 0.004183519087079163), ('super', 0.0032119265484626025), ('swasta', 0.0031640049221375805)], [('pajakperasrakyat', 0.057940382982940505), ('haram', 0.03061399285970931), ('harta', -0.02995316726589987), ('pajak', -0.011877700480457144), ('riba', 0.009327525820998465), ('sekuler', 0.008129492560891304), ('negara', 0.0005895275985596904)], [('pajakperasrakyat', 0.03218229867326831), ('rakyat', 0.02834640689628092), ('harta', -0.012154696077597981), ('negara', 0.004478727167039567), ('makan', 0.0038802992661586505), ('pajak', -0.003481394821065027), ('izin', 0.0022103228524700583), ('sembako', 0.0020447043136675586), ('syariat', 0.001996677239786452), ('pasir', 0.0012000179047556823), ('batu', 0.0010891320113578205), ('minum', 0.0009383315935996584), ('berkah', 0.0009104506762585244), ('allah', 0.0008649501415180897), ('pake', 0.000753440678978771), ('mana', 0.00045702496934161104), ('ambil', 0.0002014195387907084)], [('pajakperasrakyat', 0.029263715510832698), ('rakyat', 0.026428265840674234), ('harta', -0.016278015559201976), ('hakekatnya', 0.010639440456780263), ('negeri', -0.005721511263920749), ('pajak', 0.004723352146512399), ('makan', 0.004000341254879699), ('hina', 0.0036296499680225163), ('neraka', 0.002767931484944586), ('bara', 0.0027459392563769142), ('api', 0.0024486403635458945), ('kontrak', 0.0024310280568418255), ('perut', 0.0023421894901108774), ('isi', 0.0016964998360431648)], [('sistem', -0.11412168747808756), ('pajakperasrakyat', 0.048047862447996595), ('rakyat', 0.035460145742215385), ('jelata', -0.019275372196006516), ('peras', 0.01495143750424336), ('harta', -0.010337510255754035), ('budak', 0.004200918789992837), ('sejati', 0.003029657973609879), ('upeti', 0.001975270662888242), ('kedzoliman', 0.001845125475631981), ('bahasa', 0.0014195389842985692), ('pajak', -0.0012972371423596095), ('sebelas', 0.0006829296724635203), ('raja', 0.0006208808839337065), ('istilah', 0.0006061634479232239), ('belas', 0.00023788292374449853)], [('pajakperasrakyat', 0.056263546478024014), ('pajak', -0.0177420823080647), ('haram', 0.004926539942129), ('saw', 0.004877033746950536), ('rasulullah', 0.0044554897868653975), ('keras', 0.004250747490442121), ('cela', 0.004100750824620151), ('perintah', -0.003911893481708523), ('islami', 0.0036435483323685723), ('kapitalisme', 0.003583947069746308), ('laknat', 0.0020884468013229835), ('tegak', 0.002082627903124893)], [('gak', -0.019701088135956564), ('pajakperasrakyat', 0.019151417120251184), ('ridho', -0.009255267159174546), ('kalo', 0.0029988343272077016), ('mui', 0.0008818751120460674), ('umat', 0.0007988951202960304), ('muslim', 0.0007400429190377656)], [('nilai', 0.02858736310964411), ('pks', 0.02840178740509487), ('sepakat', 0.02797205297888843), ('laku', -0.023852406475579162), ('pajak', 0.023356510500530867), ('persen', 0.02087457223540933), ('rencana', 0.01778074211969896), ('ppn', -0.01652674092520224), ('tarif', -0.0048448615165147734)], [('pajakperasrakyat', 0.036653829376416425), ('rakyat', 0.012396195292480454), ('pajak', -0.009424817487458055), ('susah', 0.004967846652244826), ('tolak', 0.00019473060058519212)], [('pajakperasrakyat', 0.17415071048517278), ('bayar', -0.08547775149712228), ('pajak', -0.04022306215001293), ('hutang', -0.002035677073814083)], [('pajakperasrakyat', 0.10758239212470817), ('biar', -0.04459139091260942), ('gaji', 0.010571957216083852), ('pajak', -0.01026504667259258), ('lapor', 0.007196361290494826), ('kena', -0.006814956924083482), ('juta', 0.0025913103301222905)], [('rakyat', 0.03743914121051991), ('pajakperasrakyat', 0.034767511685364794), ('bebas', -0.0278859768941453), ('kaya', 0.014684539112046015), ('nelayan', 0.009433249906838949), ('nasib', 0.009353351109502904), ('ironi', 0.008991131518650693), ('sungguh', 0.008043290796668648), ('kasihan', 0.0076373843915973055), ('kena', -0.006095979280539879), ('jokowi', 0.006026642071640177), ('rezim', 0.004509913182972424), ('pajak', 0.00427200986509655), ('orang', -0.0039067953466646104)], [('urus', -0.062288457126637824), ('rakyat', 0.04308439506082972), ('tanggung', 0.006601021773113227), ('cekik', 0.006098564936822494), ('kuasa', -0.003857732182919328), ('sungguh', 0.0035385161363333806), ('terha', 0.003205995083133947), ('pajak', -0.0017650523853081645)], [('rakyat', 0.0365001899496232), ('pajakperasrakyat', 0.027034179503725494), ('bebas', -0.024015808160759364), ('kaya', 0.013872392522700864), ('nelayan', 0.008164247845678587), ('hatinuran', 0.007027353037715197), ('lindung', 0.005861766903350439), ('gede', 0.005546196010357387), ('mana', 0.005275837440886775), ('pandora', 0.005148750476192245), ('cekik', 0.004575579244823745), ('nya', 0.004085392345781187), ('orang', -0.00348426221450468), ('pajak', 0.0006238931682433298), ('gak', 0.0003721861353112816)], [('pajakperasrakyat', 0.020880505558428186), ('rakyat', 0.019189572980714165), ('milik', -0.015314382226078836), ('utang', 0.009181674537888559), ('perintah', -0.007268969715079865), ('nikmat', -0.006764174678773453), ('alam', 0.006720331860009536), ('kaya', 0.00659879617096222), ('negeri', -0.0045033023482200975), ('tumpuk', 0.003931286403021036), ('sayang', 0.0033489037325312135), ('malak', 0.003032523902914174), ('negara', 0.0026073435910923828), ('limpah', 0.0024091639707120474), ('nama', 0.0020533481452119084), ('rasa', 0.0020119806118685662), ('konglomerat', 0.0020106400627947917), ('pajak', 0.0015979239949821134), ('sekarat', 0.0014854800041949002)], [('bayar', -0.0158052481367452), ('tahu', 0.014475489378133853), ('kejar', 0.013983705072155166), ('jokowibencanabagirakyat', 0.01349782276370347), ('ngebacot', 0.013127637056240064), ('suaka', 0.011430511800009924), ('lbp', 0.011143422506861827), ('berani', 0.010668412237364465), ('cangkang', 0.010160227592951973), ('usaha', 0.006848754930814255), ('pajak', -0.0065501674497361875), ('negara', 0.005379934198198838), ('wni', 0.002407692157175151)], [('rakyat', 0.023089492402194153), ('negara', -0.012921335104299794), ('gaji', 0.006694447248395196), ('musuh', 0.005628342418353702), ('perang', 0.004611965912468141), ('asing', 0.00421967790672715), ('tni', 0.003292132038386984), ('pajak', 0.0003045019498076161)], [('aja', 0.03196700125344975), ('ya', 0.02499887488225657), ('hutang', -0.021990105594556355), ('gabisa', 0.0185721932599317), ('ga', -0.017587249620238644), ('nambal', 0.015974680097427734), ('bayar', -0.014060989991993773), ('sih', 0.012573592948603537), ('tetep', 0.012518678515296531), ('cabut', 0.012392529687131905), ('pake', 0.010800524805358019), ('pajak', 0.006635970504953415)], [('tingkat', -0.068806869634926), ('pajakperasrakyat', 0.06633069283779786), ('hutang', -0.019716366200793874), ('beatles', 0.006168882387579557), ('taxman', 0.006042224287920648), ('the', 0.0048244165815557235), ('jerat', 0.0029366357689506848), ('pajak', -3.1363842433633484e-05)], [('aja', 0.02023254131290598), ('hahahaha', 0.010017828322854598), ('kumpeni', 0.00991092038508389), ('zaman', 0.008860915595313735), ('beda', 0.0064063580832345885), ('pajak', -0.0010418465223334952)], [('pajakperasrakyat', 0.04982290626135836), ('laku', -0.03618587665188407), ('jabat', 0.013365645559289483), ('duli', 0.009660088515591457), ('bayar', -0.00885876027327016), ('ama', 0.008622323427667757), ('ngumpulin', 0.008184222754539906), ('entar', 0.008064965299753219), ('sibuk', 0.007590243347663267), ('tengah', 0.007241605518042763), ('maling', 0.0069102968446007685), ('usaha', -0.005187381024394916), ('gak', 0.004582790031687327), ('pajak', 0.0005797726500633465)], [('wajib', -0.151545962177375), ('percaya', -0.030999820590310815), ('paksa', 0.029266127505387544), ('zakat', 0.021911286147311367), ('eropa', 0.020020965160611113), ('negara', 0.01768907464233845), ('musuh', 0.01610311079630086), ('islam', 0.015277559759281129), ('liat', 0.014005862419112844), ('noh', 0.014001312949546303), ('gak', 0.013715636780968501), ('syariah', 0.013558709795586266), ('pajak', -0.005170086656218563), ('agama', 0.004543929712222555)], [('pajakperasrakyat', 0.10519133267152993), ('bayar', -0.05897814507280494), ('kuras', 0.02407148331764183), ('habis', 0.015871156815551426), ('pajak', -0.008017890768226815)], [('pajakperasrakyat', 0.028861336704665768), ('jokowi', 0.007845692458385707), ('pajak', -0.0047978269914098854), ('cepat', 0.004781908462391269), ('bijak', -0.004116847930272767), ('daftar', -0.003710751920154536), ('kereta', 0.0031739537356588452), ('amnesti', 0.0019974594073054143), ('plinplan', 0.0012396659562096365)], [('pajakperasrakyat', 0.03501278542736513), ('rakyat', 0.019119905625281618), ('harga', -0.011152239755597565), ('biaya', 0.004438476046794313), ('makan', 0.003995859466501337), ('pajak', -0.0033068418302096766), ('hidup', 0.0029268549601639882), ('barang', -0.0013573328421464638)], [('sibuk', 0.0357129335094878), ('dasar', -0.031028250730789072), ('pajak', -0.02893092699941166), ('maling', 0.014014835519460192), ('ngumpulin', 0.008526551791340565), ('tikus', 0.007913958578469124), ('rakus', 0.007746584165455631)], [('pajakperasrakyat', 0.02708803112743127), ('rakyat', 0.01435030385095814), ('angkat', 0.008332689228415062), ('paksa', 0.0067843180458379), ('pajak', -0.006273299132890054), ('negara', -0.005648126715603732), ('bantu', 0.00560605028891157), ('sejahtera', 0.005423302038234045)], [('indonesia', -0.09391765715970216), ('pandora', 0.021778372572395167), ('oligarki', 0.017842261607104407), ('kaya', 0.014874682824426678), ('lolos', 0.014743824768727041), ('pupuk', 0.014562361216038421), ('panama', 0.013260559135823748), ('papers', 0.011163104643712065), ('pajak', -0.00855119373890285), ('lapor', 0.008145779994058145), ('negara', 0.002130433866298504), ('fakta', 5.385577058850266e-05)], [('bayar', -0.0687070122682196), ('ogah', 0.03678517066554451), ('kantong', 0.034304146705463646), ('koruptor', 0.023768355124361925), ('masuk', 0.022732755420908198), ('kalo', 0.020884383817305875), ('duit', 0.005242350308100849), ('pajak', 0.0027964484409769793)], [('pajakperasrakyat', 0.025468134016014766), ('tanda', 0.023491598938408572), ('negara', -0.00760889150993308), ('khaldun', 0.006479237198134096), ('hancur', 0.005385306775055189), ('pungut', 0.00508120989370373), ('ibnu', 0.004932717408434012), ('pajak', -0.004694867865729699)], [('ktp', -0.056574524409492594), ('kena', -0.03697703651868098), ('hutang', -0.03652471531638467), ('ber', 0.030977115056620277), ('lahir', 0.027877853255241797), ('kah', 0.026994278136907704), ('pajak', -0.022151495512432506), ('tanggung', 0.021113588180964454)], [('uuhpp', -0.09944496926227969), ('umkm', -0.0945383419399351), ('adil', -0.07701154176680548), ('rp', -0.04237278009539245), ('bebas', -0.02927688395312249), ('masyarakat', -0.02925375076643572), ('bruto', -0.02813331149911594), ('pph', -0.025315960333367336), ('tuju', -0.02444802475415301), ('salah', -0.021993267107859364), ('hasil', -0.021986944387089733), ('pihak', -0.015865489013278184), ('menengah', -0.015053280015497314), ('cipta', -0.014662505872958928), ('tara', -0.011002160316457565), ('bawah', -0.010598060847245436), ('wujud', -0.009861004819218738), ('juta', -0.008449052095343084), ('tahun', 0.0004636229969679117)], [('kemplang', 0.013245043541290852), ('pajak', 0.008437269787894776), ('nyinyirsekarang', 0.00495041473788691), ('maruk', 0.004170605031019588), ('lu', 0.0038800796087975135), ('borong', 0.0036113588282614215), ('harto', 0.003564626395586285), ('jaman', 0.002428907832134244)], [('kemplang', 0.04616855081638848), ('rakyat', 0.03957229058221351), ('diskon', -0.020954604666417496), ('analis', -0.015042541493142755), ('peras', 0.011465517994243614), ('sorot', 0.006678716938268385), ('publik', 0.002741445163750891), ('uu', -0.002071635993111531), ('bijak', -0.0016651806202869625), ('pajak', 0.0008244396515158633)], [('pajakperasrakyat', 0.03786177405968789), ('urus', -0.03481894863181262), ('rakyat', 0.029329767379424387), ('tanggung', 0.004484730956700963), ('ahmad', 0.004353105714663724), ('cekik', 0.0034203819022134434), ('kuasa', -0.0020769067467987035), ('muslim', 0.0015306958076413235), ('sungguh', 0.001088262061011055), ('pajak', 0.0009018158087583944), ('hr', 0.0006919989453643001)], [('utang', 0.04938081815367087), ('naik', -0.04384852358061346), ('mampu', 0.031150019309775307), ('lihat', 0.027381174281129617), ('bayar', -0.02324443719175168), ('agresif', 0.021516301876050945), ('tambah', 0.016924746921305186), ('turun', 0.004394491431186951), ('pajak', 0.0019371288971751959), ('perintah', 0.0005739301611951737)], [('pajakperasrakyat', 0.018929116492288234), ('rakyat', 0.005742959447131305), ('kemas', 0.004782714869967915), ('paksa', 0.00389737977522449), ('sadar', 0.0034139994455511364), ('rampok', 0.0033567954538957705), ('elit', 0.0032057894298535014), ('pajak', -0.00031193780052877644)], [('masyarakat', -0.0471646231424246), ('jajah', 0.02026811158769521), ('pajak', 0.018857177174673525), ('paksa', 0.013402667149174374), ('hidup', 0.009002294740784794), ('sanggup', 0.007435829074779347), ('sebab', 0.005134778267605443), ('asing', 0.004603233642400023), ('kerja', -0.004590108740821826), ('kolon', 0.004191840712374055), ('suku', 0.003999665944159376), ('negara', 0.0036517915485915478), ('hutan', 0.0031980137540028254), ('gak', 0.0030312126318649706), ('masuk', 0.0030253969778183424), ('susah', 0.003007735939098634), ('miris', 0.002979847612600485), ('bayar', -0.001580703613357379)], [('pajak', -0.020079613960057155), ('rakyat', 0.01961653456696499), ('berani', 0.012654972908215137), ('kejar', 0.01236821282037056), ('gede', 0.004459722549920815), ('target', -0.003586451976299876), ('gak', 0.001990726361543189), ('receh', 0.001019021653124536), ('serius', 0.0008083926855166926)], [('pajakperasrakyat', 0.018336717555934558), ('rakyat', 0.009740331271997908), ('kemplang', 0.006917802379413402), ('peras', 0.002990092534328078), ('pajak', -0.0027738159245423496), ('usaha', -0.0024383331431114433), ('manja', 6.75880454062183e-05)], [('pajakperasrakyat', 0.0358802749565965), ('nelayan', 0.013243412114077377), ('ancam', -0.012253022456665383), ('pajak', -0.007227750617829694), ('pungut', 0.005088904039596212), ('laut', 0.004631957007753948), ('pati', 0.004293809045918746)], [('pajakperasrakyat', 0.05455507477167834), ('rakyat', 0.031048444122722826), ('bayar', -0.022966425773655968), ('piye', 0.012961664519535756), ('jabat', 0.01285796862568138), ('gelap', 0.011503626233023284), ('suruh', 0.009349159213157194), ('pajak', -0.005571126867056451)], [('buntung', -0.16470526078592812), ('lemah', -0.1340956136511537), ('harap', -0.13173069640848942), ('naik', -0.09382364953981104), ('perintah', -0.09002968224949745), ('opini', -0.08950998307406971), ('ppn', -0.06332729484967037), ('terima', -0.059849277820267104), ('untung', -0.04087698748611493), ('dampak', -0.03157948745514023), ('ekonomi', -0.029064440827444308), ('pajak', -0.01980734801484288)], [('pajakperasrakyat', 0.07572042751663166), ('kaya', 0.029418600349340143), ('ampun', 0.020401469868168428), ('orang', -0.019286208310306727), ('pajak', 0.015427609466430615), ('ayo', -0.01536374530602643), ('bayar', -0.011539979079907232), ('ya', 0.01063988479380454), ('program', -0.009276080710624463), ('kendala', 0.007704869360314772), ('telat', 0.007401326184719033)], [('pajakperasrakyat', 0.13183435273190847), ('rp', -0.0874054596595303), ('hasil', -0.02963618599015362), ('potong', 0.011837386988251137), ('pajak', 0.011520534209714708), ('nya', 0.010180632633448298), ('kena', 0.00910682709040751), ('ribu', 0.008922179174975451), ('juta', -0.008024757486546286), ('pegawai', 0.005453285569144775), ('uu', 0.003057014531603586)], [('moga', -0.056992823114016544), ('pajakperasrakyat', 0.03402036152109837), ('bijak', 0.014095144695321483), ('pajak', -0.013086683500650247), ('buat', 0.013071342138303339), ('kokay', 0.012918847447024285), ('aamiin', 0.012738166655210825), ('bakar', 0.012530471562609601), ('bahan', 0.012263822027415409), ('mantab', 0.011311779947861119), ('rakyat', -0.009800175948142555), ('neraka', 0.009552728418615286), ('ampun', 0.0083090399073463), ('orang', 0.005751608501207241), ('tambah', 0.00463754259497548)], [('naik', -0.06716658086845602), ('pajakperasrakyat', 0.03387963417364071), ('kemplang', 0.031381538537491184), ('ga', -0.027642542100628976), ('kaya', 0.02675302370006922), ('ampun', 0.01839652356951463), ('orang', -0.0142844256431336), ('pikir', 0.011878656299151497), ('pajak', -0.009984299700269329), ('horang', 0.008866362285169972), ('habis', 0.008853120747642185)], [('pajakperasrakyat', 0.07662758533783426), ('menkeu', -0.033769187551842064), ('nya', 0.011596170651327781), ('solusi', 0.010376853865219271), ('mah', -0.009712242608835174), ('gak', -0.006911971726871038), ('pajak', -0.005614548800713484)], [('pajakperasrakyat', 0.14352363475208724), ('rp', -0.09621954685092923), ('pajak', 0.021694163133437788), ('kena', 0.016936948401080947), ('gaji', 0.008259555268870776), ('atur', 0.006823426638898763), ('juta', 0.0015624634792352904), ('wow', -0.0012156718883274639)], [('perintah', -0.018592053401386387), ('kritik', 0.009462965078162642), ('lambung', 0.008948953921310607), ('rugi', 0.008247235544956773), ('hapus', 0.006517467578608644), ('royalti', 0.006301211983103082), ('kait', 0.00480810024641997), ('batu', 0.004658626437793657), ('bara', 0.004284427225176882), ('rizalramli', 0.0030492113358773793), ('negara', -0.002799614675795699)], [('pajakperasrakyat', 0.10679704647472868), ('bayar', -0.03677959531995258), ('ayo', -0.016498034091508792), ('air', 0.015888063998233552), ('opung', 0.015148063648331092), ('ya', 0.012059048491409628), ('pajak', -0.0003259897035100775)], [('rakyat', 0.023423348045034476), ('kejar', 0.01661426273552491), ('pajak', -0.016159841783159858), ('koruptor', 0.004859911818877318), ('miskin', 0.004757893067738825), ('agree', 0.003134967866812134), ('perasr', 0.0024920433571672756), ('akyat', 0.002187993981281368)], [('rakyat', 0.00852916121079152), ('hapus', 0.004662171350128283), ('peras', 0.004633169337157172), ('nelayan', 0.00384772118436511), ('bikin', 0.0038152228283769925), ('batu', 0.003472920326563171), ('untung', -0.003356390222805904), ('oligarki', 0.003145539332273831), ('bara', 0.003093292799954995), ('rizalramli', 0.002361266027604913), ('royalti', 0.0019405575791262507), ('persen', -0.00027273566233902767), ('pajak', 6.497426927192955e-06)], [('pajakperasrakyat', 0.023402767800809483), ('tanda', 0.021802917921678426), ('negara', -0.006088791163414042), ('khaldun', 0.005822121540298754), ('pungut', 0.005621380615915311), ('hancur', 0.005502083081617791), ('ibnu', 0.005239510728586026), ('pajak', -0.003640130060405197)], [('pajakperasrakyat', 0.05855234282601657), ('pajak', -0.04329154203461206)], [('perompak', 0.03610849767381225), ('haram', 0.03282420667002936), ('bayar', -0.01957465825625303), ('sembunyi', -0.017702575173321532), ('ampun', 0.014870722967655854), ('pajak', 0.01278680070135548), ('korupsi', 0.011362220288076524), ('denda', 0.009172007139426916), ('jaya', 0.008040593094477478), ('negara', 0.005371490445982145)], [('pajakperasrakyat', 0.10227707432372966), ('ga', -0.04097069116112542), ('bayar', -0.02611462414097798), ('nganggur', 0.023845083325002618), ('penjara', 0.021705855847433394), ('jajah', 0.020982533882636864), ('rezim', 0.018325342288427887), ('kena', -0.005014124474551815), ('pajak', -0.0019975385981740286)], [('pajakperasrakyat', 0.06061186116916901), ('tertawa', 0.016669828723735725), ('pajak', -0.006463663538070749)], [('pajakperasrakyat', 0.045206347515821144), ('hasil', -0.024137388272357264), ('rakyat', 0.02239792387045895), ('amnesti', 0.005694732811045102), ('pajak', -0.005350007963801382), ('yaak', 0.005124743801824239), ('pas', 0.005069382599082143)], [('negara', -0.021001801158511926), ('rakyat', 0.019538238835747294), ('pajak', -0.016860237385626593), ('ragam', 0.014881222668549427), ('aneka', 0.013387059147799165), ('tanda', 0.01251526441954383), ('pungut', 0.011537650270057919), ('hancur', 0.011379788174719548)], [('pajakperasrakyat', 0.2013547968102424), ('wajib', -0.06707014467479941), ('nik', -0.05024784102689027), ('npwp', -0.028416187392429487), ('milik', 0.021023419889891665), ('pajak', 0.00582489387510701), ('ditjen', 0.0023484025686925187)], [('rakyat', 0.06578676675472811), ('taxamnesty', -0.039051812795393014), ('pajak', -0.03506631426885305), ('cukong', 0.020971658258525917), ('nak', 0.0197312668317136), ('hebat', 0.018993276171162084), ('kasih', 0.018318894212899155), ('batubara', 0.01806399942700615), ('hapus', 0.015357157566681589)], [('rakyat', 0.041430239703874897), ('salah', -0.03843988828429031), ('indikasi', 0.011998846779467392), ('gila', 0.011214607436818975), ('preman', 0.010081503390646077), ('bangkrut', 0.009948275165084387), ('beda', 0.008700988969128865), ('pungut', 0.006984369132268279), ('negara', 0.006727896712092318), ('pajak', -0.004911769569438331)], [('pajakperasrakyat', 0.016008110503793995), ('bikin', 0.004231720601414941), ('bolong', 0.003558017929122372), ('kantong', 0.0034679978062213284), ('makan', 0.0027356165308617827), ('belanja', 0.002590605983432554), ('pajak', 0.0025806491594662934), ('baju', 0.0025086079023450723)], [('pajakperasrakyat', 0.029075108283113063), ('jajan', 0.0066822118106150555), ('mahal', 0.006290143903226337), ('makan', 0.005898971423774465), ('baju', 0.005730199653839423), ('pajak', -0.0018853478935776727)], [('pajakperasrakyat', 0.01886790693216627), ('rakyat', 0.005467878346300645), ('sadar', 0.003999667854050306), ('kemas', 0.00394653652194756), ('paksa', 0.0034347951574755188), ('rampok', 0.0032260663374386422), ('elit', 0.0031453725664273317), ('pajak', 0.000552570337321069)], [('pajakperasrakyat', 0.05288728236427649), ('pajak', -0.01992594074998467), ('negara', -0.019046212555755324), ('rakyak', 0.012379852319466287), ('kritis', 0.010624850856776238), ('beban', -0.004902802454180094)], [('pajakperasrakyat', 0.030323939359811734), ('ga', -0.02941596389035436), ('asn', 0.011711642254754054), ('karayawan', 0.011566491153926493), ('kaya', 0.011421036662905374), ('nyaman', 0.010364381902068556), ('jabat', 0.009524738521726835), ('fasilitas', 0.006972501964400338), ('gaji', 0.004747887296521152), ('pajak', 0.0018263845464734742)], [('hindar', -0.04516312700330606), ('pajakperasrakyat', 0.03599147108305782), ('rakyat', 0.025315249349195712), ('jabat', 0.012405412175970184), ('pajak', -0.004197759431267229), ('konglomerat', 0.003838664699537509), ('dipajakin', 0.003205326193210887), ('dibebasin', 0.002968066941382458)], [('kerja', 0.07395654453280634), ('biar', -0.06375206647980469), ('pajak', -0.014624503921678494), ('kena', -0.013508076570180898)], [('pajak', -0.029058900085749365), ('dipajakin', 0.011057175470300457), ('konglomerat', 0.009452761370360184), ('rakyat', 0.0018989231761625403), ('dibebasin', -0.0008696024102897282)], [('pajakperasrakyat', 0.029810914475354732), ('menteri', -0.028289705545910628), ('pajak', -0.011630169082149668), ('beban', -0.011587323417299256), ('nama', 0.009312997204727991), ('waras', 0.008486780580320985), ('situ', 0.0068958253392135075)], [('taxamnesty', -0.15712268652044895), ('pajakperasrakyat', 0.09865325882928917), ('rakyat', 0.05971468276648342), ('pajak', -0.027835785600950128), ('usaha', 0.006954301574089746), ('ringan', 0.0005028139545487731)], [('kemplang', 0.05637065918612513), ('indonesia', -0.05408845579194486), ('rakyat', 0.04012120551738714), ('pajak', -0.01514758312834013), ('enak', 0.012763421067001698), ('peras', 0.010383341683508336), ('sanksi', 0.007500429635360774), ('pidana', 0.007439152433748939), ('ringan', 0.006818808132753316), ('habis', 0.003632435167597832)], [('karbon', -0.2689472741621054), ('ri', -0.26498540864037545), ('seram', -0.10245013014504684), ('selamat', -0.08435516623535036), ('covid', -0.05726223980064452), ('ancam', -0.0494818493444044), ('pajak', -0.008780563111845826)], [('rakyat', 0.11877084880615839), ('bayar', -0.07114789975699166), ('hutang', -0.03410186799258677), ('utang', 0.02969039643333205), ('pajak', -0.021173607918271345), ('negara', -0.01604273118242091), ('raksasa', 0.013768042911599788), ('segi', 0.013133970537782005), ('bunga', -0.011422450171412222), ('usaha', 0.01071032013825726), ('aneka', 0.009532894188182617), ('hancur', 0.009437553576187611), ('tanda', 0.009330888239514171), ('sanggup', 0.008767575952334551), ('cekik', 0.007935386314605107), ('gunung', 0.00724527582886214), ('bebas', -0.006854281720328717), ('palak', 0.00666693206450621), ('ekonomi', 0.006357519568999633)], [('rakyat', 0.14808436826794755), ('jabat', 0.06248174017798764), ('bebas', -0.04865287126395741), ('bayar', -0.04709436578080147), ('ngemplang', 0.03721823182275464), ('suruh', 0.03445981897666104), ('biar', -0.025914767838417355), ('pajak', -0.005875906122544633)], [('rakyat', 0.15097957530763303), ('jabat', 0.0617775172107868), ('bebas', -0.048049263708395115), ('bayar', -0.04517186322593818), ('ngemplang', 0.03773140689687229), ('suruh', 0.034358131284166524), ('biar', -0.025373913051330493), ('pajak', -0.006180202606735661)], [('tingkat', -0.051830763192936946), ('rakyat', 0.03908690668818661), ('korup', 0.005878023514897623), ('pinangki', 0.005468708519392676), ('bencana', 0.004002894135728168), ('pajak', 0.0036518619734630256), ('palak', 0.003405365474329224), ('kian', 0.003145021280126261), ('gaji', 0.002602102430064615)], [('orang', -0.023276243582856102), ('pajakperasrakyat', 0.01843343782594844), ('yoo', 0.008817898555352135), ('kayah', 0.008493996632928873), ('ampun', 0.008233487768971889), ('pajak', 0.007115615099875986), ('kalo', 0.006704633473361327), ('enak', 0.00604494150554), ('ngemplang', 0.005844114982781262)], [('pajakperasrakyat', 0.0383728586413924), ('bayar', -0.01806286563707425), ('sitiii', 0.011782169020112407), ('bagong', 0.01156326319722292), ('utang', 0.010946640993097025), ('riba', 0.010548412572694371), ('sungguh', 0.008736794871176849), ('pajak', 0.00551213344469901), ('akselerasi', 0.001277707773716884)], [('rakyat', 0.0500053480460538), ('pajakperasrakyat', 0.02955164638306822), ('bebas', -0.018628143645291206), ('kaya', 0.012405110689969691), ('jabat', 0.011699733484269411), ('pajak', 0.009290391376412546), ('harta', -0.008022656838203185), ('harga', -0.006577524189198985), ('ppkm', 0.006487544493263563), ('drat', 0.005527598735752959), ('gede', 0.004402828688403179), ('koar', 0.0038853195483560025), ('nkri', 0.003687274113431389), ('dinaikin', 0.0030772984977902418), ('mati', 0.0022647811814747796), ('usaha', 0.0008466917242606479)], [('pajakperasrakyat', 0.07034117656672209), ('rakyat', 0.02430811592238067), ('dasar', -0.018892488550506417), ('kena', -0.01131898025402034), ('pajak', -0.008448115340774586), ('bikin', 0.007833148671692545), ('berhala', 0.004136516532408119)], [('hutang', -0.026473000346895238), ('grassroot', 0.02418228250689132), ('persepsi', 0.02370477797180715), ('bayar', -0.022545660496421874), ('cuman', 0.022252433210273082), ('kalang', 0.020467920011504537), ('timbul', 0.020252866114566954), ('atik', 0.01968010708238946), ('utak', 0.017998873689971977), ('naikin', 0.01699164938399948), ('pajak', 0.009518396761294972), ('negara', -0.006790700796518046)], [('rakyat', 0.06856720112193648), ('jenius', -0.04633752587039419), ('pajakperasrakyat', 0.03760071009006474), ('bebas', -0.034293792671311635), ('kaya', 0.033645407969020914), ('cuba', 0.029690204976945505), ('mobil', 0.021281993320814645), ('dipajakin', 0.020572465141696424), ('beli', 0.01964908284285994), ('sembako', 0.01689361320999048), ('mewah', 0.01526041762970734), ('pajak', 0.014129999779575905), ('orang', -0.008984922139972382)], [('pajakperasrakyat', 0.05782759148820961), ('negara', -0.025839770560898295), ('pajak', -0.025187652979604538), ('tanda', 0.006693572889406507), ('hancur', 0.005153018474516771), ('pungut', 0.004774314584400917)], [('rakyat', 0.036353638233583424), ('negeri', -0.01817138261894267), ('lepas', 0.01476438585313553), ('nasib', 0.013703297994234597), ('wakanda', 0.013370862355850536), ('bayar', -0.010794980556126875), ('bikin', 0.010660905915464118), ('suruh', 0.010077302751791285), ('pajak', 0.009983920714416202), ('ngutang', 0.009835276748543838), ('hidup', 0.009304507748454816), ('tuh', 0.009246981089001736), ('kuasa', 0.003674049023597731)], [('moga', -0.05901366078923482), ('nya', 0.022780321654663885), ('ariel', 0.020652966764844856), ('seprti', 0.019837988639733288), ('shanon', 0.019677801638640056), ('ide', 0.019094373797264415), ('dukung', 0.012011446110143749), ('pajak', 0.007065833610140868), ('tari', 0.006929345824742421)], [('rakyat', 0.04639562165491566), ('penghianat', 0.03335310971924154), ('harta', -0.013867005696178944), ('keruk', 0.012879726637072705), ('pajak', 0.008811759998294016), ('nama', 0.007247298377739793), ('orang', -0.004765876139966042), ('negara', 0.00030020122179495203)], [('rakyat', 0.050813680087605714), ('mudah', -0.04658928304395141), ('kaya', 0.030604837639730126), ('nelayan', 0.025280760759814357), ('ratus', 0.01473804085239213), ('sanggup', 0.012009186338543403), ('kapal', 0.01124296678087211), ('laut', 0.011078489420646676), ('orang', -0.0029342908038982735)], [('kabur', 0.01854116466784807), ('pandora', 0.013394504264248172), ('papers', 0.010724053963578431), ('bawa', 0.009136760704404315), ('pajak', -0.004609074432696956)], [('jalan', -0.04660198509105438), ('jelata', -0.028266291775064886), ('rakyat', 0.02797571335467062), ('pajak', -0.0029646651506700945), ('pake', 0.0012924092512492762), ('bisnis', 0.001146451611540387), ('nanggung', 0.0008132100912065107), ('telur', -0.0007472751586881241), ('bank', -0.0006849872455938786), ('tol', -0.0005871382208254079), ('nama', 0.0005856636206646745), ('pinjam', 0.00057299876472963), ('emas', -0.0004805484572396175), ('ayam', -0.0004480273588448452), ('ambil', -0.00042277768446797594), ('miliar', 0.0003886967831814285), ('dsb', -0.0003710020597076041), ('postingan', 0.00015110184220552874), ('jual', 0.0001459607298511879), ('tolol', 9.493431672285416e-05)], [('pajakperasrakyat', 0.01576088080964252), ('pajak', 0.0038580984429207326), ('mahal', 0.0035736990085400927), ('ekonomi', -0.002677783219696267), ('hidup', 0.00236492448515486), ('merosot', 0.0008768665905269073), ('stres', 0.0007473924730746907), ('bbm', 0.0007289785783594027), ('listrik', 0.0007005190466568045), ('pusing', 0.0006646770663813327), ('biaya', 0.00038808958205237845), ('dll', 0.00037112812159765623)], [('pajakperasrakyat', 0.036635972418435865), ('pajak', -0.013710910649069508), ('nelayan', 0.009031685184316594), ('hapus', 0.005111910245476821), ('oligarki', 0.005064310850067895), ('untung', -0.004666856836217829), ('bara', 0.0039530367460970505), ('royalti', 0.003949083005858338), ('rizalramli', 0.002841375712915048), ('peras', 0.00239198572052529), ('persen', -0.00034231674004486724)], [('rakyat', 0.03590140622380537), ('miskin', 0.027381958910939655), ('kemplang', 0.026291469689853988), ('taxamnesty', -0.025813630284494012), ('kaya', 0.021383445399649377), ('jabat', 0.015544697449347758), ('peras', 0.011523058643819244), ('pajak', -0.011269789079003133), ('raya', 0.00960853482243734)], [('rakyat', 0.0437769383873283), ('taxamnesty', -0.03369036666031865), ('kemplang', 0.03144628446393255), ('kaya', 0.03023238876579895), ('raya', 0.01620619562345262), ('jabat', 0.015390943936803016), ('peras', 0.01410969321687536), ('pajak', -0.013313675357034092), ('miskin', 0.012409474397575058)], [('kaya', 0.046588873072335606), ('orang', -0.045924771761343706), ('pengemplanng', 0.03634625344681836), ('pajak', 0.01118239929411693)], [('pajakperasrakyat', 0.005624478683886253), ('aja', 0.002711480836659557), ('pajak', 0.0020626983778179843), ('kejar', 0.0020476387401829202), ('rakyat', 0.0012860410632017343), ('pandora', 0.0007185884598073579), ('eiitz', 0.0006194604841793081), ('blom', 0.0006053665423706397), ('dah', 0.0004918343766982062), ('lakon', 0.00042950615107492754), ('papers', 0.00040969515415209865), ('cekik', 0.0003331895750897869), ('hisap', 0.0003028306283315436), ('darah', 0.00021556932804834525)], [('rakyat', 0.043202194404594024), ('naik', -0.03469520028496886), ('rugi', 0.02354270862180113), ('pajak', -0.01906997515625543), ('paksa', 0.012276277851426593), ('hapus', 0.010121770596616519), ('uang', 0.007366418727693942), ('tambah', 0.0060090575790244), ('dapat', -0.00504295026985319), ('keluar', 0.003861062704603094), ('bijak', 0.003242611125608759), ('negara', 0.00209223108792458), ('hilang', 0.0019166205591727445), ('bangun', 0.0013299755483693616), ('sempat', -0.0008532211343141856)], [('pajakperasrakyat', 0.06346560317972837), ('rakyat', 0.05654016316307924), ('bebas', -0.032113851334336425), ('ampunn', 0.013816034661831374), ('ku', 0.013107175372599915), ('deh', 0.012170759698894138), ('tekan', 0.011888126788916695), ('negeri', -0.009640812924723746), ('cukong', 0.009239937043697714), ('sengsara', 0.0004949603910976068), ('pajak', 0.0003056369798769946)], [('pajakperasrakyat', 0.09472374635116007), ('bayar', -0.02681308710496052), ('kerja', -0.02218665450078011), ('rakyat', 0.021137912500413684), ('pajak', -0.0048477572690368426), ('keras', 0.002146667529044362)], [('pajakperasrakyat', 0.0910757807352364), ('kena', -0.01905622809864173), ('pajak', -0.0063942635579535756), ('ntr', 0.004304706349755045)], [('bebas', -0.13519651215358514), ('rakyat', 0.1277274313395403), ('tekan', 0.04801339442136434), ('nya', 0.025939891917391118), ('pajak', -0.0172529375587138)], [('rakyat', 0.06998820334887923), ('bebas', -0.06061041809143281), ('pajakperasrakyat', 0.056646272377578674), ('tekan', 0.021844985003510477), ('emen', 0.0207075093795529), ('enak', 0.013680662289723287), ('cukong', 0.012146807365060503), ('pajak', 0.0017821188210766553)], [('pajakperasrakyat', 0.03956188016851147), ('rakyat', 0.029059046166638448), ('nikmat', -0.019122688134403803), ('jabat', 0.01782497502884206), ('pangeran', 0.009228403957307731), ('bayar', -0.008360265504722329), ('bak', 0.008108800014766413), ('budak', 0.006823430415950993), ('pajak', -0.00019873224731065697)], [('pajakperasrakyat', 0.039063112391550595), ('pajak', -0.011460447711515788), ('preman', 0.01009543864471777), ('paksa', 0.007426985692152035), ('uang', 0.0017952826835670437)], [('pajakperasrakyat', 0.02723951241620934), ('mahal', 0.0031822121883925735), ('ikan', 0.0013243424886812117)], [('pajakperasrakyat', 0.1138941000557517), ('hutang', -0.048011475049088245), ('negara', -0.045774701166362376), ('pajak', -0.0349102013895344), ('jual', 0.008508236970153174), ('tol', 0.00507986806893963)], [('bebas', -0.08262879373927175), ('pajakperasrakyat', 0.08001396250159448), ('rakyat', 0.06880871692984893), ('bodo', 0.032430137333584165), ('gw', 0.029743609538532327), ('pajak', 0.005589952041011358)], [('rakyat', 0.07939092790299113), ('yaa', -0.039568211290241316), ('ampuunn', 0.02415964707082956), ('astaghfirullohalafziim', 0.024095290720989586), ('alaikumussalam', 0.022459413044748227), ('susah', 0.018266919776918102), ('wa', 0.017384891456524373), ('negeri', -0.015345520863864068), ('kena', -0.010527703596482047), ('pajak', -0.00025829196361206494)], [('pajakperasrakyat', 0.18795980982137167), ('wajib', -0.08944101656090153), ('npwp', -0.026988261070889474), ('ktp', -0.01661002554743642), ('langsung', -0.0015211170852974852), ('pajak', 0.00016380743245240911)], [('pajakperasrakyat', 0.040909996202419106), ('pajak', -0.008664985214107806), ('bikin', 0.006840497524598024), ('rakyat', 0.006797138737584998), ('sengsara', -0.005253054465798247)], [('sistem', -0.07661392318279915), ('masyarakat', -0.07384161956334463), ('rakyat', 0.04134582781469095), ('peras', 0.030259207569617703), ('jabat', 0.02899108273086993), ('kapitalis', 0.01880749632020562), ('ampun', 0.016281831932216034), ('kongklomerat', 0.014768292782067163), ('emang', 0.013807856407661955), ('tindas', 0.013405561384998767), ('pil', 0.012077353785826445), ('sadar', 0.006787441543250027), ('pajak', -0.003241073042500599)], [('pajakperasrakyat', 0.0831369328911291), ('jenius', -0.06341792528357584), ('horang', 0.0604751036731893), ('libur', 0.022933092489947297), ('otak', 0.019661468228010723), ('kayah', 0.018068317953151777), ('miskin', 0.016529607692479648), ('pajak', 0.0003214383351639316)], [('pajakperasrakyat', 0.03587160750564434), ('rakyat', 0.020940628587932886), ('hasil', -0.018575461800863755), ('pora', 0.006390786144361883), ('pesta', 0.005657069475518398), ('korupsi', 0.00497810839760116), ('utang', 0.0048688452589596976), ('elit', 0.004414353099174286), ('negara', -0.004061573208200994), ('tanggung', 0.002983801341201166), ('pajak', -7.943498555462377e-07)], [('cepat', 0.007474006729207754), ('aja', 0.005137047698676852), ('kereta', 0.004425506447670253), ('bohong', 0.002987525681459644), ('jokowi', 0.0008932935370198231), ('tu', -0.00019620593229313974), ('tetep', -0.00016544109109495133), ('murah', -0.00015581919101109599), ('bus', -0.00011153031599989727), ('bodo', -0.00010723271240596489), ('sptnya', -0.00010571510246466589), ('gw', -3.617852617378062e-05)], [('pajakperasrakyat', 0.043598117183649865), ('infrastruktur', -0.031629469725808004), ('rakyat', 0.018680941129329776), ('aset', -0.008353468685048564), ('jual', 0.004402463060369497), ('rugi', 0.0029619424278061416), ('didzolimi', 0.0028261233310606267), ('an', 0.002483648576417549), ('ugal', 0.002120035179116671), ('ahirnya', 0.0015779243310405414), ('tambah', 0.0014224667950438587), ('los', 0.0013906371823155916), ('emosi', 0.0013704982582585753), ('korban', 0.0013604345489003575), ('kontrol', 0.0011644730103474738), ('citra', 0.0011308630684205097), ('hutang', -0.0010602361982567728), ('kelola', 0.0007989076034571399), ('hitung', 0.0005301743503849513), ('jadi', 0.0001445946434827456)], [('salah', -0.04508503916788847), ('kerja', -0.02831970495385146), ('perhati', 0.02153447872721475), ('rakyat', 0.01478873879056012), ('emis', 0.014642187611308324), ('negara', 0.01357575897101004), ('lapang', 0.012378135409516417), ('miris', 0.008938004141206508), ('penuh', 0.007652957191176032), ('pajak', 0.0069118141693048476)], [('pajakperasrakyat', 0.06040658105686911), ('cak', 0.013269223907518588), ('pajak', -0.006181707755842035)], [('rakyat', 0.010323978696494051), ('pajakperasrakyat', 0.0057008083106589785), ('beban', -0.005672701524370238), ('pajak', 0.0031999856310023227), ('utang', 0.0030332777010905875), ('inna', 0.00200094820535363), ('ujung', 0.0010745393241719781), ('ilaihi', 0.0009877316615179753), ('gunung', 0.0009297670753908411), ('wa', 0.0007702946768245112), ('lillahi', 0.000713583889139892), ('rojiun', 0.0006960836729736719)], [('pajakperasrakyat', 0.020612619848059927), ('rakyat', 0.01536821249511004), ('brarti', 0.007850648689018214), ('makmur', 0.007680140482607349), ('sda', 0.00749768771618998), ('kelola', 0.005668031896453431), ('negara', -0.0056137963765094355), ('pajak', -0.004370861134083467), ('beban', -0.0024724222349908793)], [('pajakperasrakyat', 0.10761253152927464), ('bayar', -0.047450125059446964), ('pajak', -0.018013321070649024), ('negri', 0.014782485814577756), ('nya', 0.013766983637593506), ('paksa', 0.01080981666199014), ('korupsi', 0.010086793735736328)], [('pajakperasrakyat', 0.07218834569382802), ('rakyat', 0.051937780166573994), ('bayar', -0.05103165602376973), ('pajak', -0.013581710079030307), ('utang', 0.012570210083856807), ('mahal', -0.0008802656475299803)], [('pajakperasrakyat', 0.07377743385068233), ('dasar', -0.02739128435527444), ('klasifikasi', 0.008685913271558036), ('pajak', -0.005714428152117663), ('tinggal', -0.00538318891201406)], [('pajakperasrakyat', 0.04484592895830902), ('pajak', -0.014535520850258757), ('rakyat', 0.013086006875149901), ('cm', 0.0010482902204800876), ('tauk', -0.00017091988819529925)], [('pajakperasrakyat', 0.03085837423942102), ('kemplang', 0.01276093670078475), ('manja', 0.004815687136881532), ('amnesty', 0.004358351481780427), ('pajak', -0.004127157243787784)], [('rakyat', 0.07936839627826685), ('biar', -0.06480238592890304), ('bayar', -0.048943669496716476), ('mukidi', 0.03286850714680684), ('kolaps', 0.03088393501588622), ('ni', 0.02716683075487718), ('rezim', 0.019594374539400043), ('pajak', -0.016801017679611615)], [('sengsara', -0.01564583715548369), ('walah', 0.008877531777333999), ('wa', 0.006226523073311358), ('kayak', 0.006220571231368181), ('alaikumussalam', 0.005688451126011262), ('rakyat', 0.004779811135937705), ('berhala', 0.004396247540971771), ('pajak', 0.004303949390273319)], [('pajakperasrakyat', 0.06791170021204385), ('pajak', -0.03234760213685977), ('oh', -0.010629706259288971)], [('pajakperasrakyat', 0.03202499301584024), ('kemplang', 0.012875953107430532), ('amnesty', 0.004933775315372777), ('manja', 0.004583632742606483), ('pajak', -0.004311777015470188)], [('pajakperasrakyat', 0.0698841918407035), ('pajak', -0.018789194448524837), ('uu', -0.008384984942371157), ('tebar', 0.006613976668785402), ('jerat', 0.004184694192580603), ('sah', 0.0034350484491458243), ('pungut', 0.0005734623875353034)], [('pajakperasrakyat', 0.1056309975096328), ('mudah', -0.05664472467066713), ('pph', -0.03632407252674636), ('pribadi', -0.013631123198805304), ('awam', 0.0100640817219456), ('mungut', 0.009840479605887987), ('kait', 0.008549420099745684), ('orang', -0.002594042340413666), ('pajak', -0.0016406846340234402), ('atur', 0.0005755601811322314)], [('pajakperasrakyat', 0.05581339293170515), ('ngumpetin', -0.031841263103389576), ('jabat', 0.013688188322946281), ('sue', 0.010502175049142972), ('pajak', 0.00501541264862769)], [('pajakperasrakyat', 0.012325180508305366)], [('pajakperasrakyat', 0.029804780216827496), ('menteri', -0.028930507569753686), ('pajak', -0.01196659376366205), ('beban', -0.011643654102313742), ('nama', 0.008923385401448148), ('waras', 0.008108942978028073), ('situ', 0.006840800572783811)], [('pajakperasrakyat', 0.012325180508305366)], [('pajakperasrakyat', 0.02422193231981028), ('kemplang', 0.010479272683984162), ('perintah', -0.008410821745007402), ('rugi', 0.004559884457121981), ('ultimum', 0.004439602030167323), ('remidium', 0.004336237379023645), ('depan', 0.0042941455847879915), ('ganti', 0.0038709314178223906), ('sanksi', 0.0024888746819167803), ('negara', -0.0017286383773068735), ('tambah', 0.0003215998548446511), ('sempat', 0.0003112366325814791), ('pajak', 8.411734626006734e-05)], [('pajakperasrakyat', 0.028813090090367453), ('utang', 0.007664429873725009), ('cinta', 0.006965627445787846), ('iya', 0.006483741334203331), ('mati', 0.00466311266866701), ('pajak', -0.0020756300418691385)], [('laku', -0.04254408971505475), ('pajak', 0.014253229617269964), ('dzhalim', 0.011382464063430204), ('cukai', -0.009282289030782369), ('makan', 0.00834432276968309), ('aniaya', 0.0067218568207498535), ('bunuh', 0.00568585584319449), ('mungut', 0.004717086221482042), ('barang', 0.00399032032311198), ('haram', 0.002880545443924418)], [('maju', -0.22723924331947723), ('rakyat', 0.13144640269088775), ('pajak', -0.06235351642350525), ('hutang', -0.04822460541791102), ('negara', -0.036475312364395746), ('sengsara', -0.014303741844126134), ('sejahtera', 0.006271097747266065), ('kelola', 0.0019769446609873217)], [('pajakperasrakyat', 0.03887632434522377), ('rakyat', 0.013098891265271495), ('pajak', -0.010891732853677186), ('nyusahin', 0.0041755864230227785), ('akyat', 0.0028864645762312134), ('perasr', 0.002571005821422754), ('hidup', 0.0018275952055298163)], [('pajakperasrakyat', 0.014763985614931922), ('duit', -0.00918079962894759), ('rakyat', 0.004915045397835961), ('pajak', 0.003121094182078581), ('ciprat', 0.0011637507387962046), ('mom', 0.0008870584696658869), ('kalo', 0.0007759906910003121), ('deket', 0.0007361785235235035), ('pajakp', 0.0007343846051567412), ('eras', 0.0007304079768587823)], [('pajakperasrakyat', 0.05512801158749447), ('ga', -0.0523412468926954), ('bayang', 0.01189602032974129), ('kuota', 0.011860019294058314), ('yah', 0.011797930723247864), ('pajak', -0.008357829172590642), ('beli', 0.006380712823913432)], [('ruuhpp', -0.06017747965329033), ('pajakperasrakyat', 0.051518073188257296), ('kemplang', 0.03772338103711702), ('tajam', 0.011982319402386893), ('pidana', 0.011896839421872599), ('kesah', 0.011744216452817003), ('sanksi', 0.010247437924553201), ('kritik', 0.00966194774500659), ('didu', 0.00915738321709645), ('said', 0.009087748356084252), ('hilang', 0.008798312469294536), ('pajak', 0.004100606368376351)], [('rakyat', 0.05897136190760733), ('jalan', -0.05312744953329631), ('salah', -0.03388071391189016), ('negara', 0.014591922367339414), ('gila', 0.011431070918435267), ('beda', 0.010249835960830515), ('indikasi', 0.008569062432547752), ('laper', 0.008356401343730379), ('anggur', 0.007636714568639471), ('preman', 0.006785317075435146), ('bangkrut', 0.005512664159606262), ('pungut', 0.005144700066926703), ('malak', 0.004431128675828605), ('paksa', 0.003990572546717499), ('pajak', -0.0016023875157368044)], [('terima', -0.01606730325594252), ('negara', 0.01138421971560218), ('utak', 0.004816719146607327), ('ngotot', 0.004525939134094669), ('focus', 0.0036324222073733665), ('evaluasi', 0.0035491374972122978), ('pajak', 0.0030748238306348217), ('sih', 0.0027902557683066936), ('atik', 0.002657508111879003), ('bagent', 0.0022621640761943747), ('deh', 0.002233073276514983), ('coba', 0.0014884658413291468)], [('pantes', 0.05043319102731207), ('aja', 0.033766849002676874), ('pajak', -0.02033787272140474), ('kena', -0.018140829876317745), ('dinaikin', 0.00726171974316506)], [('pajakperasrakyat', 0.04766336890953427), ('rakyat', 0.03347406549395836), ('bayar', -0.021748816808907553), ('melarat', 0.016804816974232632), ('keblangsak', 0.015535401603860802), ('jabat', 0.009518032634978283), ('pajak', 0.0005313638658053771)], [('bayar', -0.041910764032840914), ('haven', 0.041628413526426246), ('politik', 0.02952684570348746), ('cangkang', 0.028648427215282254), ('elit', 0.026450178103143283), ('bikin', 0.0250388733306032), ('tax', 0.019219217897496543), ('usaha', 0.01055968970876223), ('negara', -0.0063736435947936605), ('pajak', 0.0051528279102247)], [('rakyat', 0.0640631763328171), ('bebas', -0.06262743894369759), ('pajakperasrakyat', 0.05862106068810412), ('kolonial', 0.027495507435501502), ('zaman', 0.026494230318998623), ('serasa', 0.024990834730435497), ('pajak', 0.01085872364123468)], [('ampun', 0.024579200131189742), ('jilidii', -0.02189110537128522), ('nilai', 0.018763081299192432), ('ekonomi', 0.013906097213185914), ('bhima', 0.010908262694805826), ('yudhistira', 0.010404207811730667), ('celios', 0.009785810215501848), ('pajak', 0.009487746006984068), ('efektif', -0.0058053340333136475), ('direktur', 0.002701228041410086)], [('pajakperasrakyat', 0.03182804414564915), ('rakyat', 0.025878885888171197), ('bayar', -0.01765360663320081), ('yan', 0.011155843344877365), ('keblangsak', 0.010713450205849417), ('melarat', 0.009811133929509332), ('jabat', 0.007102325606094896), ('pajak', 0.002731211011148767), ('g', 0.0012444019788573695)], [('masyarakat', -0.105524697946466), ('jokowi', 0.039290927608630476), ('perlahan', 0.0240545311035644), ('tunjang', 0.023669836756709122), ('bohong', 0.02318585276928046), ('cabut', 0.02314900294689337), ('ambil', 0.019544730823147005), ('bijak', 0.010275649996084933), ('hutang', -0.010006797271924826), ('bilang', 0.007868523870482664), ('pajak', 0.0073484373016610965)], [('pajakperasrakyat', 0.034263662444254106), ('rakyat', 0.013435635561056185), ('pajak', -0.008054239205608637), ('sembako', 0.004504934749442444), ('cekik', 0.004258112527115253)], [('pajakperasrakyat', 0.05484577919238568), ('kena', -0.017745694016017104), ('aja', 0.009523550459534921), ('bayi', 0.004916261106580402), ('sampe', 0.004777230884049383), ('peras', 0.00345153141330349), ('pajak', -0.002884053756667535)], [('pajakperasrakyat', 0.10735285009226304), ('ppn', -0.02099083143818961), ('nelayan', 0.012658217032324413), ('kena', -0.01044853580909877), ('pajak', -0.0075291639243482205), ('iya', 0.004898433356561633), ('kali', 0.004600374775217094)], [('tingkat', -0.033271627383612216), ('hasil', -0.030734259585257065), ('pajak', -0.023028542144507613), ('mafia', 0.016094371226401318), ('miskin', 0.012789539273352931), ('ratingnya', 0.01175990200705519), ('politik', 0.011319659320681175), ('drastis', 0.007519404596439112)], [('taxamnesty', -0.1305064666420734), ('kemplang', 0.09108203854633355), ('rakyat', 0.04801404429255306), ('usaha', 0.04254966491423155), ('diam', 0.03670891295124311), ('pajak', -0.03297561668339726)], [('pajakperasrakyat', 0.015880143308585625), ('rakyat', 0.013403451643673292), ('kejar', 0.003825962799906487), ('progresif', 0.003722412039161837), ('nya', 0.003342835890118253), ('pajak', 0.0028901624023400015), ('amnesti', 0.0025022382650422793), ('usaha', -0.0021463593659157834), ('tax', 0.0011622952825883523)], [('integrasi', -0.2333226490440165), ('mudah', -0.14014156244750325), ('nik', -0.12970947399370072), ('wajib', -0.12531660330937905), ('npwp', -0.08296780686142667), ('pribadi', -0.06685432323288187), ('pajak', -0.04196586972616052), ('orang', -0.002543646799135568), ('bayar', -0.0007449137440377856)], [('uuhpp', -0.21449318347021235), ('pajakperasrakyat', 0.08678498212550628), ('kemplang', 0.07137899281928758), ('pidana', 0.022386020770106946), ('denda', 0.0195062695769088), ('pajak', -0.01140235397100462), ('hapus', 0.009041925463359081), ('manja', 0.008440873991795108), ('dikurangi', 0.007516719477155776)], [('pajakperasrakyat', 0.01859604449581145), ('rakyat', 0.01210713403317779), ('negara', -0.010963298395008795), ('pajak', -0.00948482534085665), ('ciri', 0.004349578923381873), ('bangkrut', 0.0035502536500320353), ('cari', 0.0034144994429313284), ('sumber', 0.003252606436431497), ('uang', -0.0023363061579673356)], [('menteri', -0.017317392267587828), ('pajakperasrakyat', 0.011502994425586105), ('aja', 0.011475941900126593), ('pajak', 0.006364100606120903), ('kalo', 0.005206028486630231), ('ga', -0.004687046172943755), ('kyknya', 0.003367873413183589), ('jago', 0.0031839458412548775), ('kerja', -0.003167498423895852), ('tamat', 0.0019050734819574821), ('bisa', 0.0018199206831417266), ('tetangga', 0.0018060600198697926), ('sd', 0.0017560792147480913), ('ngutang', 0.0016769001233338573), ('naikin', 0.0015043440192337501), ('repot', 0.001361076962329144), ('bu', 0.0012920512280953412), ('bantu', 0.001291552088678524)], [('pajakperasrakyat', 0.020446658952138923), ('rakyat', 0.009419559570828118), ('koq', 0.006552450782764563), ('roket', 0.005553465549217224), ('naikkin', 0.005050160264412982), ('ekonomi', -0.0030629528933200627), ('pajak', -0.0021678170198050216)], [('pajakperasrakyat', 0.0292887140460181), ('jokowi', 0.0077591090393315815), ('pajak', -0.004983825922688317), ('daftar', -0.004282909378010165), ('cepat', 0.004258336347284871), ('bijak', -0.003772265743170785), ('kereta', 0.002873667626231374), ('amnesti', 0.002375141376513382), ('plinplan', 0.0018227028798420137)], [('pajakperasrakyat', 0.041275911334229455), ('pajak', -0.007728897827533448), ('rakyat', 0.007604660383264747), ('didik', -0.0033823252321205245), ('cekik', 0.0012205540246642897)], [('rakyat', 0.01254476826402272), ('ragam', 0.010880868478631366), ('khaldun', 0.009659394746304268), ('ibnu', 0.009066007556431766), ('aneka', 0.008871677996145478), ('hancur', 0.00825870981398143), ('pungut', 0.007452670166886572), ('tanda', 0.00685681045346357), ('negara', -0.006744788328730154), ('pajak', -0.0016705224537393248)], [('pajakperasrakyat', 0.11678686615528842), ('pagi', -0.07596912720981593), ('jenius', -0.0288516487315436), ('pimpin', -0.02091577527937509), ('negara', 0.018012936640702578), ('pajak', 0.013104799586161738), ('coba', 0.00915021849129429), ('kena', 0.009142703734166929), ('cangkir', 0.009095880412625944), ('uda', 0.009090773882228044), ('kopi', 0.00835830957564455), ('hebat', 0.006398397722342193)], [('rakyat', 0.05617726329066227), ('kemplang', 0.046024137630948675), ('pajakperasrakyat', 0.041894098896767006), ('taxamnesty', -0.04155533351773661), ('naik', -0.04022090693529054), ('diam', 0.03416097149496354), ('usaha', 0.020531971482258972), ('pajak', -0.017566523282748995)], [('tingkat', -0.043905771120741656), ('rakyat', 0.027803518006113435), ('pajakperasrakyat', 0.021882053446480247), ('pinangki', 0.0040815415595109195), ('bencana', 0.003277026961507008), ('korup', 0.0032121506480331198), ('gaji', 0.0011962278401603161), ('palak', 0.0006879787408190333), ('kian', 0.00023668928820102977), ('pajak', 7.063358774181565e-05)], [('pajakperasrakyat', 0.034972605054873804), ('pajak', -0.014623445747960158), ('kemplang', 0.009776862536860982), ('ya', 0.0035055636630007226), ('sanksi', 0.0027999006475143153), ('pidana', 0.002727761769648834), ('diha', 0.0023354681293521855), ('ngemplang', 0.0004921986735613494)], [('pajakperasrakyat', 0.0347251900291518), ('rakyat', 0.009193306418128239), ('nama', 0.0058791553566934565), ('pajak', -0.005618138977473486), ('rampok', 0.0054051699817183885), ('bangun', 0.004396244065365504)], [('bayar', -0.045642340452388463), ('rakyat', 0.03585078381363946), ('kaya', 0.028804190378952526), ('foto', 0.02522947589948383), ('gitu', 0.018907205006425343), ('jelata', -0.006231315573347518), ('pajak', -0.005257438453361685)], [('pajakperasrakyat', 0.1222046947683468), ('mudah', -0.10272365378687427), ('pajak', -0.024825969649172774), ('hasil', -0.019212448739687502), ('kerja', -0.018033145702509294), ('ya', 0.013111824031905332), ('naikin', 0.007829131489007764)], [('pajakperasrakyat', 0.032745780270584175), ('kaya', 0.015912929765068164), ('kejar', 0.011145345358068154), ('orang', -0.008053160025526656), ('usaha', -0.0069539804326643785), ('dpotong', 0.005456632549147083), ('nooh', 0.005355806301681337), ('pajak', -0.004683295598769626), ('ama', 0.00448657583063257), ('gaji', -0.0021482060873878966), ('udah', 0.0012995510527104067)], [('pajakperasrakyat', 0.026344035166763015), ('gara', 0.012425539478529016), ('kena', -0.0058703371081328766), ('rakyat', 0.004992927518296734), ('ya', 0.004572643465684845), ('imbas', 0.0037597888551699076), ('dinaikin', 0.0033422216800354583), ('pajak', 0.002000176219813526)], [('pajakperasrakyat', 0.1457822792933887), ('rp', -0.09390613800328972), ('pajak', 0.01973090412129451), ('kena', 0.018387161925852817), ('gaji', 0.0062498835267421), ('atur', 0.0056194222656309935), ('juta', 0.0038428759080922944)], [('pajakperasrakyat', 0.05505078838237749), ('rakyat', 0.04140836325535543), ('yaa', -0.028884245692978223), ('alaikumussalam', 0.015668701165256142), ('ampuunn', 0.0143808892995651), ('astaghfirullohalafziim', 0.013691084884511007), ('wa', 0.011262012672414147), ('negeri', -0.009229296747700629), ('susah', 0.008687948395421112), ('kena', -0.003094987329519071), ('pajak', 0.0010542311184345001)], [('rakyat', 0.08654482898310392), ('bayar', -0.064141378099411), ('pajak', -0.030944591967487608), ('ktp', -0.026731903547302222), ('ya', 0.02512170109050231), ('situ', 0.01272725554696433), ('banget', 0.011339050632802302), ('huat', 0.011022691276182942), ('ngemis', 0.010268018448722138), ('fix', 0.007703512662674714)], [('jenius', -0.20997156385916735), ('pajakperasrakyat', 0.1897443875101688), ('bijak', 0.046306601272616044), ('sumber', 0.03444635054729157)], [('rakyat', 0.05355185700523997), ('pajakperasrakyat', 0.04431891126830654), ('pandemi', -0.03551805034141738), ('bayar', -0.008194635546045624), ('kasih', 0.005860242782786191), ('jepit', 0.005736557308555649), ('hitam', 0.005724404103619815), ('pajak', -0.004564670576759569), ('tengah', 0.003629728424985472), ('amnesti', 0.0032201865064782095), ('ekonomi', 0.0031483715268477057), ('konglo', 0.000836515360081944)], [('rakyat', 0.06562215002868042), ('bebas', -0.06022909259910361), ('pajakperasrakyat', 0.056883506962047796), ('kolonial', 0.02637792275557122), ('zaman', 0.025515911192007724), ('serasa', 0.02336153728345229), ('pajak', 0.01197565192868383)], [('pajak', -0.02798161309725387), ('redzim', 0.02056909413732816), ('tumbang', 0.008523299659132432)], [('indonesia', -0.05263705612142892), ('pajakperasrakyat', 0.05206066755432115), ('kemplang', 0.045814156873286285), ('rakyat', 0.039391809802591944), ('bayar', -0.014755112865760867), ('kejar', 0.009485747180206434), ('keringa', 0.008292809130350883), ('gilir', 0.006136132199729216), ('pajak', -0.003032636121681386)], [('pajakperasrakyat', 0.07081193707751293), ('nilai', 0.040431885769745164), ('blunder', 0.030689644641432198), ('dampak', 0.020408246814078757), ('bhima', 0.01983592411066518), ('turun', 0.014388235675016653), ('taxamnesty', -0.009777906153652264), ('patuh', -0.009189150183088142), ('jilidii', 0.006465302550150838), ('pajak', 0.00046198368230924666)], [('rakyat', 0.050034299230240255), ('mudah', -0.04555669377005349), ('kaya', 0.030205321993647765), ('nelayan', 0.025095623207802033), ('ratus', 0.01572478208894012), ('sanggup', 0.012543983677651221), ('laut', 0.011508716231777053), ('kapal', 0.01081097955809479), ('orang', -0.0036457574845491494)], [('laku', -0.0385433138415655), ('pajakperasrakyat', 0.03469352841744982), ('negeri', -0.010753179043119952), ('juta', -0.009754045243805787), ('aksi', 0.0075371302034539876), ('pionir', 0.0073485000384478074), ('coba', 0.00411146342633644), ('bah', 0.003928990657972113), ('tolak', 0.0034663650756330813), ('petisi', 0.003395545041374259), ('hrs', 0.003096399298506703), ('massa', 0.0029105595587635186), ('nya', 0.0002783514325930704)], [('pajakperasrakyat', 0.02806592944549147), ('kemplang', 0.010397109689483711), ('pajak', -0.005449008560362141), ('hapus', 0.003188353148091877), ('sanksi', 0.0028253894226082715), ('pidana', 0.0024240702835156475)], [('pajakperasrakyat', 0.04029708536975329), ('rakyat', 0.02107510867024328), ('peras', 0.008287333333584111), ('pajak', -0.005183441787083581)], [('pajakperasrakyat', 0.03424978382999763), ('efek', -0.021723975222161795), ('ikan', 0.01135546343176337), ('nelayan', 0.010661148766594237), ('pajak', -0.00893622114151982), ('kena', -0.008819995006673575), ('ppn', -0.006541531108335059), ('kendala', 0.002930307096684051), ('uang', -0.0026528181053217187), ('laut', 0.0023718646335453386), ('mahal', 0.0019823283995452075)], [('pajakperasrakyat', 0.021013993902207765), ('ppn', -0.011718489215237575), ('pajak', 0.005043767786833605), ('timbul', 0.004642966115721032), ('yakin', 0.004542037860860565), ('risiko', 0.004169288022871912), ('pulih', 0.004049802381226115), ('tetap', 0.003988143300945284), ('nilai', -0.0038721106512971744), ('april', 0.003731720483245525), ('ekonomi', -0.0035436337679575266), ('tambah', 0.001009766950827077)], [('kemplang', 0.013731837323842803), ('rakyat', 0.010259150404560206), ('pajak', -0.01020695691244248), ('cina', 0.007030945409354518), ('susah', 0.004379048773006092), ('joo', 0.003399778833644653), ('kau', 0.002826384030849422), ('luhut', 0.0027187491905372413), ('airlangga', 0.002628780545789093), ('kee', 0.0024655040860780113), ('apa', 0.0021102004549438803)], [('pajakperasrakyat', 0.025015321175401378), ('batas', -0.005850151059936596), ('pajak', 0.004492957075218684), ('kmampuan', 0.0038819977125958706), ('nya', 0.002427838488104586), ('mentok', 0.0023332004643845917), ('pikir', 0.002295558680323616), ('hnya', 0.002240462195088883)], [('mudah', -0.05071629249358185), ('rakyat', 0.0501581184982395), ('kaya', 0.033091454982287466), ('nelayan', 0.026423757172920274), ('ratus', 0.019187411874162983), ('kapal', 0.01725693306537599), ('laut', 0.015163302802049495), ('orang', -0.006966431165463218)], [('negara', -0.031688692721185546), ('perintah', -0.02348089554731199), ('khaldun', 0.02159544616584151), ('ibnu', 0.020573105515159503), ('pajak', -0.01861570711575231), ('tanda', 0.01562669401540762), ('hancur', 0.015152060496794895), ('pungut', 0.012386753413835808)], [('pajakperasrakyat', 0.07158679562119057), ('kena', -0.022650115200914363), ('batu', 0.004208656608923838), ('tukang', 0.003694814159810699), ('pajak', -0.0016596055135491572), ('kuli', 0.0016308188245505561)], [('pajakperasrakyat', 0.02383822120782983), ('lapar', 0.008211889698930969), ('krena', 0.007794032114311484), ('rakyat', 0.007568563060645392), ('pajak', -0.00332997537989833)], [('rakyat', 0.008450762496521848), ('hapus', 0.004728824166854665), ('peras', 0.00431387834395979), ('nelayan', 0.003954658639625428), ('bikin', 0.0038511787737362333), ('batu', 0.00350820686274609), ('untung', -0.003283674863660388), ('oligarki', 0.003236866521889187), ('bara', 0.003077712850013312), ('rizalramli', 0.002259309879664043), ('royalti', 0.0020109918852833047), ('persen', 9.890840877410674e-05), ('pajak', 9.246290576889164e-05)], [('pajakperasrakyat', 0.0520060228836648), ('hasil', -0.023944283494708837), ('rakyat', 0.02038652016700823), ('wakanda', 0.00604412826450348), ('pajak', -0.00565729572032813), ('land', 0.0054564241756334415)], [('pajakperasrakyat', 0.034338253060555594), ('negeri', -0.024025365827390472), ('smua', 0.008984755289209409), ('dlihat', 0.008713867506519472), ('mnjadi', 0.008461006320492086), ('ku', 0.008430915894650743), ('pajak', 0.006170110057691106), ('sumber', 0.005852956523162216), ('gila', 0.005756920411184329), ('nya', 0.004614790391951754)], [('pajakperasrakyat', 0.04935894249577792), ('orde', 0.01610651749060938), ('gila', 0.008160257826396906), ('pajak', -0.0029701954273668963)], [('pajakperasrakyat', 0.05417854012767768), ('bayar', -0.026407682679434583), ('motor', 0.02465342887838235), ('butut', 0.022080167307973454), ('rumah', 0.02081474767165619), ('susul', 0.02073288543301236), ('orang', -0.01801337901672747), ('pajak', 0.010178371420662694)], [('kemplang', 0.144510017398486), ('uuhpp', -0.09033736997858378), ('wajib', -0.06273231834069654), ('ruuhpp', -0.0419349588078506), ('pidana', 0.03631554053150432), ('sanksi', 0.02586593730147073), ('hapus', 0.024818336377919845), ('pajakmemerasrakyat', 0.018118428914080813), ('omnibus', 0.016282546409710177), ('laku', 0.014456268971308374), ('lega', 0.014320607695361514), ('ruu', 0.014221481198126886), ('kelas', 0.013703702880059154), ('law', 0.013418581970360869), ('kakap', 0.0133534684738601), ('tentangg', 0.0132739874329577), ('pajak', 0.012733583444277169), ('mulus', 0.012377532501632045), ('kerja', 0.01063569226140462), ('atur', 0.010084533969668389), ('pps', -0.0065781280217350555), ('jalan', -0.0055695983395601684), ('cipta', 0.0053925801475005995), ('program', 0.001889505316727302)], [('pajakperasrakyat', 0.03289925490353494), ('budaya', 0.006825271530464913), ('yahudi', 0.006791349046992033), ('ribawi', 0.006531082046691576), ('pajak', 0.0013219317186240367)], [('pajakperasrakyat', 0.02967683797903973), ('rakyat', 0.02461632900252743), ('pajak', -0.009369613126888757), ('susah', 0.008192893451091416), ('beban', -0.004062687886852628)], [('rakyat', 0.039701867032243224), ('diskon', -0.03584903400233867), ('pajakperasrakyat', 0.02807317104011046), ('hutang', -0.005926691400698496), ('bayar', -0.005641191896355475), ('negara', -0.004857084860410307), ('uang', 0.003743153555342559), ('sengaja', 0.0036131596496747067), ('pajak', 0.0033793589048360397), ('garong', 0.0033424405632773106), ('sih', 0.0030564038020260364), ('vonis', 0.0023531208348393563), ('harus', 0.00218652444648703), ('hukum', -0.0018373418309868236), ('korupsi', 0.0018151372898326586), ('ringan', -0.0014159884625443877), ('sejahtera', 0.0013501809531038028), ('bantu', 0.0013198922331513086)], [('pajakperasrakyat', 0.05322882979982067), ('kena', -0.014015941780536494), ('aja', 0.012287447552052556), ('mak', 0.010691519932978354), ('warung', 0.009411112481542142), ('belanja', 0.00892920146202003), ('gak', -0.0055336991114592286), ('pajak', -0.0006475761163202363)], [('pajakperasrakyat', 0.02603407223631739), ('kembang', -0.024099954949414155), ('aja', 0.012408892708726279), ('beban', -0.01016575198639974), ('sapa', 0.007101190923605983), ('kempis', 0.006076206054365852), ('pajak', -0.005365736797432525), ('udah', 0.0020463280189869527)], [('pajakperasrakyat', 0.06389402626441859), ('pajak', -0.01657617518547765), ('gara', 0.005507205174294563)], [('pajakperasrakyat', 0.013369040407590418), ('taxamnesty', -0.011594932785471771), ('situ', 0.005735939430261579), ('rakyat', 0.0052322346531845105), ('raya', 0.004834827929022478), ('horang', 0.004830837403739352), ('kek', 0.00472154475457271), ('oligarki', 0.004543105346629098), ('waras', 0.004318900272509472), ('dinaikin', 0.00429624432997001), ('bong', 0.00400867617822142), ('nelayan', 0.003617789785427232), ('kaya', 0.003215946692639963), ('pajak', -0.0007432300200507846)], [('hutang', -0.03965747642413228), ('kuasa', 0.033125203824503985), ('rezim', 0.014726457742159879), ('banyakin', 0.013407376427127615), ('cokin', 0.013355652594001158), ('hancur', 0.007740536105874698), ('gede', 0.007454245113593416), ('negara', -0.006442629903835686)], [('pajakperasrakyat', 0.03578828642073116), ('rakyat', 0.011194117468303084), ('pajak', -0.007404593912159803), ('mati', 0.004576430291341117), ('cekik', 0.003981460653551412)], [('masyarakat', -0.12319752514280899), ('serakah', 0.06979937155240862), ('jabat', 0.05860755301334026), ('korban', 0.0540544593735812), ('akibat', 0.04715830172352923), ('pajak', 0.02576062850434621), ('bayar', -0.023248250051156178)], [('pajakperasrakyat', 0.14285719616728967), ('bayar', -0.08825918604630238), ('pajak', -0.03684243614709303), ('hidup', 0.014987041379293752)], [('pajakperasrakyat', 0.06322789000230634), ('rakyat', 0.023446113764480057), ('negeri', -0.0212547892568442), ('kena', -0.00989193254388435), ('bangkrut', 0.0038302748453968344), ('ngeribanget', 0.0034622600294665225), ('pajak', -0.0015852558084937258)], [('rakyat', 0.14323583883407812), ('kaya', 0.04996794048615553), ('pagi', -0.04049863694306588), ('peras', 0.038909272958758304), ('bebas', -0.03300224061152384), ('pajak', 0.02986106397187981), ('nya', 0.023644983026580024), ('kabar', 0.023219665102662525), ('selamat', -0.021342205940165653), ('negeri', -0.018712863418817886), ('orang', 0.0012436776065019675)], [('pph', -0.055849479387527566), ('pajakperasrakyat', 0.041070562018159555), ('kemplang', 0.027518189753158794), ('buwat', 0.007582987828956465), ('apah', 0.004236879767743178), ('modalin', 0.003652976323070877), ('pajak', 0.001402776861947207)], [('pajakperasrakyat', 0.09295930463251956), ('kena', -0.015925487861447866), ('pajak', -0.008087372617750865), ('tagar', 0.005671532374009389)], [('laku', -0.03817043279133666), ('rakyat', 0.018850692117665404), ('pajakperasrakyat', 0.016065801567757023), ('pajak', 0.0052484846396889375), ('beban', -0.003231243607486317), ('konstitusi', 0.0016386948628075956), ('hidup', 0.0006285736174068865), ('pasal', 0.00040092726371581495), ('tuduh', 0.00037567274729563706), ('jamin', 0.00032338482258874014), ('uud', 0.0002481815700977327), ('langgar', 0.00023832114149212533), ('jahat', 0.00010725122725023564), ('miskin', 8.561120968162768e-05), ('duga', 3.7641950261764216e-05), ('patut', -5.196548143353474e-06)], [('pajakperasrakyat', 0.15802579908991704), ('bayar', -0.05528824472507579), ('ayo', -0.01286056413395254), ('pajak', -0.010400775011248779), ('bangun', 0.005560918394548396)], [('ri', -0.2203129629643248), ('rakyat', 0.1131244084236988), ('laksana', -0.0887629756812594), ('kanebo', -0.07294199807906444), ('or', -0.061968271996093005), ('kemenkeu', -0.053179808158408344), ('nik', -0.05137573654066609), ('orang', -0.042291473240312204), ('ugalan', -0.038960064700167156), ('tarif', -0.03505800151772978), ('sandwich', -0.032712993137143355), ('bilang', -0.03255895934730993), ('npwp', -0.02761757357483077), ('hutang', -0.027082318540052927), ('ktp', -0.01796152697086996), ('beban', -0.006384850308827987), ('pajak', -0.0038808536317602726), ('ppn', -0.002995701781402906), ('rezim', 0.0022884230830235853), ('bayar', 0.0009581312756803507), ('milik', -0.0004759468855568044)], [('biar', -0.05613457398354643), ('pimpin', -0.021893018891733076), ('bohong', 0.019234961885690693), ('bohongin', 0.018187736059594038), ('siksa', 0.01818267086689307), ('malaikat', 0.016506991590715815), ('pajak', -0.013130852307681016), ('dukung', 0.008401897314200348)], [('pajakperasrakyat', 0.0802237301047862), ('bijak', 0.053753163445063326), ('taat', -0.03510624622302953), ('ambyar', 0.025936942754395943), ('pajak', 0.010079514233537173), ('orang', -0.009334027216031208)], [('doang', -0.04457040593609713), ('kemplang', 0.02623124793222502), ('pajakperasrakyat', 0.0251438624397154), ('ampun', 0.010579829110899023), ('jokowi', 0.008692024223362606), ('pajak', -0.004164208288859246), ('eksekusi', 0.001796234115852011), ('omong', 0.001628036741902957), ('mazzz', 0.001495734945728809)], [('pajakperasrakyat', 0.05756950707403091), ('harga', -0.026200478103640203), ('atur', -0.021940185115782558), ('mahal', 0.0073406780731239294), ('kontra', 0.004385412591332888), ('pro', 0.003631337580250562), ('pajak', 0.001986410689337826), ('barang', 0.0009551654327795689)], [('nelayan', 0.029593016732289614), ('ppn', -0.019564740792602255), ('tani', 0.012859725945903141), ('kondisi', 0.012669821311984627), ('sembako', 0.00904559310841357), ('nilai', 0.008657577325812928), ('sulit', -0.004745864186797818), ('rencana', 0.004120880878990751), ('beban', 0.0004890804672056069), ('sengsara', 0.00028912280573722797)], [('wajib', -0.11496712145795457), ('rakyat', 0.1011274894528991), ('bener', 0.035583386523586925), ('males', 0.032289607090631944), ('korban', 0.02416594078325868), ('orang', 0.011988269572663824), ('pajak', 0.005130480431257527), ('bayar', 0.005018092374314234)], [('pajakperasrakyat', 0.08913141175618641), ('rakyat', 0.055366961874042976), ('bayar', -0.04148212589273502), ('ktp', -0.02150581205200963), ('ya', 0.018404247094334376), ('hutang', -0.018363707042551386), ('fix', 0.013744213020883795), ('situ', 0.012209580640596402), ('emis', 0.01167111680840682), ('pajak', -0.007307098241491604)], [('pajakperasrakyat', 0.034734098680608824), ('pajak', -0.011992953881993711), ('aja', 0.008162853192998037), ('makan', 0.006201456544029056), ('pantes', 0.0033744831329564177), ('gaya', 0.0033249042842628724), ('kenyang', 0.0032307683034295946), ('abnormal', 0.0027540897694031245), ('hidup', 0.002694091409365593)], [('sistem', -0.12270653286107121), ('potensi', -0.11672363487661817), ('utama', -0.039433187788686326), ('rakyat', 0.036090764201025256), ('pajakperasrakyat', 0.02928511408635439), ('pajak', -0.02247016316725963), ('negara', -0.017530489006678345), ('rezim', 0.0054496970705122825), ('sumber', -0.005174181335021044), ('budak', -0.004127545095256828), ('daya', -0.004078553825951211), ('tambah', -0.003154782647780117), ('kas', 0.0025158053503806783), ('gagal', 0.0018503007844088402), ('instrumen', -0.001112106945645571), ('upeti', -0.0009567346649536679), ('pasu', -0.0008508801167556075), ('kelola', 0.0008424608925274232), ('alam', 0.0003877049329043596)], [('bebas', -0.0649084614837249), ('pajakperasrakyat', 0.035617631420921544), ('nelayan', 0.02362055077579701), ('tega', 0.019463992639216957), ('banget', 0.01922326795483924), ('ya', 0.017637222760845395), ('rizalramli', 0.01557489056541062), ('pajak', 0.015139678414344862), ('gede', 0.013313294533483648), ('persen', 0.012797499253807972), ('usaha', 0.005518895997807275)], [('rakyat', 0.08423608415023671), ('pajakperasrakyat', 0.07041563148742756), ('jabat', 0.0223826668480283), ('biar', -0.022025368334824838), ('suaka', 0.014857998521881353), ('liar', 0.014368241820909466), ('bebas', -0.013801143779997662), ('ngemplang', 0.013688234038649136), ('suruh', 0.012462042482103346), ('bayar', -0.007961146133874174), ('pajak', 0.003293693220035441), ('simpan', -0.0028733221974655385), ('negara', -0.0012252200541451846), ('harta', -0.0009773450386707037)], [('pajakperasrakyat', 0.01572885869179285), ('kemplang', 0.008995704093359657), ('rakyat', 0.0064893727789499325), ('pajak', -0.006369423050691892), ('jabat', 0.0039444880362732825), ('pandora', 0.0023177945360139374), ('paper', 0.0008840419631692312), ('nyaman', 0.0007069969432638416), ('tenang', 0.00030872793043757346)], [('pajakperasrakyat', 0.034189721971055145), ('pajak', -0.011270060941911337), ('aja', 0.007323015800899157), ('makan', 0.005840358057093113), ('pantes', 0.0038619617295642286), ('kenyang', 0.0029154787421692093), ('hidup', 0.0028698505962380157), ('gaya', 0.0024004710805762626), ('abnormal', 0.0022619520760184973)], [('rakyat', 0.029086455647647145), ('kaya', 0.019658404009833028), ('orang', -0.0142583784182038), ('ampun', 0.01158703792973853), ('jabat', 0.009694388366954021), ('derita', 0.007838702678776245), ('bismillah', 0.0075047593119193425), ('pajak', -0.00658870636283902), ('korupsi', 0.0025093501822529593), ('beban', -0.0021951852742787114)], [('rakyat', 0.06259175024940702), ('pajakperasrakyat', 0.035596333635019774), ('bebas', -0.03524938714024888), ('kaya', 0.027444267544962965), ('agam', 0.012997731809346165), ('mobil', 0.01035806885502711), ('cekik', 0.009588502480301587), ('beli', 0.00915446030397394), ('orang', -0.006794424796639577), ('sulit', -0.005427368957334356), ('mewah', 0.005169250508241749), ('pajak', -0.00014158294474913297)], [('rakyat', 0.10020482944656552), ('jenius', -0.06342316027662644), ('kaya', 0.04612986843579461), ('bebas', -0.04018462813291211), ('mobil', 0.03461370201539137), ('coba', 0.029864218916765908), ('dipajakin', 0.0282229347743083), ('beli', 0.028179675261408504), ('pajak', 0.025443584394652042), ('sembako', 0.02372164583372272), ('mewah', 0.02211756727424835), ('orang', -0.010223747476290184)], [('pajakperasrakyat', 0.05369818539704164), ('kena', -0.01586861683520612), ('aja', 0.009931914704676845), ('makan', 0.005320144977412827), ('keluarga', 0.0036588392596023692), ('pake', 0.003044275460998783), ('pajak', -0.001100917544544814)], [('ruuhpp', -0.08931708742760196), ('pajakperasrakyat', 0.06211864983659601), ('kemplang', 0.04177866477300061), ('tajam', 0.02035120165074426), ('kesah', 0.018049676756995883), ('sanksi', 0.01500671128679494), ('kritik', 0.013596336578574997), ('pajak', 0.013247817990547544), ('hilang', 0.012395335971861161)], [('pajakperasrakyat', 0.04990633927073449), ('tunggak', 0.022160472680362312), ('kena', -0.010917339915289254), ('pera', 0.009828344476366747), ('pajak', 0.0045076188158579035), ('denda', 0.0044722786948960865)], [('ruuhpp', -0.1040537284198925), ('pajakperasrakyat', 0.07068981014831767), ('kemplang', 0.04954645903625022), ('said', 0.022754768664343367), ('sanksi', 0.01748191133026124), ('pidana', 0.01602679284638874), ('hilang', 0.013295145936439445), ('pajak', 0.011851477508222684)], [('ruuhpp', -0.06083819544478108), ('pajakperasrakyat', 0.03982772113119628), ('kemplang', 0.031547705883277107), ('tajam', 0.013554482309759456), ('kesah', 0.01281160610836114), ('pajak', 0.012296545998152799), ('kritik', 0.010828244731924293), ('said', 0.010226058920345333), ('pidana', 0.010177855324524372), ('didu', 0.009238079305338577), ('sanksi', 0.00897239270176502), ('hilang', 0.008363604292085955)], [('pajakperasrakyat', 0.07214749956844613), ('salah', -0.03345867635899296), ('pajak', -0.013822918946032423), ('negara', -0.01261087827531527), ('tanda', 0.012573763233406973), ('digedein', 0.012366986502830236), ('bangkrut', 0.008983754619531934)], [('pajakperasrakyat', 0.03187143753337068), ('dzaman', 0.005309389353450791), ('cekik', 0.005300411103876166), ('upeti', 0.004521104414418505), ('jajah', 0.003944047171508895), ('pajak', 0.0001768543726497564)], [('jokowi', 0.03246750850738101), ('cepat', 0.023950621843275517), ('ubah', -0.0228494600879017), ('dana', -0.019277349554134408), ('kereta', 0.018902104688721593), ('jilid', -0.013288209944759334), ('apbn', 0.012243343874534497), ('ampun', 0.009620245111242206), ('pajak', -0.005643703070252536), ('taxamnesty', 0.003317038408185324), ('proyek', 0.002724891104740127), ('jakarta', 0.002583732251291434), ('inkonsistensi', 0.0022290393772093553), ('ditaxamnesty', 0.0015617952116330812), ('telan', 0.001233213290391538), ('dekat', 0.001171773292022091), ('bandung', 0.0011359526962330475), ('bijak', 0.0011061386463790859), ('biaya', 0.0010492712684631001), ('jilidii', -0.00035787736823900276)], [('interaksiumat', 0.015299109529459277), ('hapus', 0.010855792741861213), ('negara', -0.010850506883692854), ('eh', 0.010649017488400787), ('rugi', 0.010536512173100136), ('nelayan', 0.009080213953884768), ('royalti', 0.008529701640347475), ('bara', 0.008136766902744662), ('batu', 0.008011089356015418), ('pajak', -0.0055104985307514705), ('t', -1.3412502618237498e-05)], [('pajakperasrakyat', 0.02652724330782048), ('sengsara', -0.0069988674284015475), ('tuk', 0.0064635625309356634), ('rakyat', 0.005367565717643539), ('sejahtera', 0.0025381030718212684), ('bismillah', 0.0020039106948599892), ('pajak', 0.0002561866774542634)], [('bebas', -0.04459133560291817), ('pajakperasrakyat', 0.03476360963572663), ('nelayan', 0.020250337212998962), ('gass', 0.015261706803885921), ('banget', 0.014799262286058958), ('pajak', 0.013473724048127486), ('tega', 0.013389871469316892), ('ya', 0.013026054681243479), ('rizalramli', 0.011271642149200409), ('gede', 0.009641863925397294), ('persen', 0.007020440262400508), ('usaha', 0.0014261672632186826)], [('pajakperasrakyat', 0.049357725761751715), ('nelayan', 0.012164949584141698), ('pajak', -0.009404068124421123), ('rizalramli', 0.006882581997308173), ('persen', 0.005332548073847023)], [('pajakperasrakyat', 0.05174342052116117), ('kena', -0.016667025266723666), ('pulak', 0.008206497883846158), ('ntar', 0.0075122703672775735), ('paraah', 0.007029492921157066), ('kalo', 0.005400245307904569), ('lahir', 0.005232372708373085), ('pajak', 0.002245908583159173)], [('rp', -0.0767342795424956), ('pajakperasrakyat', 0.06276844562867001), ('kritik', 0.02189681242110441), ('hapus', 0.020160315855491766), ('bara', 0.01845164102056277), ('triliun', -0.01822675063195426), ('batu', 0.01603028850886846), ('rugi', 0.01572577019023103), ('kait', 0.01455841199695158), ('negara', 0.014422566104684202), ('lambung', 0.013991161153893275), ('royalti', 0.010835052011787798), ('perintah', 0.010065035497667534), ('rizalramli', 0.006596820648302478)], [('rakyat', 0.049930536861129686), ('pajakperasrakyat', 0.0488936365557193), ('kemplang', 0.04731366741438437), ('luas', -0.0443451618143966), ('diskon', -0.016732101890672507), ('sorot', 0.005573953576387772), ('pajak', -0.005404895081053216), ('uu', 0.004474912794984607), ('didu', 0.0013273297256963616), ('said', -0.0011685706956874756), ('hmmm', -0.0004620632852943371)], [('wajib', -0.08795933465962673), ('rakyat', 0.08202243434502539), ('beban', 0.015395560202991446), ('ngumpetin', -0.015065068157038801), ('nya', 0.013147227095170928), ('nikmat', -0.009989171583052119), ('sembako', 0.009152410446953948), ('suaka', 0.008925091934878338), ('taipan', 0.008889050442474173), ('ringan', 0.008576263716272738), ('zalim', 0.008425854937089198), ('garong', 0.0077093469710926565), ('biar', -0.007435895722053269), ('fasilitas', 0.0073464416807177075), ('negara', 0.0071021436590737825), ('negri', 0.006038957453918056), ('pajak', 0.005777684723978397), ('konglomerat', 0.005694394507994973), ('penuh', 0.005490197877782907), ('dipajakin', 0.00480885847084515), ('al', 0.004488134835739201)], [('pajakperasrakyat', 0.0491325848661567), ('salah', -0.049099437707261065), ('rakyat', 0.03255705284381594), ('bayar', -0.011883836041373472), ('ummat', 0.004429959906173065), ('krisis', 0.004175230640715654), ('ciri', 0.003824503152461245), ('negera', 0.003008297023811948), ('pajak', 0.002942304429700478), ('paksa', 0.0026509243091738465), ('bangun', -0.000637850462579992)], [('pajakperasrakyat', 0.06479330916610922), ('nilai', 0.03478163032729668), ('blunder', 0.02190374768553206), ('patuh', -0.019329645561904422), ('duh', 0.014364026273422944), ('dampak', 0.01120866305398155), ('bhima', 0.01107119941023075), ('turun', 0.004120529562194738), ('taxamnesty', -0.0008614098133731186), ('jilidii', 0.0007683875880492431), ('pajak', 0.0006140367009430134)], [('ri', -0.2929062737605078), ('naik', -0.13053217715120727), ('bilang', -0.10731716228869481), ('menteri', -0.07227723415706802), ('orang', -0.06143732834443134), ('pajakperasrakyat', -0.04618167025597878), ('uang', -0.039999719822101806), ('pajak', -0.039121027949045095), ('beban', -0.03499803863560607)], [('pajakperasrakyat', 0.12178453812850801), ('paja', -0.08116766643606618), ('resmi', 0.024580921564559016), ('sah', 0.02251079893630821), ('sanksi', 0.018125628288270097), ('bayar', -0.01024416026817447), ('turun', 0.008943033402089918), ('uu', 0.007445896874235522), ('ppn', -0.0037036784020675908), ('pajak', 0.0017392517394333616)], [('hindar', -0.04940467091868146), ('pajakperasrakyat', 0.03938161981251834), ('biar', -0.030467046385483042), ('rakyat', 0.02856977561363177), ('kaya', 0.025139972782708065), ('pajak', -0.0055389232885674974), ('dipajakin', 0.004415964089294593), ('gemes', 0.0024208011350523887), ('rakyattuh', 0.0012414684158365023)], [('untung', -0.0049981419753035525), ('pajakperasrakyat', 0.004755706434339411), ('rakyat', 0.004084826354701659), ('hapus', 0.0038409650333426078), ('nelayan', 0.003783512959529888), ('oligarki', 0.0022523544510608244), ('bara', 0.0022291943975034875), ('peras', 0.0019193141490704776), ('batu', 0.0016475134776409342), ('bikin', 0.001584248919414276), ('pajak', 0.0010635026987048496), ('susah', 0.0008924995039881767), ('royalti', 0.0007831224041478914), ('rizalramli', 0.0005900322968958511), ('persen', -0.00026160913024020994)], [('pajakperasrakyat', 0.015953346877062454), ('aja', 0.0058327653429015094), ('hmm', 0.0030959122952319705), ('nelayan', 0.0027534174544812844), ('juwana', 0.0026900364943107584), ('ngeluh', 0.0026147710267383664), ('pati', 0.0022191091836706923), ('pajak', -0.0005299921988345689)], [('pajakperasrakyat', 0.08070027987670153), ('rakyat', 0.05099362399634548), ('bayar', -0.04052151789768394), ('kejar', 0.01591486568534223), ('pajak', -0.008190045610521555)], [('pajakperasrakyat', 0.09486938058660173), ('wajib', -0.06649101603923391), ('rakyat', 0.051591431604212014), ('kaya', 0.039006985880508334), ('kabar', 0.012877078679493793), ('pajak', 0.00915047801745472), ('bayar', -0.005008963062996118), ('sembunyi', 0.0047287665275490155), ('orang', -0.004272173435787645)], [('pajakperasrakyat', 0.029280748152460578), ('rakyat', 0.008120116051395726), ('republik', 0.007094097736834185), ('pajak', -0.006568713982172667), ('bijak', -0.005328141905373013), ('investor', 0.004045707788560689)], [('pajakperasrakyat', 0.07196433347913445), ('rakyat', 0.033373982881821376), ('bayar', -0.03284529306590813), ('utang', 0.01331646193861436), ('hasyuk', 0.01248585982529122), ('pajak', -0.008630384810900036)], [('pajakperasrakyat', 0.025144783871906076), ('pajak', -0.010014851475271197), ('rakyat', 0.00854411235664672), ('kejar', 0.007721571119605824), ('target', -0.002960936446136883), ('gede', 0.0019735455706535116), ('gak', -0.0017676553935256473), ('berani', 0.0012607899310744166), ('serius', 0.0006896199019682848), ('receh', -0.00026798007438123627)], [('pajakperasrakyat', 0.06453327640839102), ('taxamnesty', -0.05742532965046217), ('happy', 0.052140640506795156), ('kemplang', 0.046283777582409716), ('ekonom', 0.03644637805771925), ('bikin', 0.0313172260707804), ('pajak', -0.010968893230352527)], [('pajakperasrakyat', 0.2027367792517614), ('uuhpp', -0.1357159083043391), ('wajib', -0.10168576634534503), ('taxamnesty', -0.0424083074039443), ('kritik', 0.030863913400485585), ('jarak', 0.02310889700492962), ('sempat', 0.022191249192452158), ('konsisten', 0.020783698177055426), ('satu', 0.0201192135295628), ('balut', 0.01950735225378739), ('anggap', 0.01941486398456779), ('lampau', 0.017266476678706167), ('perintah', 0.012409775359859085), ('program', 0.010068534455092044), ('jilidii', 0.005212098315336885), ('pajak', -0.0012860796691043738), ('jilid', 0.0004932414470939153)], [('pajakperasrakyat', 0.06458418645224735), ('rakyat', 0.04448077376773117), ('bayar', -0.03740201632243867), ('kemplang', 0.027019603169174698), ('pajak', -0.004521518301453545)], [('pajakperasrakyat', 0.1266078715708443), ('paja', -0.08108098018486393), ('resmi', 0.025676189257594705), ('sah', 0.023065466762497044), ('sanksi', 0.017526580185021675), ('bayar', -0.010687488457623077), ('turun', 0.010227558234708575), ('uu', 0.010164355098523422), ('pajak', 0.004119048258541543), ('ppn', -0.002602367565073489)], [('tingkat', -0.04631165268907036), ('justifikasi', 0.025026347308804077), ('kreatif', 0.02030835797801395), ('konyol', 0.017525298561100133), ('perintah', -0.009460977708694071), ('pajak', -0.0001239991605829626)], [('pajakperasrakyat', 0.14768954432912212), ('wajib', -0.11319074129083674), ('perintah', -0.014557791645168513), ('tunggu', 0.01365441795551548), ('pasif', 0.01305424236216283), ('pajak', 0.009721427737662595), ('lapor', 0.008620171979120777), ('dekat', 0.008333045327334134), ('bayar', 0.007543486250868403), ('ambil', 0.005608322252508446)], [('rakyat', 0.08256693749000087), ('pajak', -0.02887208511338107), ('nikmat', -0.024606099917084616), ('hasil', -0.023887413718401126), ('beban', 0.008320630544799201), ('koruptor', -0.001371390085554244)], [('pajakperasrakyat', 0.06760555273085542), ('pajak', -0.03260889966961592), ('sejahtera', -0.009582102256645282)], [('tinggal', -0.03618081048176459), ('pajak', 0.011133699503586943), ('alas', -0.007486942389086271), ('nya', 0.006909053237245197), ('legacy', 0.005209846532553956), ('betlipat', 0.004971432107466121), ('beri', 0.004850981756978341), ('lipat', 0.0045521456602448185), ('taipan', 0.004509403168232568), ('jabatanmenyengserakan', 0.004393279761934476), ('paksa', 0.004379478884273465), ('historycal', 0.003974105419099004), ('senang', 0.0032124981913878416), ('rakyat', -0.0018336893554464478), ('bangun', -0.0011315356454495895), ('hutang', 0.00010091662349193962)], [('pajakperasrakyat', 0.021648492918695125), ('nelayan', 0.00831071210343452), ('untung', -0.005730101072776119), ('hapus', 0.005515360224752081), ('oligarki', 0.004180337424149283), ('bara', 0.004101625026584123), ('royalti', 0.0025506047587818027), ('rizalramli', 0.002197139788662493), ('pajak', -0.0015941509000079763), ('persen', -0.0010160028174799247)], [('rakyat', 0.05133284910685271), ('harga', -0.020993120140140084), ('atur', -0.017051778719603457), ('kontra', 0.011517115665359408), ('mahal', 0.011405058945580892), ('pro', 0.007889040380451965), ('barang', 0.002077765795649238), ('pajak', -5.1634031351444126e-05)], [('pajakperasrakyat', 0.1961768419669208), ('wajib', -0.06989138752085902), ('nik', -0.05005443918786859), ('npwp', -0.032258402524148876), ('milik', 0.023791612478465163), ('pajak', 0.0061430630250531365), ('ditjen', 0.0008002240568663754)], [('pajakperasrakyat', 0.0905857006241157), ('ppn', -0.02720430601135856), ('dampak', -0.010305909435284139), ('undang', 0.006523724453973592), ('pajak', -0.00419440466303759)], [('untung', -0.0054863501872568325), ('pajakperasrakyat', 0.0035908227923950515), ('nelayan', 0.003328657160225382), ('hapus', 0.003085292286938099), ('oligarki', 0.0024663286396210604), ('rakyat', 0.0018701436634388968), ('bara', 0.001786842879192072), ('peras', 0.0013411665810457826), ('pajak', 0.0012616521805194664), ('batu', 0.001202299578565359), ('bikin', 0.0011435870999134793), ('susah', 0.0011001082902583408), ('ya', 0.0006616175887571844), ('royalti', 0.00046444707292247925), ('ko', 0.00026056253595652785), ('tega', 0.00016760865554368167), ('rr', 7.314478518804465e-05)], [('bebas', -0.1515888039864456), ('rakyat', 0.13648064038210445), ('cekik', 0.03158796568747651), ('taipan', 0.027247894274249687), ('pajak', -0.02298743631152031)], [('rakyat', 0.051194330609107996), ('harga', -0.020208864125929957), ('atur', -0.017089569225912), ('kontra', 0.011637280887886218), ('mahal', 0.010456230148231499), ('pro', 0.009749949610316092), ('barang', 0.0017389271623886612), ('pajak', -0.00036148035457900826)], [('sepakat', 0.02833848675820235), ('ppn', -0.027031440225872536), ('pks', 0.02479486745146062), ('nilai', 0.022161952482527898), ('tarif', -0.016720363132975726), ('persen', 0.016508468457347027), ('pajak', 0.015942765038814706), ('rencana', 0.011770801365604173)], [('indonesia', -0.057377728911143365), ('pajakperasrakyat', 0.04668987255503904), ('manusia', 0.03726018723632675), ('yaa', -0.03602664173632618), ('tukang', 0.010271622610346962), ('musnah', 0.008542061625892395), ('tikus', 0.0078090443905867515), ('allah', 0.007760026399037499), ('tolong', 0.007274713274200748), ('das', 0.0069235637971916016), ('korup', 0.006499630580353517), ('pajak', 0.0014926896115306065)], [('pajakperasrakyat', 0.11744097111155352), ('rp', -0.10190411366714051), ('hasil', -0.03467249455959685), ('juta', -0.013250465439540374), ('pajak', 0.013024074820501415), ('nya', 0.011739669459034456), ('kena', 0.009636727853134295), ('ribu', 0.005616504463194476), ('potong', 0.0055473816066320385), ('pegawai', 0.00210629716289799), ('uu', 0.0010164594174456362)], [('pajakperasrakyat', 0.0346276084510079), ('bayar', -0.022081729546047122), ('legislator', 0.011878235491417203), ('ogah', 0.011356454405296578), ('bukti', 0.009602568245651229), ('jabat', 0.009261226170242848), ('pks', 0.008638237638889943), ('pandora', 0.008181327922148868), ('papers', 0.008146745987171265), ('pajak', 0.0028636959166666877)], [('pajakperasrakyat', 0.018223414796622282), ('nelayan', 0.006537436440594543), ('untung', -0.004867625301076242), ('hapus', 0.004867106276180661), ('oligarki', 0.004057717862768655), ('batu', 0.002868323002488808), ('bara', 0.0026168824911656685), ('royalti', 0.0022903212301763915), ('rizalramli', 0.0014783744695280804), ('persen', -0.0011271805134796511), ('pajak', -0.00048268760494655393)], [('justifikasi', 0.03260589235387053), ('kreatif', 0.030221735287252218), ('konyol', 0.027521894738561765), ('tingkat', -0.025399442787127642), ('pajak', -0.01549217080552984), ('negara', -0.011080230881530638), ('perintah', -0.007368848307970791)], [('buru', 0.03592683811225173), ('pajakperasrakyat', 0.028889980084397252), ('taxamnesty', -0.022156342614173527), ('kemplang', 0.02032896226120197), ('ada', 0.015030342836142133), ('receh', 0.013836391450029475), ('ampun', 0.013758800180145869), ('pajak', -0.0009310436683392379)], [('pajakperasrakyat', 0.05788331941197626), ('mahal', 0.010328268518084854), ('jasa', -0.008435292723253862), ('barang', 0.0057367250019536775), ('pajak', -0.002158282034128167), ('daftar', -0.0019698419221577984)], [('bebas', -0.044985055736740304), ('ya', 0.016644554493432516), ('banget', 0.015855647589020135), ('pajak', 0.014079652572630913), ('royalti', 0.013592134445055387), ('tax', 0.013383314029108811), ('holiday', 0.013172510991783026), ('tega', 0.013156079511378323), ('hapus', 0.012723533854904184), ('rugi', 0.012305104743016129), ('bara', 0.011899511561985186), ('gede', 0.011798067030413821), ('batu', 0.010816052850255612), ('negara', 0.008941700214674475)], [('pajakperasrakyat', 0.07388128961622484), ('mutar', 0.045762689363165576), ('bijak', 0.018605860300716297), ('pajak', 0.01721979083894), ('pintar', 0.014040245437842474), ('bayar', -0.0104428906219895), ('orang', -0.009578058014396993), ('taat', -0.00951829377842635)], [('pajakperasrakyat', 0.03437844827471473), ('bijak', -0.0030654506665970696), ('miskin', 0.0009692274148954494), ('kaya', 0.0002811506452669303), ('kurus', -8.844221719743139e-05)], [('nilai', 0.02956319787299371), ('jilidii', -0.025566604315430334), ('blunder', 0.017341982734604553), ('hmm', 0.01653597725091212), ('simak', 0.0155910285955621), ('bhima', 0.014592595347904863), ('dampak', 0.010806604352996409), ('taxamnesty', -0.006140218074183615), ('turun', -0.0019449736201740738)], [('ampun', 0.014906830982362933), ('nilai', 0.005406683167363714), ('pajak', 0.005327511328519346), ('yudhistira', 0.004410942269963519), ('celios', 0.004300610653215535), ('kumat', 0.003062499040494417), ('bhima', 0.002982354902586766), ('ngempalang', 0.0029402629172295305), ('ekonomi', 0.0013869220322311993), ('direktur', -0.0011428590948114545)], [('pajakperasrakyat', 0.013429305868448118), ('rakyat', 0.006065968934176375), ('cukong', 0.005358281935028695), ('kejar', 0.00320720005139152), ('peras', 0.002565591430232951), ('ngemplang', 0.0022538967949654706), ('manja', 0.002147907064592491), ('pajak', -0.0006494928228732976)], [('pajakperasrakyat', 0.06003589881630882), ('indonesia', -0.05305919462179802), ('rakyat', 0.04637929252030244), ('kemplang', 0.042846370906351114), ('bayar', -0.0196581805226811), ('kejar', 0.011851998201281192), ('gilir', 0.005867696600177619), ('pajak', -0.003664969879992959)], [('pajakperasrakyat', 0.02269022520706027), ('kemplang', 0.016424738613132567), ('bhima', 0.009813872418876619), ('nilai', 0.00891555783001293), ('ampun', 0.008855949560707252), ('taxamnesty', -0.007226347572336626), ('yudhistira', 0.006537272460408532), ('pajak', -0.004462874501480474), ('efektif', -0.004237872992749042), ('celios', 0.0036673898765571243), ('amat', 0.003589277464897664), ('jilidii', -0.003277045769356735), ('bahagia', 0.002434877307437418), ('sebut', 0.001810593659037959), ('ekonomi', -0.0016131507022305285), ('direktur', 0.000605716980755924)], [('pajakperasrakyat', 0.014887345408079476), ('atur', -0.010895789627511228), ('rakyat', 0.010282478387004476), ('kemplang', 0.008936384592800365), ('peras', 0.004360772876340442), ('keren', 0.00427676736691315), ('oenjajat', 0.0039280292183307315), ('pajak', 0.002276873780975915), ('manja', 0.0011278734713254517)], [('ragam', 0.018641606932987415), ('aneka', 0.017934568289709595), ('pajak', -0.01593801737150556), ('rakyat', 0.011652156291545375), ('pungut', 0.0083045308309449), ('hancur', 0.0034565387493586336), ('negara', 0.003185501044026314), ('tanda', 0.0021614749270451004)], [('pajakperasrakyat', 0.143929183912504), ('rp', -0.08745028387012704), ('kena', 0.01850407902116261), ('pajak', 0.014625704804121089), ('gaji', 0.008675317264655358), ('duh', 0.006580270866218855), ('juta', 0.005690748441697096), ('atur', 0.005588087229015968), ('akyat', 0.0037263363386227546), ('perasr', 0.002509933118556017)], [('pajakperasrakyat', 0.029126866981864043), ('jajan', 0.00638514685552006), ('mahal', 0.0058515694235946915), ('makan', 0.0057713067444564595), ('baju', 0.005270312939578627), ('pajak', -0.002312521542966393)], [('karbon', -0.20182815706620122), ('pajakperasrakyat', 0.14271189837131454), ('pltu', 0.02757402215197372), ('listrik', 0.020597218118392685), ('biaya', 0.008254580932350826), ('kena', -0.006660174330433256), ('pajak', -0.0033426410818657845)], [('bebas', -0.06589504750760267), ('pajakperasrakyat', 0.06445909924601244), ('perintah', -0.020235613875195173), ('sikap', 0.0146382344322958), ('rizalramli', 0.011889407726187679), ('bijak', -0.011440854849121282), ('alias', 0.010479024443671192), ('holiday', 0.01008120964733556), ('tax', 0.009948261578390502), ('pajak', 0.009902127802701279), ('usaha', 0.007402390591011291)], [('rakyat', 0.07819578385130029), ('kemplang', 0.062996939737965), ('luas', -0.061662265173992335), ('sorot', 0.023104140814717063), ('diskon', -0.021721741819608165), ('pajak', -0.011891268306590434), ('uu', 0.008615484654672639), ('didu', 0.001037733867728113), ('said', 0.0001319930618926162)], [('pajakperasrakyat', 0.07404924608180107), ('nilai', 0.03807595003366514), ('blunder', 0.027876792984797807), ('dampak', 0.018621527849744864), ('bhima', 0.017018190295309498), ('turun', 0.011760873340876983), ('patuh', -0.009977202237805547), ('taxamnesty', -0.005885198769084902), ('jilidii', 0.00560315651291062), ('pajak', -0.00010184240840872049)], [('pajakperasrakyat', 0.05789190742520424), ('mahal', 0.009916789004454892), ('jasa', -0.00898872943113184), ('barang', 0.004845610481323837), ('pajak', -0.0020980726882734546), ('daftar', -0.0012018651114075885)], [('pajakperasrakyat', 0.06376076184481996), ('rakyat', 0.0405782394656528), ('kaya', 0.03540928264646107), ('up', 0.018942948359991118), ('rajin', -0.017673836078106193), ('bayar', -0.01654248829791784), ('dinaikin', 0.010301692860191461), ('pajak', 0.004735440592091572), ('orang', -0.0033758393454300577)], [('rp', -0.12062875337258892), ('rugi', 0.02892151428499416), ('kritik', 0.0273259367458112), ('hapus', 0.027303191099546577), ('triliun', -0.02658371049812545), ('bara', 0.022860426343176886), ('negara', 0.022074510936656144), ('batu', 0.02203067771370034), ('kait', 0.020478286793714783), ('perintah', 0.015608345917916698), ('royalti', 0.015047263871638007), ('rizalramli', 0.013940423468324073)], [('pajakperasrakyat', 0.02813265998843887), ('nelayan', 0.007152253918723177), ('eh', 0.004871990006063091), ('tukas', 0.004808646862616541), ('pajak', -0.0042765004165695964), ('rizalramli', 0.003407318996170687), ('persen', 0.0012352345142114275)], [('rakyat', 0.09743719271850926), ('bayar', -0.04296173160493928), ('jabat', 0.04101071777703421), ('suaka', 0.03190762844214646), ('peras', 0.028783658297446727), ('nya', 0.021977891263352665), ('negara', -0.021269453366169408), ('pajak', -0.014375741837804022), ('gak', -0.009353201214720323), ('sembunyi', 0.007600675759268224), ('harta', -0.0010216305133581853)], [('sistem', -0.12062107862622692), ('rakyat', 0.05553111820892738), ('ppn', -0.047246643738969495), ('naik', -0.047005555898762985), ('laku', -0.03485077191467051), ('perintah', -0.021929340987008765), ('pajak', 0.006886499893839914), ('kapitalis', 0.005855644910850662), ('ekonomi', -0.004327189983330256), ('dpr', -0.003118029717917999), ('korban', -0.0006666695336963258), ('serbi', -0.0004541358244464793), ('akibat', 0.000431846782011046), ('april', 0.0002666231280662996), ('islam', -0.00019540860343049212), ('tetap', -0.00014317686706042483), ('mmc', 9.161859558398034e-05), ('serba', 8.858901988845985e-05), ('lantas', 2.9893419090565863e-05)], [('rakyat', 0.040933515386997715), ('pajak', -0.02698259890773656), ('amnesti', 0.007679789428271865), ('konglomerat', 0.007173847582248305), ('denda', 0.005092048208686928), ('telat', 0.0016597013425915569)], [('perompak', 0.030643849484987908), ('haram', 0.021375820576504202), ('pajak', 0.017331061325409243), ('sembunyi', -0.014614582893790754), ('bayar', -0.014510601435924744), ('ampun', 0.013146783478654464), ('jilidii', -0.012783980625728063), ('denda', 0.01070227740700409), ('nya', 0.00951477907395437), ('korupsi', 0.00860302319955724), ('halal', 0.007319706916088257), ('jaya', 0.006985724687757137), ('negara', 0.004572715078168642)], [('pajak', -0.014324522390287992), ('kejar', 0.009087557672134585), ('rakyat', 0.008312810078486967), ('pandora', 0.004664768721493201), ('masuk', 0.0030503088358103756), ('paper', 0.0029415908281307836), ('lapor', 0.0024635600910196532), ('pikir', 0.002274045109075776), ('skandal', 0.0020419554707765165), ('menko', 0.0019202398370338698), ('ironis', 0.0018945633651842935), ('duga', 0.0015836755729593266), ('mati', 0.001419742846495677)], [('taxamnesty', -0.10069664774929175), ('happy', 0.08958282831555664), ('kemplang', 0.08456421004160687), ('ekonom', 0.05363627792441034), ('bikin', 0.036985418923333534), ('pajak', -0.029239236537357157)], [('aja', 0.03622825687380463), ('naik', -0.027885059571496475), ('cepat', 0.025299210378915464), ('masyarakat', -0.02515242027656687), ('pph', -0.025111992498681783), ('kereta', 0.023316093184488285), ('amnesti', 0.018237950301563135), ('ppn', -0.0169691507626104), ('tombokin', 0.01619138111858081), ('syahwat', 0.01172192460565051), ('gak', 0.010917133755364763), ('moga', -0.009689957057907709), ('gede', 0.009320216998582742), ('rasa', 0.008459273536600462), ('tax', 0.008131616959572156), ('arti', 0.007439794074142192), ('bener', 0.007133989735657335), ('pajak', 0.004495316727665023), ('mah', -0.0033140821639813687), ('orang', -0.001786624306079302)], [('pahit', 0.017669073680933744), ('tajuk', 0.014478354780640553), ('harmonisasi', -0.013508303067186736), ('pil', 0.012077425461471098), ('pajak', 0.009969536558666609), ('uu', -0.009692135973878219)], [('duduk', -0.1267374619517574), ('wajib', -0.11960020507572619), ('rp', -0.09923940552740326), ('nik', -0.09374287277562417), ('aja', 0.047822961080138025), ('nya', 0.01712954852283617), ('bayar', 0.008237008316186649), ('lapor', 0.005612099471189704), ('kalo', 0.005090619826386394), ('juta', -0.004813416195185834), ('wp', 0.0043188890694398924), ('of', 0.0041432908130156635), ('woow', 0.003981338568153187), ('ide', 0.003768597477139639), ('pajak', -0.00364844390167689), ('trap', 0.0032566318529984642), ('gila', 0.003252452264518074), ('kind', 0.0031521238020279087), ('stiap', 0.0023205821880531276)], [('kemplang', 0.07771044973376705), ('taxamnesty', -0.05900894481183698), ('sepele', 0.053718717732445825), ('anggap', 0.0369369920935528), ('bijak', 0.02503552732506978), ('jilidii', 0.024343381272127085), ('program', 0.02241780734337977), ('pajak', -0.006687788016487661)], [('taat', -0.024400431702311775), ('rakyat', 0.013506266612200897), ('jabat', 0.013436959019543287), ('oligarki', 0.010379168685863416), ('pandora', 0.009479546251928045), ('pajak', -0.00861382577548451), ('papers', 0.007074151924852246), ('bisik', 0.005190441356886029), ('antek', 0.004993728442678515), ('suruh', 0.004777728785965151), ('ngemplang', 0.0031889475136003217), ('tri', 0.002955870495374338)], [('nikmat', -0.06067193401066955), ('pengemlang', 0.011885205222194847), ('rang', 0.011859203124903043), ('mantap', 0.010577213653462076), ('hapus', 0.009988676303585697), ('sanksi', 0.008476976591905102), ('coba', 0.007724708718996951), ('pidana', 0.007491727691583828), ('denda', 0.006654903824846785), ('pajak', 0.0036996880140884878)], [('taat', -0.023709183057041946), ('pandora', 0.0229093809429618), ('papers', 0.02108171378288579), ('rakyat', 0.019969514292118223), ('pajak', 0.01273724765054765), ('jabat', 0.012271487310407841), ('msuk', 0.010357500769680068), ('gimana', 0.009691852790484899), ('kabar', 0.008291114685962915), ('suruh', 0.007121507337122464), ('bayar', -0.005059143028815535)], [('jalan', -0.13331748471151045), ('stabilitas', -0.12161253781151067), ('hubung', -0.10478196467365322), ('dana', -0.0919849910210219), ('perintah', -0.0917510495342191), ('harga', -0.07371680965369078), ('milik', -0.07178625407745981), ('bijak', -0.05745864640807314), ('pajak', -0.025564264665118998)], [('jokowi', 0.02706798107730321), ('ampun', 0.02196093790176063), ('nilai', 0.021377790151991757), ('joko', 0.010656543723140385), ('presiden', 0.010336675367980438), ('widodo', 0.009814554504104605), ('pajak', 0.009624045635734196), ('izin', 0.009274079929651553), ('program', -0.0082322504150158), ('jilidii', -0.006763356749856554), ('langkah', -0.0028163553403858523), ('turun', -0.002384966030432761), ('taxamnesty', -0.0006913077670200329)], [('mah', -0.0194022672934683), ('pajak', 0.017919020195228733), ('nelayan', 0.008385817474720064), ('bayar', -0.006693000429634965), ('cuan', 0.005069183414128024), ('sanggup', 0.0029429361748792987), ('lokal', 0.002939727179777064), ('monggo', 0.002834421203192245), ('kapal', 0.0028236202427392512), ('saing', 0.002146058588650775), ('ancooor', 0.001991874477199959), ('sdm', 0.001946982227350282), ('minggir', 0.0018662423818253317), ('cina', 0.0015346739591924697), ('naker', 0.0006500599150254719), ('asing', 0.0005327890727570932), ('opung', 0.000499566253162582), ('balang', 0.00048455806308307165), ('silah', 0.00030621360935590265)], [('aja', 0.013437968279575562), ('rakyat', 0.012427560405580417), ('susah', 0.0059685329325951655)], [('rakyat', 0.05045888974795086), ('pajak', -0.02428228913973343), ('taxamnesty', -0.024132418915811745), ('jeniius', 0.019982175417277665), ('pilpres', 0.018311021731565447), ('plang', 0.017534612234092466), ('cukong', 0.016508806289387685), ('kasih', 0.016324883732132662), ('pesan', 0.015611281318913971), ('bijak', 0.009663371215115233)], [('hasil', -0.07449616159833546), ('imbas', 0.010077933268981699), ('panen', 0.0100468153045569), ('th', 0.010021425775080924), ('hancur', 0.009260225870653558), ('tani', 0.007285920235062069), ('pajak', -0.006130718188458813)], [('pisah', -0.044868357842942894), ('aja', 0.028206745990078004), ('dana', -0.027608447115606168), ('pajak', 0.01035206834052185), ('nkri', 0.006587383215672433), ('anggar', 0.004612250085194662), ('cinta', 0.003712729799800411), ('timor', 0.003679780426862815), ('dikaltim', 0.0034994426253012983), ('logika', 0.0028191765933567917), ('bumi', 0.0024019588626548118), ('ngotot', 0.0023733775084391817), ('ambil', 0.0020791644334551937), ('ikn', 0.0019618270222055314), ('banding', 0.0019386597397760433), ('kaltim', 0.0019245004553319332), ('dar', 0.0019028168790048757), ('leste', 0.001688591372554112), ('makmur', 0.0013113516517604753), ('mesti', 0.0013049485221367804)], [('rakyat', 0.13211170440318662), ('bebas', -0.07072847550373737), ('hasil', -0.034672363001405485), ('pemerin', 0.026428892270129685), ('pajak', -0.020475630611529662), ('makmur', 0.020187910442894183), ('beban', 0.018834768513085156), ('gagal', 0.018664210628909623), ('negara', 0.004842993689520461)], [('taat', -0.05980743249144844), ('bijak', 0.048831736984813974), ('makan', 0.0260254259006597), ('pajak', 0.007801396768263541), ('jahat', 0.006068504404277071), ('orang', -0.0043827001934828354)], [('rela', -0.22926703291524558), ('nggak', 0.043537578148365626), ('sulit', -0.02465469678767922), ('ekonomi', 0.02447360786551829), ('pajak', -0.022646098015372325), ('balik', 0.02017737876375792), ('imbal', 0.018574376321305135), ('asa', 0.018115043235627737), ('kurang', 0.016245039611810872), ('mahal', 0.015224492941917366), ('harga', -0.012286105778505264), ('aman', 0.009578893661838846), ('bayar', -0.008628408344055734)], [('rakyat', 0.02807428888059041), ('bayar', -0.021639526106487998), ('risol', 0.018457221172200906), ('chuakkkss', 0.01719522809053699), ('nis', 0.016856526803787016), ('dijadiin', 0.016129064772265094), ('peras', 0.013945584730142402), ('nya', 0.009019675480924815), ('pajak', 0.007468121442689634)], [('hasil', -0.028319841005765956), ('rakyat', 0.017287946003469647), ('kaya', 0.012586037445915188), ('apbn', 0.006546862143671505), ('pajak', 0.005431823798682812), ('proyek', 0.003856129143695656), ('duit', -0.0035750675187958313), ('buanyaak', 0.002377038595948264), ('raya', 0.002028434336969671), ('tuk', 0.0015507080968807732), ('gaji', 0.0011921936824224886), ('tkanya', 0.0010895275551925459), ('ironis', 0.0010758193087002682), ('tka', 0.0010710528776398094), ('ngegaji', -0.0009137975802531011), ('udah', -0.000702464518739246), ('jibun', 0.000694355729142142), ('pungut', 0.0006210223614799374), ('apah', -0.00047657351009796316), ('indo', 0.0004345696827672212), ('mgk', -7.43213951026202e-05), ('tsb', -1.7911787001907933e-05)], [('jokowi', 0.026247271753710846), ('nilai', 0.019851277289210673), ('perintah', -0.018961538038191412), ('ampun', 0.018896536423678396), ('kredibilitas', 0.01077446520202582), ('presiden', 0.010435774046978213), ('joko', 0.009902319889626436), ('widodo', 0.009327944940938554), ('program', -0.009083600969286837), ('pajak', 0.008372244674343646), ('jilidii', -0.007237035566901154), ('izin', 0.00685569151616273), ('langkah', -0.0012801026283477523), ('turun', 0.0008576145720800674), ('taxamnesty', -0.0003353149816044891)], [('ingkar', 0.08057239780363776), ('taxamnesty', -0.07434015489938503), ('cepat', 0.07408417354029759), ('kereta', 0.06839236136637804), ('jokowi', 0.06441497657037915), ('apbn', 0.0625518282361104), ('janji', -0.020262894369042918)], [('rakyat', 0.0797063766105125), ('kaya', 0.036465418643479665), ('cukong', 0.017812557629721518), ('opini', -0.017113914006866578), ('harta', -0.016660834551575247), ('oligarki', 0.015039177287497997), ('das', 0.01294820384613331), ('bentuk', 0.011229386072812052), ('hidup', 0.011113757995735545), ('bayar', -0.010189200519799274), ('zalim', 0.00895109138260499), ('bangun', 0.00888304607593272), ('taxamnesty', -0.007996023986120966), ('nikmat', -0.006549444354740341), ('bijak', 0.006412227577763705), ('kuasa', 0.005212521623856581), ('pajak', 0.003624594395381177), ('sulit', 0.00024710986329110995)], [('nikmat', -0.06356753331371037), ('beda', 0.006690379317339602), ('muslimahnews', 0.006356047028790664), ('cukon', 0.0059764093197591005), ('cerita', 0.005664624696969655), ('bakwan', 0.005444685001232917), ('udang', 0.005281723144361378), ('makan', 0.005138082304352394)], [('pagi', -0.2113642299187988), ('infrastruktur', -0.1669400205068961), ('selamat', -0.16465592432281914), ('genius', -0.13952763591921566)], [('negeri', -0.03525453917965044), ('titip', 0.021856304687783795), ('surga', 0.02177172344151275), ('news', 0.021383872183739747), ('muslimah', 0.02135746965939613), ('das', 0.0187229485961246), ('cukong', 0.01724282709057423), ('oligarki', 0.01653990201117957), ('taxamnesty', -0.0076634695386883145)], [('kemplang', 0.0746659550293379), ('taxamnesty', -0.05900689966968509), ('sepele', 0.054451535046980123), ('anggap', 0.036303224535678524), ('bijak', 0.027905877493663668), ('jilidii', 0.02544041437381706), ('program', 0.0207444033739347), ('pajak', -0.005094817512145987), ('b', -0.002634509640500459)], [('wamenkeu', -0.21706508263875393), ('klaim', -0.1670419811934696), ('proses', -0.13848152385809281), ('sederhana', -0.13630671512005252), ('taxamnesty', -0.09796050262762526), ('jilidii', -0.06451732299063177)], [('plinplan', 0.007303608988904577), ('parah', 0.0013131316785373937)], [('jakarta', 0.042624113280229844), ('bandung', 0.042156074636004175), ('jokowi', 0.03787982431999869), ('ingkar', 0.037517005827492726), ('cepat', 0.037251578638525094), ('kereta', 0.036546652945259696), ('taxamnesty', -0.032952557241322185), ('janji', -0.013725282621233877)], [('taxamnesty', -0.14994722752865136), ('konglomerat', 0.09166038443953468)], [('rakyat', 0.02446213154428349), ('nikmat', -0.019884257209499278), ('bakwan', 0.00985977400067276), ('cerita', 0.009688603794834365), ('udang', 0.009543100569544433), ('das', 0.00851141639987237), ('makan', 0.008261031529791282), ('beda', 0.00803965085651749), ('muslimahnews', 0.007854557768228097), ('cukong', 0.007649018054118582), ('rugi', 0.007434250673231603), ('taxamnesty', -0.003344679995751368), ('negara', 0.002763508657703821)], [('cegah', -0.2599960801653211), ('tuju', -0.23386103240592765), ('taxamnesty', -0.1398012944545149), ('hindar', -0.0991635650233081), ('jilidii', -0.07370165873444609), ('pajak', -0.035559922338764295)], [('rakyat', 0.07740110403281518), ('kaya', 0.023080722965050142), ('harta', -0.018279211977021353), ('oligarki', 0.017045030186305286), ('cukong', 0.016533584437506674), ('das', 0.014334229747643637), ('taxamnesty', -0.013605020782538037), ('bentuk', 0.013563666427295139), ('hidup', 0.012811064112219573), ('bayar', -0.011364929627060321), ('zalim', 0.010257258603536976), ('nikmat', -0.009563462916619176), ('bangun', 0.006819210721931887), ('kuasa', 0.0057911178099226525), ('bijak', 0.004901967620401125), ('pajak', 0.003407212818463546), ('sulit', -0.002258025611733036)], [('cepat', 0.03292873802241689), ('ulang', 0.03245775103257645), ('jokowi', 0.031867012073917324), ('ingkar', 0.031210560665483878), ('kereta', 0.03067918979705747), ('taxamnesty', -0.03021202454981332), ('utang', 0.02752124344499676), ('apbn', 0.024624012531088577), ('janji', -0.00853939838749317)], [('kemplang', 0.1013045536872811), ('tuju', -0.05339193104079845), ('indonesia', -0.04658246920525206), ('naik', -0.03789954130540195), ('direktur', 0.031661211177055504), ('abdullah', 0.02441433592440384), ('riset', 0.023614706488075506), ('core', 0.02233321852727894), ('pajak', -0.018260311598493055), ('piter', 0.01748005596267561), ('apa', 0.01426137007010222), ('pasu', 0.01379110212203963), ('sempat', 0.012105601994846968), ('negara', 0.0059358582423140556)], [('jokowi', 0.020006619571163304), ('cepat', 0.018510464315504075), ('kereta', 0.01754067801767441), ('jilidii', -0.014455693721218337), ('parah', 0.011089276027295638), ('taxamnesty', -0.010904561371687478), ('plinplan', 0.01076200003404757), ('apbn', 0.010230431974113047), ('emang', 0.009664603978943951), ('pakai', 0.008149586004616603), ('bijak', 0.005749627414338)], [('ga', -0.01660127909684592), ('rakyat', 0.015411859126869938), ('pulak', 0.011560368878086135), ('kaya', 0.011285099695798839), ('super', 0.010211351885971916), ('wqwqwq', 0.010203993768974743), ('dipajakin', 0.00976032381651808), ('apbn', 0.009124922319391795), ('defisit', 0.008723802868631116), ('kasih', 0.007688097205812128), ('taxamnesty', -0.0069434371917569924), ('sembako', 0.006124783590272997), ('miskin', 0.005679927088411763), ('orang', -0.001857737808320083), ('udah', -0.0013646605730085496)], [('kemplang', 0.13913467806700677), ('uuhpp', -0.11737180311265931), ('sanksi', 0.034425185306735225), ('pidana', 0.03338124779760562), ('dpr', 0.028105222688181282), ('atur', 0.0240495859374126), ('hilang', 0.022386045499003308), ('uu', 0.017254033542488383), ('harmonisasi', 0.011869218586221458), ('tuju', -0.011073176747716499), ('perintah', 0.008667235330587391), ('pajak', 0.0021657387205402485)], [('uuhpp', -0.17804880857160163), ('gotong', -0.06979855328307222), ('kemplang', 0.05874917575321663), ('rakyat', 0.028590195677514422), ('kaya', 0.0243401045670968), ('pajakperasrakyat', 0.016103730441035233), ('taxamnesty', 0.014996960295347726), ('jabat', 0.009489496270332014), ('konten', 0.008652170432938618), ('profesor', 0.006736423927875365), ('nasional', -0.005910651378907613), ('selasa', -0.0033326218073282433), ('rezim', 0.0032986326591708305), ('seminar', 0.0028562225278796027), ('kabinet', -0.002344668949702765), ('baim', 0.0020401848524440364), ('pajak', -0.001795580029698241), ('garong', -0.001750956771120769), ('orang', -0.0010807305772958227), ('kolega', -0.0009069771676837739), ('babi', -0.0008598543340968848), ('buta', -0.0007732670329748712), ('mustahil', -0.0005891614797248636), ('beban', -0.0005881600166309004)], [('uuhpp', -0.2021840720782325), ('pajakperasrakyat', 0.09034106702611108), ('rakyat', 0.047631517536264184), ('taxamnesty', 0.02004260128819537), ('klepon', 0.015052207246623239), ('dewe', 0.01387841438310423), ('profesor', 0.013321176373589542), ('uber', 0.012330420577294426), ('konten', 0.010917159553270665), ('seminar', 0.008201041142611575), ('prestasi', 0.007789114984198391), ('konco', 0.007553864939243661), ('baim', 0.00534304232097656), ('nasional', -0.001981657893852327), ('pajak', 0.0010369506180767558), ('selasa', 0.00023761979892845825)], [('jokowi', 0.00873781997101738), ('kereta', 0.008085362959375073), ('taxamnesty', -0.008064404921642475), ('ii', -0.006534727487499952), ('ekonomi', -0.0052535406473004625), ('cepat', 0.004503644247094999), ('utang', 0.00391347925704267), ('ingkar', 0.0036325421019135053), ('apbn', 0.003620257491839607), ('bohong', 0.00234343063331774), ('bidang', 0.0018155705112799815), ('politik', 0.00167025503531614), ('pling', 0.001351451037076702), ('alias', 0.0013215583556815632), ('plang', 0.0012292877999415318), ('konsisten', 0.0011804575144579795), ('janji', -0.0009593720654007355), ('pake', 0.000887437515394954)], [('kereta', 0.008344808695653301), ('jokowi', 0.008209171261835017), ('taxamnesty', -0.007967802271436408), ('ii', -0.0063235065485840415), ('ekonomi', -0.005091070676016504), ('cepat', 0.004480628453344567), ('utang', 0.003919596765138836), ('apbn', 0.0033406086552235396), ('ingkar', 0.003130415750693787), ('bohong', 0.0023716993518194843), ('pling', 0.001596983648272111), ('alias', 0.0014733580935510579), ('bidang', 0.001411745239294297), ('politik', 0.0013733145944442714), ('janji', -0.001335549031390821), ('konsisten', 0.0008447079208567583), ('pake', 0.0008129505759547274), ('plang', 0.0008108229844669208)], [('periode', 0.13123956518726404), ('ulang', 0.12717641789473427), ('kelompok', 0.12682294109478093), ('bebas', -0.08900750549513811), ('taxamnesty', -0.021341930718349746), ('pajak', -0.006778307389084037)], [('tutup', -0.09979822154335709), ('rakyat', 0.09362213314035944), ('kemplang', 0.05350744905686188), ('rugi', 0.03312724385477638), ('bijak', 0.025161643495721718), ('pajak', -0.012167035637460206), ('setor', 0.011511336479054645), ('beban', 0.010935790754957939), ('dapat', 0.009837069878541447), ('jenius', -0.008033716913141734), ('hilang', 0.007145381945365223), ('negara', 0.00657168474903685), ('taxamnesty', -0.006333428175498753), ('mata', -0.00470469848643557), ('jilidii', -0.00023822310830839348)], [('jokowi', 0.02941463309473473), ('jilidii', -0.0221365745515634), ('ubah', -0.021991276535593025), ('cepat', 0.01782924107423224), ('apbn', 0.016367383941488026), ('kereta', 0.014811446174235295), ('ditaxamnesty', 0.006500561734494379), ('inkonsistensi', 0.005877409153100613), ('kebija', 0.005612946029902316), ('telan', 0.004971392427545179)], [('taxamnesty', -0.019359716813335547), ('kepentingn', 0.018007481164762123), ('utang', 0.01743580726832928), ('plang', 0.014238982120354828), ('presiden', 0.013303586370248885), ('penuh', 0.013108867366152096), ('blbi', 0.01306800181427394), ('jelas', 0.012791288012629267), ('bijak', -0.004608124521488292), ('usaha', -0.003244811332858636), ('pajak', -0.002843346140298779)], [('wajib', -0.15602750626829526), ('indonesia', -0.06167129279902901), ('petinggi', -0.047793676738676336), ('pandora', 0.04557309214379867), ('harta', -0.03600255594522456), ('ampun', 0.03415720052287944), ('taxamnesty', -0.027653345289947104), ('papers', 0.0219390047304872), ('pajak', -0.01750357400601503), ('negeri', -0.011641527722476102), ('sembunyi', -0.011258838983449461), ('seret', 0.007584920328496408), ('nama', 0.007442027786558862), ('heboh', 0.007060765179101382), ('amanatinstitute', 0.006855271817372549), ('skandal', 0.00528431265456343), ('k', 0.00044246348464606615), ('a', -0.00040979517557010166)], [('jenius', -0.38738588106576155), ('bebas', -0.2551851459061571), ('orang', -0.12322234556908118)], [('reformasi', -0.2734903082189587), ('paja', -0.14962469338506182), ('sistem', -0.09697135290185652), ('kumpul', -0.06894538068519197), ('salah', -0.0616311962256037), ('taxamnesty', -0.046054973276043853), ('perintah', -0.04416154828318784), ('program', -0.036188831316122785), ('pajak', -0.030232200493440248), ('target', -0.017653493746271287)], [('jokowi', 0.01705032421377707), ('jilidii', -0.012912736560512166), ('kereta', 0.01252106038648359), ('cepat', 0.011771160435473466), ('apbn', 0.007206642798544769), ('plinplan', 0.0064239526985123046), ('lol', 0.00608563711519968), ('inget', 0.005237353079378136), ('buzzer', 0.005234064300254482), ('bangga', 0.005118242515533023), ('nkri', 0.004707046524700071), ('pakai', 0.004227458631784745), ('bijak', 0.0032733194433705766), ('taxamnesty', -0.0025632815382690393)], [('taxamnesty', -0.11489308924322085), ('pajakperasrakyat', 0.08744433595946718), ('ampun', 0.02415525914280626), ('hasil', -0.0006526262964755725), ('cukong', 0.00017568231251363398)], [('pajakperasrakyat', 0.026779632675803394), ('taxamnesty', -0.020240814237246104), ('entu', 0.013751487644951923), ('rakyat', 0.013046191762857946), ('ade', 0.012797343195724738), ('bang', 0.010930829703257375), ('kalo', 0.0075874137442070565), ('kaya', 0.0033293695864682806)], [('taxamnesty', -0.011718242156533408), ('rakyat', 0.007789590787256648), ('situ', 0.0066455269278771125), ('pajakperasr', 0.0062483429495381665), ('oligarki', 0.005584996389270341), ('dinaikin', 0.005096833237994422), ('raya', 0.005071599813467192), ('horang', 0.0050299652685903634), ('bong', 0.004982652969320715), ('kek', 0.004776778822497435), ('nelayan', 0.004611487284513009), ('waras', 0.004565841869095721), ('kaya', 0.0033922509393649086), ('pajak', 0.003299227347108016)], [('dorong', -0.22161373285194846), ('properti', -0.10982902060068744), ('sektor', -0.10773949034238985), ('indonesia', -0.10710151101843654), ('rei', -0.09810751408020203), ('berarap', -0.09700705320318122), ('tumbuh', -0.061859695593895335), ('taxamnesty', -0.04543618899355684), ('pajak', -0.03464787211862863), ('ampun', 0.029519500234900103), ('bijak', -0.010732110530065713)], [('ngutang', 0.17977148450774152), ('rp', -0.17752335377321782), ('aja', 0.12072520114401723), ('raup', -0.10690827977761685), ('taxamnesty', -0.057897991544236596), ('ah', 0.05109916664567313), ('up', 0.050705758368619944), ('ppn', -0.024797451641973143), ('menkeu', -0.020932729750162464), ('triliun', -0.005185472429496193)], [('taxamnesty', -0.031501083191481725), ('cepat', 0.030442742101155593), ('kereta', 0.029968399288851904), ('jokowi', 0.026813145519590395), ('apbn', 0.02278982716642912), ('pakai', 0.021317691791267933), ('plinplan', 0.020775390306553447), ('bijak', 0.011492691296327927), ('jilidii', -0.0091566304498709)], [('rakyat', 0.06447400518779035), ('taxamnesty', -0.037768252865197846), ('pajak', -0.036680171850242), ('cukong', 0.021270945843986068), ('nak', 0.020394046237117346), ('kasih', 0.018562456608048283), ('batubara', 0.018384523859357713), ('hebat', 0.01725267810907146), ('hapus', 0.015197318362600971)], [('taxamnesty', -0.15494730333400566), ('pajakperasrakyat', 0.10041032412425863), ('rakyat', 0.05944801591469516), ('pajak', -0.02783246921379029), ('usaha', 0.008557758746156182), ('ringan', 0.0015241441572878397)], [('taxamnesty', -0.07630575036106901), ('gagal', 0.07626143067574022), ('kabar', 0.06819503446624564), ('eh', 0.06554384991986366), ('ya', 0.06042490381684731)], [('rakyat', 0.04595903943867814), ('taxamnesty', -0.03283128196198613), ('kemplang', 0.032406878357671684), ('kaya', 0.031401282070496604), ('raya', 0.017701226237584758), ('jabat', 0.016503965734828693), ('pajak', -0.014370816915475447), ('miskin', 0.013745539247183114), ('peras', 0.013343179395828136)], [('rakyat', 0.04449774059301918), ('taxamnesty', -0.0330365799786569), ('kemplang', 0.032828633425124154), ('kaya', 0.028297527692228944), ('raya', 0.016767189114171025), ('jabat', 0.015139028507491463), ('peras', 0.01492542479103436), ('pajak', -0.0137275178867058), ('miskin', 0.012138997406744489)], [('ampun', 0.024868748036045326), ('jilidii', -0.022167708406715236), ('nilai', 0.019204506352228663), ('ekonomi', 0.014160484062604091), ('bhima', 0.01158450510739493), ('yudhistira', 0.010721597645071279), ('pajak', 0.009645271703738554), ('celios', 0.009477969315720602), ('efektif', -0.006258549054288731), ('direktur', 0.0025819231373205224)], [('strategi', -0.05971417024686812), ('dampak', -0.012585568256028579), ('jokowi', 0.011928867969656394), ('bohong', 0.00973467174127472), ('uang', 0.0073509850761965265), ('proyek', 0.004556322209397507), ('negara', -0.004375577833801061), ('berita', 0.004217191892621082), ('kelola', 0.004213679555550985)], [('percaya', -0.041053199047237615), ('bohong', 0.02683669036803629)], [('taxamnesty', -0.08525500858366705), ('daiam', 0.07462682237492917), ('kemplang', 0.07268023617809384), ('rakyat', 0.03978298287365207), ('usaha', 0.03402620245377752), ('pajak', -0.021854661732886308)], [('bisnis', 0.053162319685862935), ('cepat', 0.046989434292773594), ('ingkar', 0.04239482779420569), ('kereta', 0.04222134943401438), ('taxamnesty', -0.03999721019178178), ('jokowi', 0.03986299511651795), ('apbn', 0.03351232899459966), ('janji', -0.010212372555249186)], [('rakyat', 0.05667078665949143), ('kemplang', 0.045941486690149115), ('pajakperasrakyat', 0.04200313092266762), ('taxamnesty', -0.04160528095264894), ('naik', -0.04089021703994694), ('diam', 0.03600883546251483), ('usaha', 0.01928330073900696), ('pajak', -0.014689162747841244)], [('pajakperasrakyat', 0.07034543796730094), ('nilai', 0.0391696961679843), ('blunder', 0.02826810445400151), ('dampak', 0.020652357492764485), ('bhima', 0.01967378110418809), ('turun', 0.012786515323335319), ('patuh', -0.01047171374051711), ('taxamnesty', -0.008245035122160899), ('jilidii', 0.006176829653610363), ('pajak', 1.117488093609514e-06)], [('mutih', 0.12421085539400036), ('pajakperasrakyat', 0.10978346672917084), ('taxamnesty', -0.08257785767405304), ('jahat', 0.06634001567594514), ('jilid', -0.06005956606208254), ('kasus', 0.025700905199996914), ('uang', 0.017317939376463462), ('anggap', 0.01423976086070707), ('pajak', 0.010299672139621846)], [('pajakperasrakyat', 0.03915596529210056), ('dosa', 0.026899854224014776), ('jilid', -0.02470069988009394), ('jabat', 0.011410418297957371), ('ting', 0.011394636837104474), ('pasca', 0.01070607812881631), ('uang', 0.010267253695734547), ('era', 0.009538177822905098), ('putih', 0.009464112018223229), ('taxamnesty', -0.004968494837340991), ('usaha', 0.002506001363330532)], [('kereta', 0.022310699170464802), ('cepat', 0.02225249100881278), ('jokowi', 0.02220132383285531), ('taxamnesty', -0.018999887049778143), ('plinplan', 0.016929281244993603), ('apbn', 0.016779411854945803), ('rockygerung', 0.01613124996245504), ('pakai', 0.012891708887016668), ('jilidii', -0.012885326715694494), ('bijak', 0.010308475260921806)], [('pajakperasrakyat', 0.012742630163152503), ('taxamnesty', -0.012190661886697703), ('situ', 0.006803910531038782), ('raya', 0.005761284322293097), ('dinaikin', 0.005169907896030194), ('rakyat', 0.004971393125306771), ('bong', 0.004554315806995712), ('nelayan', 0.004453456827842299), ('horang', 0.004402779841024682), ('kek', 0.004366776464530668), ('waras', 0.004216375025681391), ('oligarki', 0.0035950302038282277), ('kaya', 0.003025807304396059), ('pajak', -0.00022942837748689976)], [('gara', 0.16103149271217412), ('taxamnesty', -0.07002363604988841), ('kemplang', 0.0425039573608422), ('pajak', -0.018603096640087195), ('fakta', -0.009006805887132282)], [('jokowi', 0.025161469748147508), ('ubah', -0.023773995443470884), ('cepat', 0.01901093323174136), ('dana', -0.017479062402914473), ('kereta', 0.015143090856070242), ('pajakperasrakyat', 0.01348056577926227), ('apbn', 0.010508027699412285), ('jilid', -0.010356036602943973), ('ampun', 0.007877840326992512), ('pajak', -0.005638931868861338), ('jakarta', 0.0027389940654435), ('proyek', 0.0022983099446477104), ('inkonsistensi', 0.0019355585909597533), ('taxamnesty', 0.0017835834215761), ('jilidii', 0.001600497358754228), ('ditaxamnesty', 0.0015150131979368277), ('dekat', 0.0009775959785277885), ('telan', 0.0007203056799701779), ('biaya', 0.00039710420452445744), ('bijak', 0.0002936783194652453), ('bandung', 0.0001404545013522411)], [('ruuhpp', -0.11124992079206533), ('ampun', 0.07038103783602201), ('kontroversial', 0.05869226225363514), ('poin', 0.04094889942311404), ('taxamnesty', -0.03610661536800416), ('salah', -0.03339978133833552), ('program', 0.0019468460176579197), ('pajak', -0.0013551570406619305)], [('pajakperasrakyat', 0.05515699964676708), ('patuh', -0.03160316605336189), ('nilai', 0.02812874570449507), ('blunder', 0.013240738495569844), ('duh', 0.007678432871581067), ('taxamnesty', 0.007053475638799584), ('id', 0.0060663102469499935), ('bhima', 0.005720691297200671), ('tirto', 0.005711195602835174), ('dampak', 0.0035467012286786886), ('pajak', -0.0012588126188012693), ('turun', 0.0008281358788522706), ('jilidii', 0.0004796980889143039)], [('taxamnesty', -0.044215679177902825), ('uang', 0.041055268223007126), ('kecewa', 0.038588050423727545), ('mulyani', -0.03475239932106596), ('sri', -0.02640739616246781), ('terima', 0.020267246048981787), ('menteri', 0.01889451904812816), ('negara', 0.017194076885592845)], [('jangka', -0.027947049270135943), ('btw', 0.015063170405373694), ('taxamnesty', -0.013882683880412533), ('korup', 0.013024506776549368), ('goblok', 0.011965960141497417), ('kali', 0.01096371457108747), ('bijak', 0.009403505551791222), ('alias', 0.009284546861743398), ('bodoh', 0.005750972616892897)], [('pajakperasrakyat', 0.06430682435731437), ('taxamnesty', -0.05713546170249052), ('happy', 0.05182313948632013), ('kemplang', 0.046799217332933206), ('ekonom', 0.034785923470068814), ('bikin', 0.03134040419443104), ('pajak', -0.007793517299393389)], [('pajakperasrakyat', 0.04705475533091182), ('mulyani', -0.02426362354916169), ('kecewa', 0.020578898426093478), ('uang', 0.01764757323685806), ('signifikan', 0.01707911283881133), ('negara', 0.01679583601386292), ('sebut', 0.01601425937420854), ('sri', -0.013832016372605204), ('terima', 0.009950457622647456), ('taxamnesty', -0.007163932102583333), ('menteri', -0.0035623897605184655)], [('pajakperasrakyat', 0.19876113276970425), ('uuhpp', -0.1328145299088969), ('wajib', -0.09599510064773964), ('taxamnesty', -0.0406750950033114), ('kritik', 0.030211355172744182), ('sempat', 0.022282459026926577), ('jarak', 0.020977913703435276), ('satu', 0.020028066131713228), ('anggap', 0.01947338578783613), ('konsisten', 0.018503377322711716), ('balut', 0.018013313015462912), ('jilidii', 0.014708403621561995), ('lampau', 0.014699513181357439), ('perintah', 0.009998439891790854), ('program', 0.008236210222630958), ('pajak', -0.0014495998173790368), ('jilid', 0.00012505098526574437)], [('pajakperasrakyat', 0.11643594887367442), ('ruuhpp', -0.07635625692023248), ('ampun', 0.03877516705066094), ('kontroversial', 0.03136955997717926), ('ada', 0.02856341065147994), ('salah', -0.028365944097501074), ('taxamnesty', -0.02204481404076753), ('poin', 0.02180677831330024), ('pajak', 0.004047030485904675), ('program', -0.0038835989162256546)], [('buru', 0.03586667605359384), ('pajakperasrakyat', 0.02815283862793444), ('taxamnesty', -0.02163488119072899), ('kemplang', 0.019645386194131798), ('ada', 0.01552531639767879), ('ampun', 0.014727137655447005), ('receh', 0.013133966238076715), ('pajak', -0.0009192414434621102)], [('nilai', 0.024887165937693422), ('jilidii', -0.023661320015756028), ('penuruna', 0.015329771743408733), ('bhima', 0.014806674628355597), ('hmm', 0.014200556974649546), ('simak', 0.014114365187037658), ('blunder', 0.011327743460122778), ('dampak', 0.009592623857643834), ('taxamnesty', -0.00902012243049866)], [('pajakperasrakyat', 0.0613510149604782), ('jilid', -0.025311812336788527), ('ampun', 0.020380194047882695), ('cukong', 0.014314528941954428), ('hasil', -0.010044896167238446), ('musti', 0.008746035732188576), ('evaluasi', 0.00873650061329739), ('pakai', 0.007648633747105788), ('transparan', 0.007193123097611031), ('sih', 0.006684469028436326), ('taxamnesty', -0.005313850645838009), ('kasih', 0.004486261600608754), ('gak', 0.002585992670678013)], [('ampun', 0.014965460431500842), ('nilai', 0.0052303541927860185), ('pajak', 0.0049840076056999), ('celios', 0.004290030402517124), ('yudhistira', 0.004015947670850063), ('kumat', 0.0032434114451294297), ('bhima', 0.0031056040176511094), ('ngempalang', 0.0029257376075991176), ('ekonomi', 0.0014755539366349145), ('direktur', -0.0009874043906435428)], [('rp', -0.20043760774469926), ('ngutang', 0.1978459525403623), ('aja', 0.12074066417877624), ('raup', -0.1080085717150298), ('taxamnesty', -0.07606098961820665), ('ah', 0.05499032288008027), ('ppn', -0.02907033864720025), ('menkeu', -0.022176785549117128), ('triliun', -0.013800219833940634)], [('pajakperasrakyat', 0.02347139601197153), ('kemplang', 0.017186997123581575), ('bhima', 0.009791800014781154), ('ampun', 0.00903431794224054), ('nilai', 0.008800288151004811), ('taxamnesty', -0.008535964683825729), ('yudhistira', 0.006877066808197454), ('celios', 0.004130731310069436), ('pajak', -0.004012442603360005), ('jilidii', -0.00398552496673692), ('amat', 0.0036733101518800337), ('efektif', -0.003070964643455639), ('bahagia', 0.0026853356133866354), ('sebut', 0.00264228606524575), ('ekonomi', -0.0010532842806761575), ('direktur', 0.00032598379721915165)], [('pajakperasrakyat', 0.07441694218366104), ('nilai', 0.03878742162965411), ('blunder', 0.026910007685696773), ('dampak', 0.018284740350612044), ('bhima', 0.01703521400350014), ('patuh', -0.013511981607814973), ('turun', 0.011037004883383878), ('taxamnesty', -0.003473477727673229), ('jilidii', 0.0028007187846111597), ('pajak', -0.0007743337705667066)], [('ubah', -0.058652977087040016), ('sulit', -0.03354829664040876), ('dusta', -0.012924262791352743), ('inkonsistensi', 0.004724435943372737), ('darah', 0.00017147241442118667), ('jokowibohonglag', 8.425579790013206e-05), ('daging', -3.136878647417369e-05)], [('jenius', -0.20957671183933946), ('taxamnesty', -0.08073369439054849), ('utang', 0.07502457431685593), ('cepat', 0.07255002895219759), ('kereta', 0.0678729100155426)], [('ingkar', 0.07816076513560065), ('taxamnesty', -0.07233921277851103), ('cepat', 0.06931998732294045), ('kereta', 0.06618883446156211), ('jokowi', 0.06414108653009025), ('apbn', 0.060270059847770494), ('janji', -0.018562075177111725)], [('kemplang', 0.04066997558307126), ('taxamnesty', -0.030359767474936312), ('bukanberitabiasa', 0.029968093580883405), ('sindonews', 0.029682679570079633), ('happy', 0.026437839115999933), ('ekonom', 0.024116754214496865), ('bikin', 0.022357084670523227), ('pajak', -0.004817119548423683)], [('jokowi', 0.06954188990071279), ('jenius', -0.06738909207750483), ('tempo', 0.032018188965204704), ('tapiboong', 0.03093104276374199), ('catat', 0.030009069619763632), ('bidang', 0.028979785695800828), ('bukti', 0.02470684375497014), ('ekonomi', 0.02142633344570029), ('b', -0.0005970948067341983), ('janji', 0.00034619233234929933)], [('janji', -0.13792562370755557), ('ingkar', 0.09769467152152139)], [('kemplang', 0.07776950119206683), ('taxamnesty', -0.0581716085713965), ('sepele', 0.05407094331583751), ('anggap', 0.03602622715009648), ('bijak', 0.029539953707866926), ('program', 0.02746890805763343), ('jilidii', 0.02725981596622812), ('pajak', -0.007161385259626518)], [('jokowi', 0.02667532547859521), ('nilai', 0.02251052342970633), ('ampun', 0.021622037192942034), ('presiden', 0.011994341151202683), ('joko', 0.011258697542822691), ('widodo', 0.010347660579456505), ('pajak', 0.00868657699948159), ('izin', 0.008524619088884508), ('program', -0.007044221321127036), ('jilidii', -0.006656365764797109), ('langkah', -0.0025940519322840637), ('taxamnesty', -0.0016740142487012897), ('turun', -0.001461528271773446)], [('rakyat', 0.049494651742746594), ('taxamnesty', -0.023689294155654787), ('pajak', -0.022719430728745892), ('jeniius', 0.02015200322634437), ('cukong', 0.017959127859301198), ('kasih', 0.01746735832002629), ('pesan', 0.01707235359106003), ('pilpres', 0.01683627454497297), ('plang', 0.0160965843740153), ('bijak', 0.00631527960209828)], [('jokowibohonglagi', 0.07550600793443168), ('ingkar', 0.07499182296401895), ('cepat', 0.06552215350904064), ('taxamnesty', -0.06181039222293723), ('kereta', 0.05937602707921675), ('apbn', 0.053374839717388045), ('janji', -0.019440796158860994)], [('sektor', -0.09618229504027366), ('pajak', -0.02946747004884192), ('ampun', 0.02900788536590706), ('gagal', 0.01673872212629515), ('asik', 0.016416065622250054), ('ngulang', 0.016406953821384445), ('juni', 0.015334286661125029), ('ndak', 0.01212920946494539), ('pasu', 0.011891523970428907), ('januari', 0.011307891635705196), ('nambah', 0.009693312914421295), ('taxamnesty', -0.007695465012304365), ('negara', -0.004505542844226512), ('jilidii', 0.0001872162338750782), ('program', 9.536877128142904e-05)], [('jokowi', 0.02321554583205058), ('cepat', 0.022907875483591085), ('kereta', 0.022127020182036806), ('taxamnesty', -0.018860565764950037), ('rockygerung', 0.016995320401048133), ('plinplan', 0.01602285552528201), ('apbn', 0.015812342396109753), ('pakai', 0.012827450574390682), ('jilidii', -0.01192950294834793), ('bijak', 0.008976227377679731)], [('kereta', 0.022616531643917342), ('cepat', 0.022317182058665754), ('jokowi', 0.020817914483941158), ('rockygerung', 0.017198443049127373), ('taxamnesty', -0.017105518113671055), ('plinplan', 0.015023222637778484), ('apbn', 0.014795028142476904), ('pakai', 0.012338883628911284), ('jilidii', -0.012300737413509795), ('bijak', 0.00639732303899214)], [('saking', 0.06934446944943366), ('jokowi', 0.06518918650282976), ('situ', 0.057811943821563544), ('taxamnesty', -0.014036751938125064), ('genius', -0.012752600196369366)], [('bijak', -0.01430620475509161), ('plinplan', 0.005825861783645061), ('jokowibohonglagi', 0.00424664194358121), ('udah', 0.003928486405226621), ('mencla', 0.0035179242517056936), ('mencle', 0.0033553843469352174), ('alias', 0.0028503297004157037), ('laah', 0.001564801092628761)], [('jenius', -0.3490303308608161), ('janji', -0.04750184164626427)], [('trik', 0.04479936064470134), ('rakyat', 0.027597165494268865), ('menzolimi', 0.025981419437498515), ('rugi', 0.01822501134089938), ('dalih', 0.017647992766926156), ('taxamnesty', -0.016946032828487598)], [('kemplang', 0.108035412795529), ('ampun', 0.06148885010317073), ('sesuai', 0.05965295381764045), ('hukum', 0.04417494395361541), ('atur', 0.03474642423551113), ('jilid', -0.0345487961541805), ('pajak', 0.01684443296032315), ('taxamnesty', -0.014158449166245804), ('patuh', 0.002045694653786537)], [('janji', -0.13752400760039676), ('ingkar', 0.03947587677032425), ('kaget', 0.03641749771541759), ('bikin', 0.03627167598509748), ('tepatin', 0.02955802170626216), ('jago', 0.02643420161137676)], [('kemplang', 0.07456239740669235), ('taxamnesty', -0.058385062507743356), ('sepele', 0.050854710687298146), ('anggap', 0.037640386449307665), ('jilidii', 0.025733137578618732), ('bijak', 0.02357244122322531), ('program', 0.02320908034144465), ('pajak', -0.005547793718641688), ('i', -0.0033653690308265228)], [('jokowi', 0.018986299587851686), ('cepat', 0.013818221624071507), ('kereta', 0.012555035114852365), ('taxamnesty', -0.0118410595164992), ('htt', 0.009666932806352427), ('ingkar', 0.00943795008019679), ('apbn', 0.00879607557957823), ('bohong', 0.007875452926455955), ('tagar', 0.0073940479165869105), ('pas', 0.007198359858608724), ('janji', -0.005098226686658649)], [('pribadi', -0.041344226847745656), ('jokowi', 0.010595257374219425), ('bohong', 0.006295561055691475), ('rezim', 0.005055305631163629), ('tetpi', 0.0037082093013361297), ('munafik', 0.0018065978144741084), ('lekat', 0.0015052289374511748), ('serang', -0.0004757040836400715), ('bicara', -0.00044041757498610953), ('oraig', -0.00022765748810575054)], [('cepat', 0.024113336350243545), ('jokowi', 0.020722849262056225), ('taxamnesty', -0.020465717602129604), ('kereta', 0.019318009629391134), ('ingkar', 0.01921325031704492), ('tagar', 0.01723280406685002), ('pas', 0.016916657498189158), ('apbn', 0.01622304194315287), ('jokowibohonglagi', 0.015539622987633192), ('janji', -0.008044441046585598)], [('pribadi', -0.039378373194104664), ('orang', -0.03435447194067928), ('jokowi', 0.012484641984788485), ('munafik', 0.011351731988051885), ('ingkar', 0.008429942319970806), ('rezim', 0.005918910106887686), ('bohong', 0.005529742624865652), ('tetpi', 0.005394910580876574), ('lekat', 0.003893846841743287), ('sifat', 0.003146352152920705), ('serang', 0.002649131508332594), ('hati', 0.0022299307100620146), ('jokowibohonglagi', 0.0021919073953040723), ('islam', 0.002061836063988753), ('mulut', 0.0019889354327263047), ('bicara', 0.00121611162806149), ('janji', -0.000979858098860843)], [('lalai', 0.194873311941944), ('tiada', 0.14098337721378795), ('taxamnesty', -0.0804440261723909), ('masyarakat', -0.030707961446774266), ('bayar', -0.017793785699674856), ('pajak', 0.013368137712453938), ('jilid', -0.010722285046342898)], [('pandora', 0.06443748600495766), ('selamat', -0.06100117083804558), ('pampers', 0.05804781199112547), ('tenaang', 0.05200328754520666), ('taxamnesty', 0.004281166138151084)], [('tuju', -0.20486815348607304), ('beritaterkini', -0.10135842890591643), ('pajak', -0.08427989643615895), ('taxamnesty', -0.07541066576497577), ('perintah', -0.06424626118136904), ('terima', -0.06175888319529911), ('genjot', -0.052159204790969686), ('jilidii', -0.0479715619163194), ('ampun', 0.04740330341315535), ('rencana', -0.03807460155734003), ('ekonomi', -0.03358760372790744), ('negara', -0.020364695914368073), ('bijak', -0.01610738376838859)], [('apbntotal', 0.21062344717945655), ('taxamnesty', -0.11108177538059451), ('janji', -0.09579744167822908), ('cepat', 0.06325818791447167), ('kereta', 0.0498564919034818)], [('efek', -0.020770032462222775), ('geninus', 0.017169115098250157)], [('aja', 0.01232686701897548), ('ya', 0.007718747461483123), ('elah', 0.0033671197578307916), ('bener', 0.003038590635329662)], [('hestek', 0.00992908395247111), ('jokowibohonglagi', 0.005937012729974311), ('lho', 0.0039386397787187786)], [('pedas', 0.05051113285650713), ('taxamnesty', -0.0457900519409312), ('tanggap', 0.044758516172637985), ('kritik', 0.03237802731042035), ('ii', 0.030689161552167738), ('perintah', -0.013299864791750016)], [('jokowi', 0.011106074973472874), ('tapiboong', 0.006866777600462251), ('bukti', 0.0054992521156381445), ('bidang', 0.004893596141294415), ('tempo', 0.004569857309819616), ('tepat', 0.004050549417604694), ('ekonomi', 0.0035568689824127366), ('catat', 0.003472680243689928), ('rinciannya', 0.00288110562539894), ('janji', -0.0028415991315898564), ('jokowijenius', 0.0022841059253636995)], [('jokowi', 0.09445588748727797), ('jenius', -0.0588268311614104), ('ya', 0.04205415669880787), ('minggu', 0.03881105155733013), ('bijak', 0.03238323050797244), ('plinplan', 0.02695870339375915), ('genius', -0.0237990640745152), ('ga', -0.01481328460770026)], [('janji', -0.12748481079612378), ('tinggal', 0.06220828024021796)], [('jokowi', 0.040717228690006685), ('plinplan', 0.034344564547013776), ('jenius', -0.03106122451983758), ('jilid', -0.02728670468350755), ('bijak', 0.027097933425379767), ('plongo', 0.018345961473102062), ('kerap', 0.01788058640240538), ('linglung', 0.016451888815211025), ('planga', 0.014672517322041334), ('kali', 0.013873671900581386), ('alias', 0.01320505810859446), ('orang', 0.005294933005626585), ('taxamnesty', 0.003414473043207608)], [('kemplang', 0.12033688105747554), ('taxamnesty', -0.08213495079976038), ('ampun', 0.051849819514181696), ('atur', 0.027952537544237217), ('jilid', -0.026894950941513056), ('hukum', 0.014999857352212896), ('sesuai', 0.013618862455940286), ('jelas', 0.013253635033964963), ('pajak', 0.011480489111869874), ('patuh', -0.006649161048878721), ('jilidii', 0.0054328264367473145)], [('cepat', 0.022515023527353316), ('jokowi', 0.02197040370341234), ('kereta', 0.021529164761314376), ('taxamnesty', -0.018544156715534346), ('rockygerung', 0.017850684764132745), ('apbn', 0.017634608310118812), ('plinplan', 0.016094246646925006), ('jilidii', -0.013406719215859373), ('pakai', 0.012800281482434976), ('bijak', 0.00981404343936017)], [('nilai', 0.07156368964365617), ('blunder', 0.05387089263129976), ('dampak', 0.03454410966873948), ('taxamnesty', -0.03237175339882279), ('turun', 0.02956172005955789), ('pajak', -0.02179692722897779), ('jilidii', 0.013180989331207671), ('patuh', -0.0074980116789941445)], [('ubah', -0.03153472360585194), ('jokowi', 0.016755971775595027), ('bohong', 0.007860058456993666), ('bijak', -0.0047115675529045544), ('presiden', 0.004305212507064544), ('widodo', 0.003628725273065147), ('joko', 0.0014761383292980412)], [('perintah', -0.03209309442487738), ('poco', 0.02247510894645146), ('cepat', 0.016510884476031884), ('jokowi', 0.0158585342746103), ('kereta', 0.013683985860318857), ('daftar', -0.011036261842626644), ('jilidii', -0.010542714974615012), ('taxamnesty', -0.009647300922690739), ('bijak', -0.00892313154310974), ('apbn', 0.007812345967709942), ('inkonsistensi', 0.006465240175598409), ('jokowibohonglagi', 0.00587210517065139), ('bohong', 0.004502119983191209), ('telan', 0.0038155954218107597), ('tambah', 0.0030262816321933858)], [('kemplang', 0.13379065849513408), ('sesuai', 0.07806838745015011), ('hukum', 0.059679958793492166), ('atur', 0.050845427432060705), ('jilid', -0.049359950803582464), ('taxamnesty', -0.030216422323761663), ('pajak', 0.017068781885562812), ('patuh', 0.009423784654298767)], [('indonesia', -0.08979770662474865), ('abdullah', 0.028352285796906112), ('core', 0.02507630104134011), ('riset', 0.02199445038300581), ('piter', 0.020656618889570124), ('kredibilitas', 0.019684872623557587), ('bukti', 0.017794944308717444), ('direktur', 0.016548803397633417), ('bijak', 0.016098807023808018), ('taxamnesty', -0.014297525082457225), ('jilidii', -0.010455659347262981), ('turun', 0.008863776252318208)], [('ya', 0.012463112353420342), ('plinplan', 0.01051019466800281), ('jelek', -0.004988123531552358)], [('aja', 0.037956558636126486), ('harta', -0.033215077127008816), ('pajak', 0.025738144046881187), ('pejabrutnya', 0.0229613474478139), ('revenue', 0.021375306969439248), ('diumpetin', 0.02106410764778472), ('ga', -0.01760356902702375), ('pas', 0.017025027244533107), ('negri', 0.01634949201863115), ('gimana', 0.014654578452580686), ('tau', 0.014545821099899825), ('tax', 0.012371917571538885), ('bayar', -0.012036001544333009)], [('taxamnesty', -0.22876959215731882), ('kemplang', 0.13275354654678054), ('pajak', -0.059686798464658394), ('jumatberkah', -0.03996044484699502)], [('jokowi', 0.021445214707749547), ('jenius', -0.0178042836029858), ('plinplan', 0.016540680734692503), ('bijak', 0.012712745469029422), ('kereta', 0.011988268386025241), ('cepat', 0.011966583485526814), ('apbn', 0.009265054934911891), ('plongo', 0.008396272753390397), ('kerap', 0.008081839169592295), ('linglung', 0.007879347053909008), ('jilidii', -0.007672402414596296), ('planga', 0.006817968070669893), ('kali', 0.006390843557771175), ('alias', 0.006287464258749317), ('pakai', 0.004181884405566204), ('orang', -0.003651025988828432), ('taxamnesty', 0.0003257893907481086)], [('tuju', -0.07704119205687249), ('kemplang', 0.06083985137014785), ('ampun', 0.02394462455741207), ('jokowi', 0.022083918753982107), ('direktur', 0.019220843495176713), ('riset', 0.006363286602990123), ('eksekusi', 0.006030194117321451), ('co', 0.0035291945210011223), ('pajak', 0.0025260701282543443)], [('tuju', -0.08593816797811536), ('kemplang', 0.06402639867692825), ('ampun', 0.02764646247010405), ('jokowi', 0.023208179487291463), ('direktur', 0.021316393724795164), ('eksekusi', 0.007826182229087545), ('pajak', -0.002341743256251226)], [('indonesia', -0.07895700058030308), ('abdullah', 0.03141502309655978), ('core', 0.02636227567859607), ('perintah', -0.0249398440866436), ('piter', 0.02488802586108628), ('riset', 0.024027395110095173), ('kredibilitas', 0.021874848678455617), ('bukti', 0.019935322236257397), ('direktur', 0.01512395537173768), ('turun', 0.014956389453370635), ('taxamnesty', -0.01388071108257887), ('jilidii', -0.008736486510243358), ('bijak', 0.0023842840601557835)], [('jokowi', 0.012481986948246225), ('cepat', 0.012147257325748782), ('kereta', 0.007161206543261785), ('apbn', 0.006974232870086734), ('jilidii', -0.0058954729832287845), ('bangun', -0.005136434485595299), ('bijak', 0.003981928519125076), ('pakai', 0.0027776015479637163), ('plinplan', 0.0020579033761818524), ('nggak', -0.0019293519607157014), ('buzzer', 0.001445099754079651), ('nkri', 0.0013162954321547422), ('apbd', 0.0011446182247552559), ('inget', 0.001099400077808553), ('lol', 0.0010799664858493385), ('taxamnesty', 0.0008981039723794989), ('bangga', 0.00078464980683982), ('banggain', 0.0007511494386743832)], [('urus', -0.05548852110305712), ('rakyat', 0.052661975778395036), ('taxamnesty', -0.02717654326762503), ('exploitation', 0.019466392407526047), ('tuntas', 0.019273973698416647), ('kelompok', 0.018163145112219743), ('tax', 0.015490581438916243)], [('taxamnesty', -0.16925398517912604), ('rakyat', 0.07302297116201559), ('periode', 0.025419516746665906), ('ulang', 0.022437257861221867), ('kelompok', 0.021653621219241106), ('tententu', 0.020380391827231104), ('nggak', 0.018956485171701155), ('pajak', -0.01570924616242331), ('untung', 0.008219687226263699), ('negara', 0.005679232590077041), ('bebas', -0.005134286988439793)], [('jenius', -0.35798112105114793), ('service', 0.07171408370251245), ('lip', 0.05842468986740459)], [('aja', 0.037938216056685646), ('harta', -0.03773122297183909), ('pajak', 0.023840089656274134), ('taxamnesty', -0.022034916587618202), ('revenue', 0.013044629060108209), ('pejabrutnya', 0.012480005200805739), ('ehhh', 0.010897034112913422), ('langsung', 0.01077226289717748), ('bayar', -0.010575253963402417), ('gimana', 0.010572620686061535), ('diumpetin', 0.01028839794909033), ('ga', -0.010200298554328047), ('diadain', 0.009893686068209038), ('negri', 0.008905923986861118), ('pas', 0.007572154299439925), ('tau', 0.006899181776475284), ('tax', 0.00659546319712213)], [('ubah', -0.06326526531391666), ('sulit', -0.03326119121989768), ('dusta', -0.010380402979752045), ('inkonsistensi', 0.004046040882662148), ('jokowibohonglagi', 0.002345708263902502), ('darah', -0.0005538382745259155), ('daging', -0.0003818257009318121)], [('taxamnesty', -0.05100499128830319), ('pingin', 0.04987385780152989), ('mboh', 0.047917420784992726), ('itunganya', 0.047816692720714093), ('gimana', 0.04539337496576611), ('ppn', -0.04369630082833899), ('genjot', 0.03797392869369965), ('u', -0.002915618838685445), ('pajak', 0.0005606589214417618)], [('mata', -0.07717289070192375), ('kah', 0.00415997348445388), ('buta', 0.0019513990268112326), ('banget', 0.00038027515544298917), ('aduh', 0.00020336380862676036), ('dewa', 0.00017147670381903888), ('beliau', 0.0001681463471224309), ('hati', 0.00016272411184082598), ('slalu', 0.00015272489046199152), ('jokowipresidenjenius', 0.00013662972440395534), ('junjung', 0.00012395447166828585), ('pikir', 8.951363726716476e-05), ('kasihan', 4.765784576013149e-05)], [('lucu', 0.029609550804982343), ('ujung', 0.026936817688362258), ('judul', 0.025894857553063214), ('ngejar', 0.025076266761893607), ('emg', 0.024230777957315288), ('taxamnesty', -0.020290883781684838)], [('ya', 0.052445349880288476), ('sesuai', 0.037191805862524074), ('salah', -0.03440209284909106), ('taxamnesty', -0.03099523620745298), ('target', 0.01967338570101946), ('ga', 0.00669869162609621)], [('pajak', -0.0248927638128037), ('rakyat', 0.02392980579316903), ('kemplang', 0.020885890413747124), ('taxamnesty', -0.016038596736903207), ('yah', 0.008409103666680766), ('royalty', 0.00802635171864925), ('rugi', 0.0075223323285163075), ('buru', 0.006022180172645693), ('hapus', 0.0055866713024007725), ('tega', 0.005215951329447646), ('bara', 0.005074659825897698), ('rezim', 0.004629598073409048), ('batu', 0.004178015730354885), ('negara', -0.0037581730209305993), ('banget', 0.0036909237871160096)], [('jilid', -0.31386179942525033), ('janji', -0.27082023784357134), ('taxamnesty', -0.2697276362329758)], [('kemplang', 0.02518714019038273), ('pajak', -0.022711455040105313), ('rakyat', 0.021430173834150027), ('gila', 0.01618367197863746), ('taxamnesty', -0.01360660586837723), ('ya', 0.010657950339136378), ('trus', 0.009912025708127463), ('lho', 0.008329738464905846), ('tagih', 0.00813929668283004), ('amnesty', 0.006146491756962471)], [('jokowi', 0.020186258427404832), ('kereta', 0.0185895913658609), ('cepat', 0.017330981412100842), ('jilidii', -0.014737889593219403), ('ooo', 0.0118561526746605), ('apbn', 0.011673599748286112), ('ooohhh', 0.011469991349018015), ('plinplan', 0.010298959272998731), ('taxamnesty', -0.009911874813133834), ('pakai', 0.007914200548417316), ('bijak', 0.004841575182396343)], [('apbn', 0.011923970964769555), ('thn', 0.003939146797049957), ('defisit', 0.003937523347027373)], [('taxamnesty', -0.19923053899516024), ('jilidii', -0.011984697071526175), ('efektivitas', 0.0007951610571490351)], [('rakyat', 0.11528702016661321), ('peras', 0.09838620887576326), ('ii', 0.04187620976665835), ('taxamnesty', -0.04028964257848201), ('januari', 0.039840876195942926), ('jilid', -0.03506524761309528), ('beda', 0.034337751056671785), ('pajak', -0.009537393963730838)], [('naik', -0.030551064595594358), ('sri', -0.027626961447978292), ('uang', 0.01671979786520527), ('pandora', 0.016246888169663826), ('papers', 0.01260813830177639), ('taxamnesty', 0.010912853724325312), ('gimana', 0.010908713656966093), ('lumayan', -0.009810687374181665), ('jabat', 0.009658760639397797), ('ehh', 0.006388926572728002), ('mboh', 0.004448776462569792), ('bal', 0.0036857667804488267), ('negara', 0.003625314469672118), ('ndang', 0.0032456016455053847), ('mbangun', 0.003208974688170175), ('itunganya', 0.0024598933592234933), ('ppn', -0.0024381518993459575), ('papa', 0.0023765604258906754), ('berita', 0.0017664197631089484), ('yo', 0.0016306872272405792), ('pingin', 0.0015054898844412035), ('menteri', -0.0014993946268662451), ('genjot', -0.001232253811632777), ('pajak', 0.0006416645874629831), ('bu', 0.0005043137437648245), ('g', -0.00018789899845745006)], [('kemplang', 0.02659545415578231), ('kaya', 0.01563974942438235), ('orang', -0.00915099012128262), ('pajak', 0.00856665309585297), ('sanksi', 0.007627515467097406), ('msd', 0.0070712972552086865), ('pidana', 0.006812307342676258), ('hilang', 0.0038731575882863635), ('untung', -0.003135308052996214), ('uu', -0.00017401760487664419)], [('rakyat', 0.07022872902870896), ('peras', 0.06657346436369778), ('taxamnesty', -0.04585997299806932), ('skema', 0.04089648850763662), ('pajak', -0.024774951205245296), ('bijak', 0.02291419610257758), ('jilidii', -0.013397271980411922)], [('rakyat', 0.08067192942862061), ('peras', 0.06600247678197012), ('taxamnesty', -0.0447808213357933), ('januari', 0.030853902281552877), ('ii', 0.027759950591123263), ('pajak', -0.025646511470431534), ('beda', 0.025307970966900246)], [('rakyat', 0.05232443209808154), ('temen', -0.05104012294001975), ('cekek', 0.003598244469092515), ('dibebasin', 0.0014711284735071173)], [('rakyat', 0.0548259576470086), ('incar', -0.030511047119627537), ('spt', 0.01660703015760801), ('npwp', -0.01343196597367638), ('ktp', -0.010365619732822809), ('lapor', 0.008928573825967746), ('denda', 0.008055323796476655), ('ganjar', 0.004822668677771405), ('gak', 0.0041193244948570604), ('tuh', 0.0036301885686722854), ('tahun', 0.003574046160480043), ('perintah', -0.0034298539493286726), ('tengah', 0.003261824234718388), ('tambah', 0.0031421826685259072), ('th', 0.0029835324204204893), ('galak', 0.00279315589510476), ('rb', 0.0001808571896902698), ('x', -0.00016049556874054012), ('taxamnesty', -4.1951210115452244e-05)], [('kaya', 0.028523847101413154), ('pajak', 0.021150349695455806), ('kdrun', 0.01130918215853387), ('sok', 0.01122592138782324), ('orang', -0.010180666801400163), ('lu', 0.00961776938037424), ('kecuali', 0.009163039176510302)], [('kemplang', 0.04139667732054429), ('kaya', 0.0232427430484559), ('harta', -0.022278226307293866), ('taxamnesty', -0.019017633520911265), ('pajak', -0.011580446015976277), ('pidana', 0.011197093863856866), ('sanksi', 0.0074615988688770335), ('tiga', 0.007087193344859637), ('aku', 0.006685568190344424), ('kurang', 0.006422267279819583), ('sukarela', 0.005853645212534277), ('orang', 0.005825010687589374), ('tunggak', 0.005777134913297684), ('denda', 0.0054408077616129626), ('msd', 0.004737451869879514), ('hilang', 0.004027444841634907), ('untung', -0.0030173962592059023), ('uu', 0.000573009235465046)], [('sektor', -0.048413788373709216), ('tambang', 0.03852744596240482), ('uang', 0.03294535684029058), ('untung', 0.01925561976287561), ('taxamnesty', -0.01867254596170821), ('usaha', 0.01356519964303476), ('jilidii', -0.006961906709987671)], [('runding', 0.11882041908719308), ('taxamnesty', -0.09495998090536592), ('maling', 0.04162457702197438), ('ii', 0.03299677126111278)], [('rakyat', 0.02736314451903929), ('dap', 0.02423609727595493), ('taxamnesty', -0.022728002397666227), ('wow', 0.02150060082311002), ('kena', -0.021043883358414853), ('taipan', 0.01864999861670419), ('negri', 0.018118507398282895), ('makmur', 0.018087863558241928), ('jelata', -0.015841759963518307), ('pajak', -0.0052032296478661715)], [('taxamnesty', -0.16253444301912848), ('sakit', 0.1508845880294681), ('bayar', -0.13310267007935936), ('hati', 0.13260638390772778), ('bikin', 0.09727586499635625), ('pajak', -0.05362085756460523)], [('nik', -0.07149785026090294), ('npwp', -0.04296764381367517), ('cepat', 0.03775511448655919), ('taxamnesty', 0.02795451791374396), ('cukong', 0.025936532237991423), ('patut', 0.022357893718887818), ('dana', -0.019623857197107204), ('segar', 0.01780684137320048), ('sibuk', 0.01449609458762575), ('cina', 0.013788313993486132), ('cari', 0.013776621905236009), ('jdi', 0.012897778873777873), ('ubah', 0.00035814986057328564), ('n', 0.00018672804406839698)], [('tugas', -0.05784040721986282), ('pajak', -0.04618280814980491), ('salah', -0.03632287744846659), ('curang', 0.020564949347928897), ('om', 0.020229568914201003), ('kerjasama', 0.018813543750752313), ('nama', 0.018803283191196885), ('ngemplang', 0.018645273268347876), ('modus', 0.017271996184621826), ('satu', 0.014251554909134583), ('lapor', 0.013554353050176004), ('mesti', 0.01028930928550489), ('taxamnesty', -0.0045853246653272395)], [('uuhpp', -0.22648301599650103), ('kemplang', 0.07297330791207703), ('harta', -0.01974699235385928), ('pidana', 0.015727291003702955), ('jokowi', 0.012412758107192683), ('bikin', 0.010019193844776932), ('kuasa', -0.00878632848541493), ('hilang', 0.006953874727869904), ('bijak', -0.00533192332516848), ('harga', -0.004420410054546851), ('pajak', 0.0026248940466694526), ('taxamnesty', -0.0019667110880191457), ('nkri', -0.0019160038656628113), ('gimana', 0.0018944167396007163), ('pancasila', -0.0018245694239923066), ('mega', 0.0016860233264494031), ('gak', 0.0014639284925418673), ('senang', 0.001439563632740627), ('maling', 0.0014292368836290785), ('blbi', 0.0011323271357136888), ('pro', 0.000877396147875993), ('iming', -0.0007750527976753053), ('ngomong', 0.0006893895739936237), ('subsidi', -0.0005060510353271084), ('pdip', -0.00031636658515759404), ('jaman', -0.00028988029094087313), ('sangsi', -9.122313748101774e-05), ('mati', -8.240218262079909e-05), ('sukarela', -1.8167634548827099e-06)], [('kemplang', 0.1484548055410013), ('jerat', 0.08407795832287855), ('hukum', 0.07835111007620942), ('bebas', -0.061615168829449785), ('taxamnesty', -0.0345602110211326), ('jilidii', 0.028397207929508828), ('pajak', -0.0027494362076403264)], [('tentu', -0.18191368393969823), ('bayar', -0.09677329280543437), ('syarat', -0.08832932257619854), ('hasil', -0.08475335647204851), ('pengaruh', -0.07349688751077188), ('laku', -0.06552463803247033), ('alesan', -0.0644553906248515), ('taxamnesty', -0.06272642276052671), ('bunga', -0.049040647421685575), ('pajak', -0.04871485279723931), ('tiada', -0.02939646820731179), ('negara', -0.025305184151366584), ('ringan', -0.014667364632279583), ('usaha', -0.012687427762775231), ('nya', 0.003135910451385524)], [('taxamnesty', -0.12858834639672528), ('ampun', 0.08193818605628873), ('aju', 0.08183303631658384), ('pajak', -0.04570962789816402), ('hindar', 0.02577299396781647)], [('buruk', 0.022349535326506905), ('ebt', 0.020738604865507677), ('bisnis', 0.020115940743679818), ('ekonom', 0.014915822534104898), ('taxamnesty', -0.013854476746125995), ('ii', 0.011879832237692602), ('dampak', 0.006543786102810733)], [('tuju', -0.09858977605797736), ('ruuhpp', -0.056652679809218974), ('yawlah', 0.04037108244577194), ('jugaa', 0.038989905744209366), ('nggk', 0.03732357801672841), ('tidur', 0.035040853185766205), ('sah', 0.03409018417850456), ('mah', 0.02251918749242718), ('program', -0.004398067108079393), ('taxamnesty', -0.0008952085872185853)], [('ri', -0.22606441613334569), ('data', -0.16771274842097092), ('sosialisasi', -0.0865177966900147), ('blok', -0.047654675983320204), ('perintah', -0.03927918066179984), ('ln', -0.03919537091474586), ('triliun', -0.038833726086786466), ('aset', -0.02822342749398665), ('jokowi', 0.026419438619862033), ('ta', -0.02612452068777323), ('rb', -0.0259609715623298), ('elu', -0.024006428951567994), ('taxamnesty', -0.02366885343824984), ('bodoh', -0.023329144029148888), ('paham', -0.023299905664731373), ('wni', -0.021340491874175145), ('problem', -0.02092323750799735), ('duit', -0.01584391747216713), ('masuk', -0.0025384851492117563), ('t', 0.0008799939571716112), ('kalo', -0.0002366410527240013)], [('taxamnesty', -0.11778677075102592), ('pandora', 0.0904534626118605), ('paper', 0.061126261047141126), ('ampun', 0.048751636809295146), ('pajak', -0.02823674412627896), ('hindar', -0.021641835655351778), ('hubung', -0.0067505071523008735)], [('maling', 0.00878435204495234), ('runding', 0.0007970905903596695)], [('paja', -0.12910934450385864), ('pks', 0.06046468336415724), ('tolak', 0.04587920485786395), ('voluntary', 0.02762955202099305), ('disclosure', 0.02394444128686985), ('asset', 0.019889704672599432), ('atur', -0.01893099784271086), ('ruu', 0.01631593189420865), ('kup', 0.015163644682143323), ('hpp', -0.006764458189618225), ('harmonisasi', -0.0023604594264599837)], [('moga', -0.06137830047377804), ('tsb', 0.03099831601035128), ('transaksi', 0.03030712472993885), ('hasil', -0.022244009403349644), ('bersih', 0.019780527675441095), ('taxamnesty', -0.002150480443227335)], [('wamenkeu', -0.22546453136078695), ('klaim', -0.17522856153680502), ('proses', -0.14544148200852383), ('sederhana', -0.1417960295027371), ('taxamnesty', -0.09930339720066496), ('jilidii', -0.07559891261654991), ('jumatberkah', -0.03039427023864562)], [('kemplang', 0.0800560144867775), ('taxamnesty', -0.053669973955615236), ('karpet', 0.05227468984490065), ('merah', 0.04915896410772783), ('nilai', 0.04510723801814917), ('jilidii', 0.011705582274376677), ('pajak', -0.005712759607655996)], [('kemplang', 0.10759498367756166), ('djp', 0.05697274882212685), ('ngasih', 0.05584721094690628), ('data', -0.0523818689928575), ('taxamnesty', -0.04471365293581105), ('pajak', -0.0133006068342206)], [('umkm', -0.3399396164519284), ('infustrigula', -0.09569479331741337), ('tingkat', -0.09232600459007033), ('patuh', -0.08281757930501171), ('upayapemerintah', -0.05396527081092396), ('taxamnesty', -0.050428268308036275), ('program', -0.04532441357932257), ('pajak', -0.0329356468181277)], [('taxamnesty', -0.36021009602569454), ('bayar', -0.23169929541702888), ('pajak', -0.11172914916944576)], [('sri', -0.06940278281153774), ('ayo', -0.05137653825230902), ('jeng', 0.025369671620619816), ('piye', 0.02307008100180215), ('taxamnesty', -0.02181487842946397), ('manja', 0.016170560742859302), ('cekik', 0.011847162854439505)], [('wajib', -0.17625892922833336), ('tuju', -0.11305768705756515), ('taxamnesty', -0.10132486757482866), ('tingkat', -0.05967544534281246), ('pajak', -0.02343116327206826), ('sukarela', 0.017866283336339904), ('program', -0.017795606201619043), ('mulyani', -0.01714231520078736), ('sri', -0.013606770541100325)], [('mulus', 0.0633367242236198), ('jalan', -0.05609232866455301), ('taxamnesty', -0.0456718402271069), ('sih', 0.03940964230220504), ('ii', 0.03873689095287873), ('jahat', 0.036256239694417054), ('pajak', -0.02931349921283346)], [('donk', 0.10046717257443517), ('batal', 0.07796828853206918), ('bebas', -0.07482890802761678), ('uber', 0.06714647005512549), ('ngemplang', 0.06103536473544415), ('pajak', 0.027288232276215064), ('taxamnesty', 0.007343867069732101), ('orang', -0.004869294883720439)], [('kemplang', 0.1051292800915889), ('uuhpp', -0.08600773701808534), ('tuju', -0.03662888026582795), ('kaya', 0.03611795215449797), ('pidana', 0.02453414531336401), ('sanksi', 0.018797678404914208), ('ampun', 0.017272698377494903), ('taxamnesty', -0.015278137296076178), ('dpr', 0.015196516381539307), ('hilang', 0.014693696912053526), ('atur', 0.013827963773140406), ('orang', 0.012643251902587705), ('harmonisasi', 0.012554071282023257), ('harta', -0.01126703592437955), ('uu', 0.011117244684058742), ('ringan', 0.009667870042432221), ('denda', 0.008702701006961975), ('aku', 0.005634339467558781), ('pajak', -0.004633981211955049), ('perintah', 0.004323371093427924), ('sukarela', 0.00392482809998597), ('duberikan', 0.003317389472636142), ('tunggak', 0.002475868982333627), ('kurang', 0.0021008260961024134)], [('aja', 0.01929287231321335), ('bilang', -0.016576869994740583), ('negara', -0.012393908077988944), ('kas', 0.005806744150776233), ('kosong', 0.0019509059067643645)], [('jokowi', 0.015903935385944205), ('eh', 0.014442162812740208), ('mutaroe', 0.01141919561864339), ('kayak', 0.010586382150952546), ('lagu', 0.0105173659243702), ('bugis', 0.010234731852886989), ('januari', 0.010229434285558512), ('janci', 0.010180377621418032), ('janji', -0.007544153436737754), ('taxamnesty', -0.004097952282310261)], [('dorong', -0.09452056633412546), ('kuat', -0.09350335709965137), ('wacana', 0.045636706345373895), ('gulir', 0.041532596099784304), ('ekonom', 0.034793278449019756), ('duga', 0.0340196508739443), ('taxamnesty', -0.03324479942200182), ('in', 0.03123492654220012), ('orang', 0.023746046682536433), ('program', 0.01827967379767805), ('jilidii', 0.015004601097384334)], [('ruuhpp', -0.14040229851039895), ('pks', 0.039054251627964595), ('demokrat', 0.03854800175071104), ('oposisi', 0.030882118885345852), ('kompak', 0.030131182167655663), ('tolak', 0.024070146203569124), ('dukung', 0.01288089045759754)], [('kemplang', 0.13742609004819717), ('bebas', -0.05524581297574314), ('kriminal', 0.047594715548639935), ('wajib', -0.04320832435187186), ('pidana', 0.03536278675835504), ('pajak', 0.025968900032232727), ('s', -0.007023285146894815), ('bayar', -0.0042660984668234396)], [('kemplang', 0.13340973426825684), ('jokowi', 0.061090759965365136), ('kriminal', 0.04114263106841175), ('pidana', 0.03841358231932131), ('wajib', -0.03680605925297168), ('taxamnesty', -0.03463600726886836), ('pajak', 0.023674191827568976), ('gelar', 0.011722828531612195), ('mulyani', -0.009103415008570783), ('sri', -0.008156244368193743), ('bayar', 0.007796648368110948), ('bebas', 0.002746709644929917)], [('rp', -0.2857146097919172), ('raup', -0.16597808545367065), ('taxamnesty', -0.14186158293232284), ('ppn', -0.09102121560003501), ('menkeu', -0.07728312300178036), ('triliun', -0.05228200027772223)], [('utang', 0.0506411095888707), ('ya', 0.038603933163381045), ('pro', 0.03737410508938961), ('ujung', 0.03685819224400367), ('pph', -0.035706064164079815), ('bayar', -0.033719497127076566), ('badan', -0.03315741170766672), ('bijak', 0.023133396392646922), ('turun', 0.011882505514022638), ('taxamnesty', -0.0009048069622018533)], [('rp', -0.28260668865694055), ('raup', -0.16150484320817182), ('taxamnesty', -0.13641040518817776), ('ppn', -0.0931414440410785), ('sri', -0.06317626719821548), ('mulyani', -0.05744184391177806), ('t', 0.010954804230499516)], [('dorong', -0.08155848768865415), ('kuat', -0.0761384647475241), ('gulir', 0.04637197831421849), ('wacana', 0.04547411116756357), ('duga', 0.04187091894051855), ('ekonom', 0.03685322736542918), ('program', 0.025887061559443838), ('mulus', 0.025441028953883588), ('orang', 0.023469863817681503), ('taxamnesty', -0.022372939604934743), ('jalan', -0.020149807380832517), ('jilidii', 0.019034276091287477)], [('jokowi', 0.03987090712351421), ('ampun', 0.03787609091202748), ('nilai', 0.03377780876545725), ('ingkar', 0.030460491904220253), ('taxamnesty', -0.030129121052561447), ('ekonom', 0.027144615494698602), ('presiden', 0.02705207209339865), ('program', 0.014078914219704943), ('janji', -0.008885321626611306), ('pajak', 0.004499870063656823), ('jilidii', 0.002327244843101488)], [('paja', -0.2220550359559823), ('terap', -0.13497427198213727), ('memperole', -0.08225878282751098), ('atur', -0.07478482859899804), ('perintah', -0.06560071409206741), ('hpp', -0.060064971664660995), ('harmonisasi', -0.04276713088441188), ('negara', -0.03491693996931558), ('uu', -0.023642143940946464)], [('paja', -0.15561435509667085), ('terap', -0.1136682286038743), ('rp', -0.08017124263383228), ('tambah', 0.05007052158046635), ('atur', -0.0403977336654984), ('hpp', -0.018990400840262947), ('harmonisasi', -0.017229594695849042), ('oleh', 0.017032112586259963), ('perintah', -0.012532119831936538), ('triliun', -0.005404238126345367), ('uu', -0.00402660133275698), ('negara', 0.003109736921971163), ('dapat', 0.0014228518713725361)], [('rakyat', 0.07877083858806876), ('indonesia', -0.06586926048630798), ('dana', -0.024560285397774245), ('kasihan', 0.01820821285096688), ('oknum', 0.017427751077832097), ('panjang', 0.01622591154090349), ('senang', 0.013375109432839153), ('habis', 0.013337414257514071), ('korupsi', 0.011806161400854167), ('covid', -0.010720664333304418), ('ppkm', 0.009420513801291566), ('pajak', -0.0019592214020462827), ('bayar', 0.0015732843369251216)], [('adil', -0.11091386518771643), ('tolak', 0.037590503506029514), ('salah', -0.03195845688166558), ('pks', 0.02617491677340249), ('satu', 0.018311559953436262), ('taxamnesty', 0.01765987345117501), ('rencana', 0.01693995065952033), ('kait', 0.016375729184800993), ('fraksi', 0.015963662567749993), ('ruu', 0.01553263086818841), ('sisi', 0.014567382264811711), ('sejahtera', 0.013253101697572121), ('muncul', 0.012579331934355729), ('naik', -0.011844807103806828), ('ppn', 0.005947211142241941), ('partai', 0.001986258278835806)], [('taxamnesty', -0.2206963505874191), ('program', -0.026830144609207233), ('ya', 0.024163595385743523), ('gagal', -0.01531032955070293)], [('wajib', -0.23937953454649027), ('data', -0.1865075712818631), ('atur', -0.05309702226125379), ('harta', -0.0526440831666605), ('informasi', -0.042441161287295526), ('taxamnesty', -0.041328350038264364), ('nyata', -0.03770873237908124), ('jenderal', -0.03164707716489966), ('surat', -0.0258399998475047), ('pajak', -0.02535076947087364), ('bersih', -0.022386659181885177), ('temu', -0.02135241838039546), ('sempat', -0.01632231775607279), ('direktur', -0.011704927477170965)], [('salah', -0.13981749711166763), ('patuh', -0.09664357742276973), ('mulyani', -0.09522844317245428), ('incar', -0.0862493664411091), ('sri', -0.07466070020816168), ('taxamnesty', -0.06677150953209575), ('gelar', -0.0642131604606488), ('jilidii', -0.035285954958177136), ('usaha', -0.02861681308331347), ('pajak', -0.016882401903004166)], [('banning', 0.057206648850033875), ('shell', 0.054559819730926044), ('companies', 0.054321436936530625), ('jiah', 0.047370295280073534), ('taxamnesty', -0.024060571482206937)], [('pandora', 0.013648311443157436), ('jabat', 0.01260478032416647), ('belo', 0.00861826237420273), ('bom', 0.008617921663294644), ('taxamnesty', -0.008477785648017443), ('paper', 0.0065163121188563196), ('nama', 0.005514086569935544), ('dipajakin', 0.005329942566529554), ('kasih', 0.0047699802698049415), ('gede', 0.003686648128012607)], [('jokowi', 0.05012265437667668), ('ampun', 0.0494995270686498), ('nilai', 0.046906905349729036), ('ingkar', 0.04581072916189299), ('taxamnesty', -0.04243673080244691), ('presiden', 0.04054199121522498), ('program', 0.023563533275409314), ('jilidii', 0.009952965496683114), ('janji', -0.008559826475166479), ('pajak', 0.0017702399149521835)], [('taxamnesty', -0.10084161254745035), ('pks', 0.06721667358569094), ('tolak', 0.046244008381041486), ('tarif', -0.014117796144723091), ('ppn', -0.009881369375273664), ('jilidii', -0.002564798022007649)], [('jilidii', -0.03434450247594228), ('prediksi', 0.02969778607168394), ('minat', 0.02659682167393508), ('taxamnesty', -0.025880936816862266), ('perintah', -0.023044469660913496), ('sepi', 0.02297237731708095), ('kredibilitas', 0.019944758392400525), ('turun', 0.004801755250973074)], [('negeri', -0.01413636903062276), ('kah', 0.008720519267037203), ('cabut', 0.007358318937241602), ('sayap', 0.006491383487341413), ('taxamnesty', 0.005997365042551841), ('absurd', 0.00542314819424914), ('msh', 0.005323127344069685), ('limut', 0.005131831410459493), ('narkoba', 0.005127476089080308), ('raja', 0.004801062449090535), ('malaikat', 0.004731960252711819), ('mandat', 0.004524765189069637), ('sindikat', 0.0041173387101495155)], [('jilidii', -0.0436176639519087), ('prediksi', 0.04148187691764885), ('duh', 0.04133830571666225), ('sepi', 0.03979088533727616), ('minat', 0.039726646196030696), ('taxamnesty', -0.03854975919123694)], [('melipatgandakan', -0.08038570574135157), ('skala', -0.07938956920758113), ('dst', -0.07810724348082179), ('melipargandakan', -0.0746478883311577), ('dilipatgandakan', -0.07206823561598386), ('baik', -0.06867913125414236), ('lembaga', -0.060355860400268274), ('taxamnesty', -0.05811864544474174), ('orientasi', -0.05769409554246587), ('keburu', -0.050756021603800175), ('islamphobia', -0.0484675701412681), ('konglo', -0.04213255680046081), ('partai', -0.03621357260066547), ('jelek', -0.021796574495196137), ('kuasa', -0.019478413980914443), ('jahat', -0.01115121284438438), ('negara', -0.009216785696607346), ('kalo', -0.000593470181347049)], [('jokowi', 0.05291603167965863), ('nilai', 0.04700256804825528), ('ampun', 0.04678960334898434), ('ingkar', 0.046031299500430216), ('taxamnesty', -0.044801154235386456), ('presiden', 0.042421947682826165), ('program', 0.026934664953874247), ('janji', -0.011397894234079546), ('jilidii', 0.01070315111575321), ('pajak', 6.570372557207745e-05)], [('kemplang', 0.1003151960260144), ('taxamnesty', -0.09773159393777406), ('bahagia', 0.07789933704393749), ('bikin', 0.062280883639866245), ('pajak', -0.019910730906018716), ('jilidii', 0.016047377184065483)], [('wajib', -0.1915157021773315), ('tingkat', -0.08218055176937125), ('ampun', 0.07509549368305841), ('patuh', -0.06604706841300928), ('harap', -0.047742604350497674), ('terap', -0.03127065321424043), ('taxamnesty', -0.026633711969433448), ('jilidii', 0.015385042946290851), ('pajak', 0.006492222140095668), ('januari', 0.004954976103348026), ('juni', 0.0028128569599904733)], [('taxamnesty', -0.019618231324541564), ('pks', 0.018567807006818295), ('ampun', 0.016358512709047795), ('tolak', 0.013778382545369231), ('ahli', 0.012977693008588096), ('voluntary', 0.012944193140862453), ('asset', 0.012171200254308782), ('publik', 0.010584732144285456), ('disclosure', 0.010267006672210657)], [('wajib', -0.23975969575693568), ('tuju', -0.12405306193120272), ('patuh', -0.12236930611230872), ('tingkat', -0.10342179170809512), ('program', -0.039960139431481204), ('pajak', -0.020012478076824977), ('menteri', -0.009950451513654927), ('yasonna', -0.0025465972730810958), ('ham', -0.0013926089823827752), ('laoly', -0.001156861977814418), ('sukarela', -0.0007269912252332805), ('hukum', -3.3896434020479694e-05)], [('produk', 0.0747310314979775), ('pesan', 0.07308441308916509), ('taxamnesty', -0.06603840652495234), ('salah', -0.021072059949590223)], [('taxamnesty', -0.1777197541489519), ('kaya', 0.11694020154384223), ('hehehe', 0.0590833898446515), ('pajak', -0.05212057275816972), ('orang', 0.01735109564972065)], [('laku', -0.041730206257458075), ('pandora', 0.014750925733394786), ('pilih', -0.014475199012657161), ('badut', 0.007086420833714222), ('konsekuensi', 0.006629401645275762), ('lenggang', 0.006550331631378268), ('golong', 0.006441449330484972), ('mem', 0.006335166480299936), ('ber', 0.005580732108080386), ('kangkung', 0.0035873838966068773), ('plot', 0.0033011139570603134), ('apa', 0.0031969430550055095), ('push', 0.0030732627920384277), ('twist', 0.002314275285908308), ('taxamnesty', -0.0012880918416086917)], [('halal', 0.03425484816004947), ('berkah', 0.03390183526526343), ('taxamnesty', -0.030905571207447868), ('masuk', 0.028053920124596867), ('pandora', 0.027856817348332626), ('bikin', 0.02500470130409178), ('uang', 0.021421512591336657)], [('rp', -0.12583994588697658), ('uang', 0.028563121266096593), ('triliun', -0.021007202508372014), ('transaksi', 0.016416069022320255), ('kait', 0.013975278342540682), ('milik', -0.01326576738621904), ('jumbo', 0.012988494856065855), ('pajak', 0.011745179636659553), ('taxamnesty', -0.010853369291347155), ('sindikat', 0.009061547087039865), ('kapolri', 0.008361437274123213), ('wow', 0.008353215130757468), ('ikut', 0.007639955785377313), ('rekening', 0.007464957165746184), ('narkoba', 0.007070959089554742), ('bayar', -0.006818802799026604), ('cilakanya', 0.006662865653545196), ('mesti', 0.005673925981943073), ('sidik', -0.005324758106555895), ('panggil', 0.004795915952320161), ('temu', -0.0011389262488401904)], [('kaya', 0.043394782485046565), ('ampun', 0.035102878322640696), ('bareng', 0.030493315778634383), ('lantas', 0.02531250041121126), ('super', 0.021413662605266785), ('pajak', 0.020619257234574653), ('tarif', -0.01967107829319699), ('kena', -0.018074716052072996), ('taxamnesty', -0.015256950000955578), ('orang', 0.006864986456235581), ('atur', 0.0003414780909986873)], [('anda', 0.09677621603682454), ('imbang', -0.08280132265053539), ('moga', -0.040531203153444634), ('dua', 0.025596007218172245), ('sanksi', 0.024939077540887433), ('gagal', 0.020147970648195717), ('negara', 0.016816934765035384), ('taxamnesty', -0.005285161925567815)], [('rakyat', 0.0785031041789902), ('nyembunyiin', 0.0654798400003232), ('lho', 0.05805679094109887), ('taxamnesty', -0.0571321171238247), ('aja', 0.05616928520422708), ('yah', 0.05465233281351826), ('bayar', -0.045143428787695186), ('pajak', -0.030733551943898416), ('gak', 0.007840318133870715)], [('masyarakat', -0.19071361197973558), ('ruuhpp', -0.1785356809529008), ('gegara', -0.1572558940249866), ('taxamnesty', -0.14398547317053378), ('ngumpetin', -0.08310589237039358), ('harta', -0.06884354987086903), ('tolak', -0.015481095920215092), ('ya', 0.003378111999592631)], [('jilid', -0.055167602980007356), ('pantesan', 0.04345260040422623), ('no', 0.0395505928412877), ('otomatis', 0.03831996808595579), ('sembako', 0.03241034407623372), ('dipajakin', 0.029287486813844778), ('npwp', -0.023664949843425744), ('ktp', -0.015350638240623058), ('taxamnesty', 0.00890581974467843)], [('taxamnesty', -0.16504303016274913), ('pajak', -0.0593277942430879), ('gak', 0.04368700429451388), ('taat', -0.006422262625095388)], [('kemplang', 0.06820849328858543), ('rakyat', 0.062201490807072726), ('taxamnesty', -0.058139571701681644), ('cangkang', 0.045509639733865195), ('jabat', 0.03429224923452957), ('usaha', 0.02169541251138312), ('pajak', -0.019974652841306572), ('milik', 0.013996645344185597)], [('ruuhppuntukmasyarakatkecil', -0.220835785519631), ('taxamnesty', -0.21497267252118407), ('gegara', -0.13199185927868942), ('ngumpetin', -0.07737155373057611), ('harta', -0.0738077940505529), ('tolak', -0.022119357906417385), ('ya', 0.010391511494927142)], [('ruuhppuntukmasyarakatkecil', -0.36252262140866853), ('taxamnesty', -0.2540654508335007), ('taat', -0.06674547549648466), ('pajak', -0.062453214134949024)], [('ruuhppuntukmasyarakatkecil', -0.2913452073123394), ('nolak', -0.10720435428926096), ('taxamnesty', -0.10255310920596754), ('harta', -0.07544305165455735), ('ngapain', -0.07184671469742002), ('bela', -0.056379854082811806), ('sembunyi', -0.05637663115860769), ('ga', -0.04587788243959513), ('duit', -0.02272250306618753)], [('investasi', -0.16568191967521873), ('sbn', -0.10771018852338343), ('taxamnesty', -0.08742331790966), ('jilidii', -0.04757423475441967), ('potensial', 0.02922260856575629), ('tingkat', -0.02804384752329711)], [('wajib', -0.23822057209916492), ('basis', -0.1665289517487478), ('taxamnesty', -0.14536296710718097), ('perintah', -0.06186498626739316), ('pajak', -0.0600217549403277), ('dasar', -0.056141557293615516)], [('rakyat', 0.06234626967799217), ('bongkarpandorapapers', 0.030380661647205598), ('taxamnesty', -0.025632604841861216), ('harap', -0.02321487255942592), ('bayar', -0.015735126493050443), ('catat', 0.015608224702380212), ('perintah', -0.014707686280211237), ('pandorapapers', 0.01462017999361688), ('nya', 0.011551848721450955), ('kali', 0.011180466659350633), ('pajak', 0.0017436070860399866), ('menteri', 0.0006015497477813172)], [('kemplang', 0.1593376589968009), ('ruuhppuntukmasyarakatkecil', -0.142927791350822), ('masyarakat', -0.0651190455057451), ('ruuhpp', -0.05874655185171059), ('teriak', 0.046850300008497894), ('pasti', 0.037743632576201835), ('alas', 0.03145330856367665), ('bela', 0.02640314090941911), ('tolak', 0.023978920574886237), ('taxamnesty', -0.012715828088706494), ('pajak', 0.012598405535728014)], [('investasi', -0.16480031028624234), ('potensi', -0.16372534572598393), ('sbn', -0.11284248870841598), ('taxamnesty', -0.1012165033827319), ('jilidii', -0.06578132032168646)], [('taxamnesty', -0.18775876993052854), ('rakyat', 0.03387495339186551), ('calo', 0.020866260894676615), ('puas', 0.01748020477262298), ('arang', 0.0170198682683762), ('gataunya', 0.015898886589262248), ('lo', 0.013159445945177022), ('suruh', 0.006454004033151794)], [('koar', 0.0446647399166676), ('kroni', 0.03681269060028537), ('kali', 0.03625996045243538), ('akibat', 0.03228800644603977), ('taxamnesty', -0.03118903458671291), ('kaya', 0.01636875103894264)], [('cuci', 0.03538318384319189), ('via', 0.0286968314337958), ('pandora', 0.024847130484146197), ('jokowi', 0.024070864436609368), ('taxamnesty', -0.020842819437033783), ('duit', 0.013357476061919213), ('untung', 0.005297921289616926)], [('jabat', 0.03571107729105963), ('bikin', 0.03549808015791631), ('pandora', 0.028577152184432113), ('jilid', -0.022086974570365694), ('papers', 0.018197754275286842), ('eeeh', 0.016281375997414763), ('menkeu', -0.016150202013377506), ('gencar', 0.01398091546917026), ('taxamnesty', -0.01179400078051657), ('masuk', 0.010643166713149117), ('gimana', 0.010279520843971518), ('daftar', 0.0017576386566288684), ('program', 0.0005060600483828045)], [('kemplang', 0.06389774265949533), ('taxamnesty', -0.05869288951672452), ('cangkang', 0.05236414413976799), ('rakyat', 0.04641514532118761), ('jabat', 0.0368062791746729), ('usaha', 0.022502387241438156), ('milik', 0.020139554178590483), ('pajak', -0.011339435993307338)], [('masalah', 0.045711983884120594), ('dasar', 0.044439273732810806), ('gak', 0.04247306640199897), ('tuju', 0.036057456488402105), ('wujud', 0.03222917534774383), ('taxamnesty', -0.03056516597641803), ('ruuhppuntukmasyarakatkecil', -0.023475781187798677), ('ruuhpp', 0.02071708666465724), ('adil', -0.018271614903935064), ('taat', 0.017827903994166786), ('paja', 0.014979143090906585), ('sistem', 0.009042603939157998), ('pajak', -0.004958499770567765), ('masyarakat', -0.0016285588118167668)], [('rakyat', 0.05652614889931296), ('patuh', -0.04551701522889414), ('bikin', 0.0315664022732492), ('menkeu', -0.029306949201010242), ('ngemplang', 0.019625753697311947), ('hehehe', 0.014824906141117996), ('capek', 0.014365914321657736), ('situ', 0.011971238270888042), ('taxamnesty', -0.01190031157233645), ('paksa', 0.010717774374885587)], [('kaya', 0.031930134095104194), ('gapapa', -0.0207635419115264), ('program', -0.020301064886878745), ('bilang', -0.013570102734299948), ('negara', 0.010762256706080619), ('suka', 0.009876637135871036), ('pajak', -0.009711392871367821), ('heaven', 0.009159217504534497), ('warga', 0.00896499159610616), ('oy', 0.008569607332054148), ('ngehindarin', 0.008469550927752407), ('naro', 0.007393300987763414), ('banyak', 0.00675462398022184), ('kalo', 0.0048442576256443265), ('bangun', -0.004624009379127276), ('uang', 0.004400204023749411), ('tax', 0.0041846179473382155), ('udah', 0.004161960570144564), ('orang', -0.0023847566926128204), ('elu', 0.001562801881181596), ('taxamnesty', -0.0007399485449959115), ('gak', 0.00027243581681284204)], [('selamat', -0.05368921442755328), ('jalan', -0.037212043817612506), ('taxamnesty', 0.033483001903131575), ('jabat', 0.032647449602549776), ('panama', 0.010319344413627168), ('ah', 0.00992955323955623), ('paper', 0.009579418695105828), ('dahulu', 0.008282892798635559), ('muka', 0.007524833323562684), ('ii', 0.0053051854144871395), ('lbp', 0.0034081260025723772), ('i', -0.0017308203550636294), ('ta', -0.0013702089989851233), ('hasil', -0.0007551472833437255)], [('rupiah', -0.2614918950355469), ('sentimen', -0.23320744206911975), ('kuat', -0.22466998897222648), ('taxamnesty', -0.07012296287129426), ('positif', -0.04432604722905716), ('picu', -0.023275527969339613), ('ii', 0.0031052275125280785)], [('tolak', 0.08209920580426792), ('pks', 0.03755911062993754), ('taxamnesty', -0.0358954023886671), ('ppn', -0.030793444567169634), ('kup', 0.02689021662055895), ('ruu', 0.026807156125287928), ('alas', 0.015088424545861949)], [('masyarakat', -0.2048032311724037), ('ruuhpp', -0.15687615953303838)], [('rupiah', -0.22354993035383103), ('sentimen', -0.20402527522214559), ('kuat', -0.1550385610135268), ('dorong', -0.13605147801677284), ('taxamnesty', -0.04785845570636239), ('jangka', -0.028836138074079785), ('solid', -0.021294854045421804), ('pt', -0.02105086582392397), ('gold', -0.01780486024534355)], [('rupiah', -0.2220348644859355), ('sentimen', -0.2003849187661081), ('kuat', -0.1804983367858839), ('dorong', -0.13522472873601732), ('taxamnesty', -0.0631932701070504), ('jangka', -0.022683487740219548), ('gold', -0.020151929355130063), ('solid', -0.019761943130770614)], [('jilid', -0.23078681604269174), ('mudah', -0.09139896388257542), ('taxamnesty', -0.022984260827847994), ('pa', 0.013314304111831505), ('walo', 0.013304462170873659), ('putih', 0.013104739302635029), ('gakan', 0.013022658331385564)], [('kaya', 0.059268871759997496), ('pandemi', -0.040080124204054046), ('taxamnesty', -0.03597710475786198), ('jilid', -0.029455212633506277), ('koar', 0.024781820066070867), ('meng', 0.021740975628071462), ('kan', 0.018940182685503078), ('akibat', 0.018924391982430473), ('ilegal', 0.01829415167754767), ('legal', 0.016877419004142344), ('kroni', 0.015829144080312024), ('duga', 0.01356681436809115), ('tambah', 0.013271296587635885), ('kali', 0.012836014474231851), ('dapat', 0.0010010554789118802)], [('taxamnesty', -0.08174099815460123), ('kaya', 0.04394052953100917), ('nyata', 0.04246345986850385), ('konglo', 0.020425149319577587), ('program', -0.012127112466224789)], [('rakyat', 0.05904769526075059), ('npwp', -0.03737013478879414), ('lapor', 0.012840358762588657), ('pajak', 0.009046182110459891), ('bayar', -0.007898432967219187), ('bangga', 0.0068282591908112485), ('tuk', 0.0064720496606514525), ('lucu', 0.006011345062795227), ('sedikit', 0.00592722801710223), ('konglo', -0.005864772231799859), ('mesti', 0.005680159904033188), ('sentuh', 0.005664781146235257), ('uber', 0.005043230702961701), ('repot', 0.004235022229101703), ('taxamnesty', 0.0035052821872236005)], [('menkeu', -0.03441119978011535), ('bikin', 0.0293762834310503), ('pandora', 0.027115804543402615), ('jilid', -0.02334766436464693), ('jabat', 0.022138585871873413), ('papers', 0.011736085051646927), ('bongkarpandorapapers', 0.004834452364486802), ('gimana', 0.0040266077706101345), ('fakta', -0.003563195868379843), ('daftar', -0.003153530140522259), ('taxamnesty', -0.0028961739675767155), ('eeeh', 0.0027023957987802145), ('gencar', 0.002697339471677258), ('asa', 0.0023616514914933575), ('tau', 0.0018555792498129461), ('masuk', 0.0017455610654264556), ('tuh', 8.93884800431048e-05), ('program', -8.455204236848636e-06)], [('rupiah', -0.23372202365740652), ('rp', -0.19307161302158538), ('positif', -0.05065399660045725), ('as', -0.03684691217845827), ('dolar', -0.03102232589314095), ('taxamnesty', -0.026999811831158006), ('bukanberitabiasa', 0.009414304228619177), ('otot', 0.00873118348028131), ('sambut', 0.0074907831724825335), ('sindonews', 0.006978681865374469)], [('taxamnesty', -0.08037400881395844), ('rasio', 0.06026461733349308), ('tingkat', -0.053901528390550735), ('ekonom', 0.04009553975415473), ('efektif', 0.018613085485128196), ('pajak', -0.00921706498506192), ('jilidii', -0.008511521682510995)], [('nik', -0.07150205057536536), ('naik', -0.03280866779699573), ('taxamnesty', 0.02685128812425556), ('cukong', 0.0221299067284688), ('npwp', -0.0206417184025406), ('ngap', 0.01658153155557444), ('triliun', -0.016003798447415146), ('gak', 0.015607103717495524), ('hahaha', 0.01507698680312003), ('bilang', -0.013972086915252108), ('nunggak', 0.012824030961747414), ('pajak', -0.012723987718416709), ('bansos', 0.010365129608644797), ('warga', 0.009532962633440535), ('berisik', 0.007718333899199102), ('stnk', 0.00761734059307806), ('miskin', 0.007582347755193108), ('udah', 0.007155555833318419), ('ngakak', 0.006787036102614091), ('an', 0.006152038491987121), ('orang', -0.005016341665658288), ('bodoh', 0.004398032069526104), ('pkh', 0.0023050096547995278), ('k', 8.804595530804052e-05)], [('daftar', -0.02462172781108648), ('rezim', 0.010558173300099819), ('sedia', 0.0056081898403530775), ('duluan', 0.003594263812001682), ('pandorapapers', 0.003594231911137466), ('bajing', 0.002909776080746855), ('karpet', 0.0023167239877587846), ('merah', 0.0009912348043590983)], [('ros', 0.04831083154370403), ('ambil', 0.016343011660725874), ('taxamnesty', -0.015818522069795244), ('rakyat', 0.01562855453935896), ('cari', 0.014571925960336577), ('jabat', 0.014047219673750909), ('usaha', 0.009181173282094751), ('celah', -0.008878061124247898), ('pajak', -0.0033541281176127646)], [('pks', 0.025202152643926508), ('ampun', 0.024928102170122056), ('jilidii', -0.024286390498006242), ('taxamnesty', -0.02397007992307819), ('asset', 0.018923797280483257), ('ahli', 0.01807241106423492), ('publik', 0.017840910134668375), ('voluntary', 0.017710109368720917), ('disclosure', 0.017163019416464576), ('tolak', 0.013082981588801194), ('pajak', 0.005437554672754178)], [('rakyat', 0.024676979084263476), ('perintah', -0.016372709917587658), ('pajak', -0.012441563597402797), ('investor', 0.009243305952023335), ('jerat', 0.007396925153544002), ('cina', 0.006821234734443107)], [('karbon', -0.15511313565461302), ('ruuhpp', -0.0860790218037919), ('taxamnesty', -0.07719632580623696), ('pks', 0.07291944753473904), ('tolak', 0.04937532237425842), ('pajak', -0.03883564670565871)], [('rupiah', -0.23266161345017106), ('kuat', -0.19266857819680408), ('sentimen', -0.18972963695941805), ('tutup', -0.1328825724142256), ('taxamnesty', -0.07734066053537073), ('dukung', -0.007054422779600919), ('jilidii', 0.00021170279961635306)], [('rakyat', 0.07084500586421143), ('selamat', -0.03772017794776306), ('nikmat', -0.02419749253032578), ('ha', 0.022994483947389954), ('jilid', -0.01489028478181965), ('eh', 0.00837714862088599), ('ngemplang', 0.00772552612727106), ('mo', 0.006314373553687635), ('taxamnesty', 0.006228490525140341), ('uber', 0.006116408369231304), ('do', 0.006115522861816138), ('sj', 0.005769745238024452), ('naikin', 0.005617701082252435), ('bobrok', 0.005286353153780436), ('pajak', -0.005233618735343128), ('negri', 0.004746354233550553), ('suruh', 0.004459592146524103), ('i', -0.00041688987186936044), ('udah', -0.00019471798177466933), ('bayar', -0.00012288453315854634)], [('jalan', -0.05161436662509602), ('uang', 0.02825505341136757), ('cuci', 0.01955720880688449), ('ye', 0.013812162142624708), ('hasil', -0.013091528607167401), ('hancur', 0.010989059343386611), ('sulit', -0.008978877390606228), ('ekonomi', 0.00755086790825229), ('korupsi', 0.007331383516561438), ('taxamnesty', 0.006805717670587189), ('tiada', 0.002258329252587053)], [('laksana', -0.07169518534794463), ('ruuhpp', -0.0340274641015049), ('instrumen', 0.011243539937435603), ('rugi', 0.010277364922886608), ('taxamnesty', 0.01008700059256674), ('nilai', 0.009804051613305794), ('kontra', 0.009612921247498319), ('diskusi', 0.009079509671090917), ('rencana', -0.008147152541621102), ('kait', 0.007991741540473898), ('judul', 0.007446137141670099), ('publik', 0.006955478369172259), ('takar', 0.006852873927079454), ('indef', 0.005423115518659737), ('muncul', 0.00514231748858823), ('untung', -0.003861455960583402), ('pro', 0.0036427750298657483), ('bahas', 0.0035301869141361767), ('tambah', 0.0023305811289954775), ('pajak', 0.0022379056229230596), ('ppn', 0.0021654915999855745)], [('jabat', 0.01004581953875984), ('taxamnesty', -0.007975510190044813), ('pandora', 0.007538831123233354), ('kemntrian', 0.006572389492067461), ('jokowi', 0.005110390802560772), ('daptr', 0.004800614309823502), ('pa', 0.004432363644413583), ('batal', 0.004215753020094587), ('papers', 0.003985519886227688), ('masuk', 0.0030237529371750305), ('dpr', -0.0014567700242186909)], [('wajib', -0.23155201900033792), ('taxamnesty', -0.16347612071161466), ('tingkat', -0.11663515505111122), ('patuh', -0.10567572056896117), ('konsep', -0.10185189097863279), ('fitra', -0.0860405186453933), ('program', -0.04864848075221911), ('pajak', -0.027943063346690934), ('bijak', -0.011916430166017884)], [('indonesia', -0.17414760256932632), ('termasu', -0.13449519499746465), ('negeri', -0.09173112298097956), ('taxamnesty', -0.08778932111092537), ('program', -0.07183155065425016), ('simpan', -0.06951875707609649), ('kaya', -0.04564471732492514), ('usaha', -0.04020403651795557)], [('indonesia', -0.06776008658196205), ('swiss', 0.050721234718524084), ('bank', 0.045066598472679194), ('simpan', -0.04036445157488508), ('program', -0.03293510404641966), ('miliader', 0.019651480536211626), ('ungkap', 0.018385135151028513), ('lokasi', 0.01814460281043438), ('favorit', 0.016419854700703146), ('negeri', -0.015885219039582583), ('taxamnesty', -0.015142299944733408), ('timbun', 0.014749870404729989), ('usaha', -0.007819678162225934), ('kaya', 0.004781829581876756)], [('periksa', 0.022419219623587366), ('kayak', 0.019765625334671747), ('panama', 0.019497398270627714), ('pesimis', 0.01933806514186824), ('taxamnesty', -0.01793327998018117), ('lolos', 0.016596057886717728), ('papers', 0.012169687216884354)], [('rupiah', -0.24495306784978788), ('kuat', -0.2112895854304812), ('sentimen', -0.1959415970407994), ('tutup', -0.12176095214764326), ('taxamnesty', -0.07773408384655256), ('dukung', -0.006581787677648022), ('ii', 0.0063293625074243584)], [('rupiah', -0.2560842944577444), ('sentimen', -0.20585213547210648), ('kuat', -0.2022963183010146), ('tutup', -0.14672378241055206), ('taxamnesty', -0.05587593317837352), ('barat', 0.011936285251676977), ('kurs', 0.010422152426402985), ('news', 0.010104037242913854), ('jawa', 0.008955402179764326), ('dukung', -0.006936241468731221), ('jilidii', -0.005160398505030143)], [('sentimen', -0.2374974309595448), ('positif', -0.09086107944850706), ('laku', -0.07093959324173338), ('jangka', -0.06019249067882881), ('taxamnesty', -0.05594343846789314), ('pt', -0.03754274166603886), ('garuda', -0.032706037255231454), ('pasar', -0.03175548477581629), ('ibrahim', -0.031009589032760677), ('trfx', -0.030611647316220406), ('rupiahhariini', -0.030489427549022983), ('jilidii', -0.02667423614578138), ('tangkap', -0.025568908387954763), ('assuaibi', -0.0245385528590661), ('bawa', -0.008143489641334668), ('direktur', -0.0031473260867341396)], [('rupiah', -0.21166752200915182), ('kuat', -0.17107776997455715), ('tutup', -0.13952497735972097), ('rp', -0.11731007523416823), ('level', -0.047630469733964204), ('dolar', -0.0439279670705706), ('as', -0.04143656297749065), ('dagang', -0.020983772055497608), ('nilai', 0.018688920005947383), ('tukar', -0.018057245494455962), ('selasa', -0.01193648468640722), ('pon', -0.011607279457256812)], [('wajib', -0.11103976320512118), ('tri', 0.07252190370955588), ('ampun', 0.0666736736577427), ('via', 0.061668095744781985), ('negatif', 0.06011447948623085), ('taxamnesty', -0.05200904156546127), ('uang', 0.0510182874053041), ('bayar', -0.032775738777819655), ('pajak', -0.022083459766167792), ('negara', 0.007190618033424625)], [('rupiah', -0.22853703645562015), ('kuat', -0.1933719749430347), ('sentimen', -0.1832600415371958), ('tutup', -0.133098661013042), ('taxamnesty', -0.07432133784711345), ('analis', -0.018469537066035862), ('jilidii', -0.002547285350873716)], [('rupiah', -0.22245415673488306), ('kuat', -0.1837504408998943), ('tutup', -0.14509015370961684), ('rp', -0.13662384823770468), ('as', -0.04735296550085024), ('dolar', -0.044920555321448064), ('kursrupiah', -0.016493257504834824), ('poin', -0.012456217886763574), ('posisi', -0.012053031264132843), ('persen', -0.009625669565948855)], [('rakyat', 0.017791751203137262), ('pajak', 0.012794626718905066), ('hpp', -0.011437271064750822), ('kaya', 0.008507076889516905), ('pr', 0.00619480465646374), ('smeentara', 0.006121292767433519), ('ganti', 0.0058343313178536), ('nama', 0.004842502126156925), ('kup', 0.004701718495407427), ('ni', 0.004259981723540406), ('gimana', 0.004077664533688579), ('ruu', 0.004030059766961054), ('sedih', 0.0038990142146151396), ('beban', -0.0028885590266126164), ('orang', -0.00193685815772239), ('gak', 0.001466124100692536)], [('tuju', -0.07836986837525928), ('kemplang', 0.05379651054789215), ('rakyat', 0.03536140996756828), ('taxamnesty', -0.020828218798177065), ('aja', 0.01869212946463993), ('hasil', -0.012641949692602693), ('tipu', 0.0077102270449067355), ('sapu', 0.007378874654167182), ('repatriasi', 0.007362538106201893), ('bumn', 0.006309346337308987), ('tabung', 0.005705202197283376), ('jiwasraya', 0.004820316295973598), ('ga', -0.004457728881685307), ('bahas', 0.004326131981515365), ('asuransi', -0.0036858032607617226), ('duit', 0.0029746351364042713), ('bri', 0.0013610274055979906), ('stafsus', 0.0012546586674188588), ('perintah', -0.0012068006816597047)], [('rupiah', -0.21778338292232552), ('sentimen', -0.19623815065739983), ('kuat', -0.1766295294011438), ('dorong', -0.11600599523399605), ('taxamnesty', -0.10034075201630206)], [('nikmat', -0.026864874124443647), ('kaya', 0.020231248754299692), ('utang', 0.015191692612831108), ('jabat', 0.00923186577365867), ('kalang', 0.007862741318266498), ('dib', 0.00766340917767949), ('negara', 0.007482202833476856), ('anomali', 0.007410939543304845), ('nama', 0.00722340606380338), ('rezim', 0.007144635876042284), ('logika', 0.0066360925973319), ('elit', 0.005569309321737047), ('orang', -0.0028464346084950305), ('duit', -2.5757246454961065e-05)], [('kemplang', 0.09091855091658621), ('ehh', 0.06630444777489243), ('capek', 0.06454063116215783), ('taxamnesty', -0.06180215400613474), ('taat', -0.057033844305852885), ('kasih', 0.04426625081516695), ('pajak', -0.014303091763320877)], [('rakyat', 0.07425173368511213), ('bebas', -0.0624632335768325), ('kemplang', 0.05225977342425197), ('kaya', 0.030698002874750947), ('legal', 0.015710904747203238), ('investor', 0.014862288196138348), ('ppnbm', 0.013775673478305129), ('banyak', 0.012985735410033606), ('jerat', 0.012189559612616687), ('koruptor', 0.012163008975416017), ('cina', 0.008849677025289339), ('jilid', -0.007402709970845908), ('perintah', -0.006024172741979915), ('pajak', 0.002796654761794616), ('taxamnesty', -0.002652593285030771)], [('jilid', -0.05228329336529338), ('eh', 0.0230833159684083), ('basi', 0.020980336829458292), ('pilpres', 0.020689242825151104), ('konfirmasi', 0.01977713566185168), ('kalo', 0.018819654681869354), ('berita', 0.017884787054345434), ('hoax', 0.017506843587664044), ('anggap', 0.013791033469125146), ('taxamnesty', -0.0062725301738122425)], [('pantesan', 0.029556208398630577), ('aktif', 0.027315250512430323), ('luhut', 0.024712036965234437), ('taxamnesty', -0.02097839507562105), ('dpr', 0.014435661201578832), ('bahas', 0.012269657947568662)], [('wajib', -0.084631646628305), ('ampun', 0.04960424770383408), ('bayar', -0.045718657611278704), ('kaya', 0.04045543436477149), ('taxamnesty', -0.032226547984480665), ('konglomerat', 0.02970984498500103), ('uang', 0.0284776469297021), ('tri', 0.02054753933509947), ('via', 0.013797715474963725), ('negatif', 0.011370529219401373), ('sok', 0.010734413472443211), ('mikir', 0.00845051463474823), ('enak', 0.007634232606922062), ('tuh', 0.007000045333807564), ('bebas', -0.005646266694643863), ('bodoh', 0.004414194698027836), ('negara', -0.003931055501021046), ('pajak', -0.0031937483283549927)], [('karbon', -0.1599076307171984), ('ruuhpp', -0.0896743642271392), ('taxamnesty', -0.07544797536235813), ('pks', 0.07007067921570043), ('tolak', 0.048098507629079995), ('pajak', -0.03685320708859841)], [('nilai', 0.07213550212711442), ('mundur', 0.05835512676757691), ('tingkat', -0.04225666340416997), ('langkah', 0.03824235101788514), ('taxamnesty', -0.03408643004783262), ('patuh', -0.03143414205700964), ('jilidii', 0.008405961836222664), ('pajak', -0.00016647392124406052)], [('bebas', -0.07134195639495963), ('usaha', 0.06509046662333427), ('jabat', 0.059564381065546006), ('indonesia', -0.05263721057177265), ('uang', 0.037866822249185736), ('banyak', 0.033626843491177366), ('cangkang', 0.03244839356163871), ('masuk', 0.02865608974676292), ('negara', 0.02650375899951316), ('kait', 0.021375110545482294), ('pajak', -0.01823397669629836), ('moga', -0.006198107663853899), ('taxamnesty', 0.0035474820575960714)], [('jilid', -0.047086171680746754), ('bebas', -0.04324491111475811), ('pajak', 0.031090071539464696), ('bikin', 0.017530259408126542), ('banget', 0.010674930015602794), ('taxamnesty', 0.008900389658011846), ('anjir', 0.008299723804615956), ('telek', 0.00823267880753922), ('atuh', 0.008094700232334584), ('danzo', 0.007828796956065975), ('perusahaa', 0.007784951531095319), ('yahh', 0.007631484737275137), ('duh', 0.005686073569504286), ('sadar', 0.004703185938049976)], [('ga', -0.029272414221681052), ('pandora', 0.017921589285653725), ('suudzon', 0.013260040820741597), ('sm', 0.012712915270015352), ('jlid', 0.012694050670726919), ('bagent', 0.011044904397019087), ('berita', 0.008147457008697793), ('susah', 0.00799521152114013), ('papers', 0.007988667673032118), ('taxamnesty', -0.002143838872266755), ('ii', 0.0013781175788078173)], [('lembaga', -0.019690716171937184), ('hasil', -0.01844708197126506), ('ln', -0.016023439076268742), ('rezim', 0.01290320870278892), ('jabat', 0.012385326181226203), ('uang', 0.012280495661240148), ('program', -0.01021978417614222), ('korupsi', 0.006867725604394115), ('ungkap', 0.005700063571614719), ('menyimoan', 0.005546236047820106), ('berani', 0.005402428679295119), ('perintah', -0.005200432653279658), ('orang', -0.004156175132100195), ('lawan', 0.0036180151813804827), ('lokal', 0.003298045010209114), ('ngajak', 0.0032922330512518195), ('indikasi', 0.002915903207584311), ('buruk', 0.0026122917129062285), ('aman', -0.0016781610979897496), ('taxamnesty', -0.0007439349661727163), ('ada', 0.0003153473207613019), ('upaya', 0.00012519904843378294)], [('curigation', 0.0927893179146349), ('cuci', 0.08224669181708177), ('duit', 0.06591060374672127), ('taxamnesty', -0.05401420528053807)], [('tinggal', -0.013218596042356916), ('kalo', 0.008685244013690537), ('tar', 0.0068074774567062875), ('bongkar', 0.006697122424737596), ('the', 0.0049013999153823), ('gin', 0.004541213429380152), ('trus', 0.004350210144141467), ('to', 0.00431397074722735), ('taxamnesty', 0.003910148644514529), ('li', 0.003759622731496669), ('cik', 0.0036636343171995564)], [('ya', 0.052888671890247), ('mw', 0.0415406495067853), ('nyimpen', 0.04044786807378943), ('apagunanya', 0.03887930671388642), ('jilid', -0.02586187064045628), ('negeri', -0.017753281114759885), ('mah', 0.017018878128837994), ('duit', 0.015807208044140313), ('usaha', 0.01124135888312982), ('ga', -0.008488325840613596), ('taxamnesty', -0.005027730453534829)], [('pph', -0.14801240297651272), ('tarif', -0.09179618510914204), ('final', -0.08721158198826198), ('manfaat', -0.08159329594922698), ('ketok', -0.0712523429135918), ('denga', -0.0641319385160866), ('rendah', -0.05381526691777199), ('taxamnesty', -0.047252350910996214), ('lumayan', -0.042426506992792636), ('lupa', -0.042372575672957366), ('dpr', -0.027665595927268378), ('kena', -0.021160042351735985), ('bahas', -0.020781830048406103)], [('paja', -0.09496712987660622), ('pks', 0.040980527491018756), ('tolak', 0.02601268525702407), ('voluntary', 0.02249931176962224), ('disclosure', 0.01862372880125503), ('asset', 0.01551766034696555), ('ampun', 0.014969507053205348), ('ruu', 0.011820336713770805), ('publik', 0.011455188946001555), ('kup', 0.01079321216822672), ('atur', -0.00976718948262151), ('tolakruukup', 0.00829347399555909), ('pajak', 0.006322759627929014), ('ahli', 0.006234181212242625), ('taxamnesty', 0.006226420006663054), ('jilidii', 0.00595915189944774), ('hpp', 0.0017999217189754821), ('harmonisasi', 0.00029491988466693036)], [('tuju', -0.1530438541423051), ('rakyat', 0.05695613483346075), ('taxamnesty', -0.037571843456741846), ('hutang', -0.01607648548279009), ('bikin', 0.013887120715071581), ('pajak', -0.010163987428419938), ('ajar', 0.007114432478477626), ('total', 0.006951516950562552), ('retail', 0.00662555930095729), ('gagal', 0.0058631854368435985), ('invest', 0.005357940886622852), ('kejar', 0.004239242456854572), ('horang', 0.003890668227077696), ('pintar', 0.0038723752985446573), ('kayah', 0.003786084533909414), ('surat', -0.002164449399346118), ('orang', 0.0020139218980653436)], [('taxamnesty', -0.04373931921348082), ('pks', 0.03730875892014134), ('tolakruukup', 0.037047558025937684), ('ruu', 0.028869804982950505), ('tolak', 0.02228506811905587), ('jilidii', -0.017687488737660993), ('harmonisasi', 0.011033036430029538), ('atur', -0.01014443060733623), ('pajak', 0.005618562336617977)], [('ampun', 0.051476210572707126), ('kritik', 0.03426734723667216), ('tuai', 0.032826596171442804), ('wacana', 0.031684830239380735), ('terap', -0.028244804323403507), ('perintah', 0.021898953320599376), ('taxamnesty', -0.02177693088974314), ('pajak', 0.006789010393829793), ('jilidii', -0.006561320441262632)], [('kaya', 0.02393093791594971), ('dasar', -0.013944538986396231), ('nikmat', -0.012255401645861062), ('taxamnesty', -0.01032552988040384), ('utang', 0.009763171186722555), ('duit', -0.008526684484560334), ('orang', -0.0064465108028228585), ('jabat', 0.005997787428847077), ('miskin', 0.004322033263217103), ('nama', 0.004201600132279003), ('rezim', 0.0033844508034675974), ('si', 0.0028176371910721364), ('subsidi', 0.002510400213459095), ('elak', 0.00225635168636422), ('kejar', 0.0021120697171559642), ('negara', 0.0020232453731959497), ('elit', 0.001964098581450161), ('kalang', 0.0018679596122687851), ('anomali', 0.0018453407972284512), ('kata', 0.0016064573862267983), ('kuantitas', 0.0013128351280443783), ('receh', 0.0011794942849178384), ('sembako', 0.0011485267309160232), ('logika', 0.0007299965280335056), ('pajak', 7.656892694545262e-05)], [('terap', -0.05475452137356067), ('ampun', 0.03291634542641024), ('kritik', 0.022876594396043132), ('tuai', 0.013530330883377514), ('wacana', 0.012715554272522965), ('pajak', 0.012597451656710246), ('jilidii', -0.007373483957974865), ('taxamnesty', -0.0066252448388857835), ('sebut', 0.004460569789586606), ('konsisten', 0.004370589113161824), ('perintah', -0.004360674994110067), ('amat', 0.003195438418415413)], [('ampun', 0.04957610634478392), ('kritik', 0.034809090380438876), ('tuai', 0.03329527531773871), ('terap', -0.029475526918875308), ('wacana', 0.029128713108867665), ('taxamnesty', -0.024058636905059022), ('perintah', 0.023623589948836093), ('pajak', 0.007539883252927066), ('jilidii', -0.007441035562212477)], [('rezim', 0.007754511567450698), ('kuasa', -0.00671076417657953), ('buka', 0.006286901195065868), ('blingsatan', 0.004479743582807479), ('mas', 0.003971813623560108), ('taxamnesty', 0.003771737263100126), ('main', 0.003759721123372005), ('lord', 0.0032024127251193807), ('keran', 0.002209558187349325), ('tabir', 0.0021298491025959704), ('sah', 0.0019541234437735534), ('mafia', 0.0015089848448204919), ('jahat', -0.0004923735209500012)], [('harta', -0.03491215046485336), ('sembunyi', -0.02950522168429625), ('pilih', -0.019521118718531835), ('celah', -0.017025396759586555), ('pandora', 0.015393196926866156), ('oligarki', 0.014887976300697676), ('papers', 0.012673298276451807), ('taxamnesty', 0.008493902845870187), ('buka', 0.006394000250562761), ('sisi', 0.005877522316791939), ('ketat', 0.005158381836296456), ('wacana', 0.0039015973442177326), ('pajak', 0.0038240595320307336), ('regulasi', 0.0031209732172523693), ('move', 0.00266259262051783), ('kelas', 0.0023833488572385115), ('warga', 0.002316334190251711), ('brilliant', 0.0022322519580712146), ('menteri', 0.0020529363284180284), ('tengah', 0.0019743827521959165)], [('pks', 0.018965503069772398), ('taxamnesty', -0.018705224307327743), ('kam', 0.012246827942768778), ('disclosure', 0.010964322109236488), ('ahli', 0.010764110021247334), ('asset', 0.010291869269406425), ('voluntary', 0.009928033848186626), ('publik', 0.008215552622421685), ('tolak', 0.0075410009769902525)], [('pks', 0.057245372777296105), ('objek', 0.052051220112103065), ('tolak', 0.047854027088839145), ('taxamnesty', -0.04666872801456242), ('luas', -0.024419414406030737), ('ppn', -0.01998631338397119)], [('tolak', 0.0046252075542099575), ('ahli', 0.003242511720052904), ('publik', -0.002111617096743329), ('voluntary', 0.001898898888473367), ('asset', 0.0012354897174425205), ('disclosure', -0.0001424388929736321)], [('ngapain', -0.038696265413354934), ('taxamnesty', -0.022460335235803076), ('pro', 0.02107563406546298), ('xx', 0.020952889605364077), ('nunggu', 0.020238529932379404), ('hahaha', 0.02009462094179244), ('be', 0.018972134614718326), ('like', 0.018515167810668576), ('max', 0.017734655112008814), ('jilid', -0.016870015973877505), ('tunggak', 0.013140183554654449), ('pajak', 0.010364443753460406), ('bayar', -0.00029684282112725345)], [('pks', 0.02123046671084826), ('jilidii', -0.021055412302705544), ('ampun', 0.02024603877582563), ('taxamnesty', -0.016370907788268323), ('tolakruukup', 0.014599764075395953), ('asset', 0.013529932846241237), ('ahli', 0.012914848259151252), ('publik', 0.012765648331462213), ('voluntary', 0.012385700855466749), ('disclosure', 0.011941029405504526), ('tolak', 0.008972305482887627), ('pajak', 0.008896750385627306)], [('imbang', -0.0890307235965333), ('tolak', 0.0772929033864927), ('jilid', -0.06990409621975457), ('persib', 0.030737551711641005), ('main', 0.022997467927562777), ('kup', 0.020695257260652723), ('ruu', 0.018631462367303335), ('taxamnesty', -0.00041931880658752465)], [('taxamnesty', -0.0961818620008705), ('kemplang', 0.09613935775027828), ('kabar', 0.07035398763263243), ('wujud', 0.04219407880599535), ('pajak', -0.02899357491770103), ('jilidii', 0.0017332798338453225)]")

In [ ]:
a = input.split(",")
print(len(a))

In [ ]:
text = []
angka = []

for i in range(len(a)):
    if i % 2 == 0:
        text.append(a[i])
    else :
        angka.append(a[i])

In [ ]:
data = {"Kata_0" : text, "Bobot_0" : angka}
Tabel_Pengaruh = pd.DataFrame(data)
print(Tabel_Pengaruh.head())

In [ ]:
def remove(tweet):
    tweet = re.sub(r'\[', '', tweet)
    tweet = re.sub(r'\(', '', tweet)
    tweet = re.sub(r"\'", '', tweet)
    return tweet
Tabel_Pengaruh['Kata'] = Tabel_Pengaruh['Kata_0'].apply(lambda x: remove(x))

In [ ]:
def remove(tweet):
    tweet = re.sub(r'\)', '', tweet)
    tweet = re.sub(r'\]', '', tweet)
    return tweet
Tabel_Pengaruh['Pengaruh_jadi_Negative'] = Tabel_Pengaruh['Bobot_0'].apply(lambda x: remove(x))

In [ ]:
Tabel_Pengaruh = Tabel_Pengaruh.drop(Tabel_Pengaruh.columns[[0,1]], axis = 1)
Tabel_Pengaruh.head()

In [ ]:
Tabel_Pengaruh.to_csv('data_pengaruh.csv',encoding='utf8', index=False, sep = ';')
Tabel_Pengaruh.head()

In [ ]:
print(len(Tabel_Pengaruh['Kata']))
print(len(Tabel_Pengaruh['Pengaruh_jadi_Negative']))